## Paper 1 - eating behavior

### Data preparation

#### Define the relevant directories used in this paper

Data is pulled from the standardized data folder; subsequently, it is stored and managed in the paper 1 folder. 

In [1]:
import os

# Define the source and output directories
source_directory = r"C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\DB2_standard"
paper1_directory = r"C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional"

# Ensure the output directory exists
os.makedirs(paper1_directory, exist_ok=True)

#### Create a research question-specific SQL database subset 

Check those medical records where any/3+/all emotional values are available, and filter the database to contain only the specified patients and medical records. Save the data to 3 new SQL files - one with any, one with some, one with all values available. For research purposes, the last one is most likely to be used. The first two may be relevant if trying to increase the sample size for one or a few specific emotional values. 

In [5]:
import sqlite3
import pandas as pd
import os

# Use the above defined directories
db_path = os.path.join(source_directory, "pnk_db2_colclean.sqlite")
conn = sqlite3.connect(db_path)

# List all tables in the database
query_tables = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query_tables, conn)
table_names = tables['name'].tolist()

# Define criteria for filtering for any/3+/all emotional values available
def create_filtered_database(criteria, output_filename):
    # Set up the appropriate query based on the criteria for the three scenarios
    if criteria == "any":
        # to select records where at least one emotional variable is not null
        query = """
        SELECT medical_record_id, patient_id
        FROM medical_records_colclean
        WHERE hunger IS NOT NULL
           OR satiety IS NOT NULL
           OR emotional_eating IS NOT NULL
           OR emotional_eating_value IS NOT NULL
           OR quantity_control IS NOT NULL
           OR impulse_control IS NOT NULL;
        """
    elif criteria == "3plus":
        # to select records where at least three emotional variables are not null
        query = """
        SELECT medical_record_id, patient_id
        FROM medical_records_colclean
        WHERE (CASE WHEN hunger IS NOT NULL THEN 1 ELSE 0 END +
               CASE WHEN satiety IS NOT NULL THEN 1 ELSE 0 END +
               CASE WHEN emotional_eating IS NOT NULL THEN 1 ELSE 0 END +
               CASE WHEN emotional_eating_value IS NOT NULL THEN 1 ELSE 0 END +
               CASE WHEN quantity_control IS NOT NULL THEN 1 ELSE 0 END +
               CASE WHEN impulse_control IS NOT NULL THEN 1 ELSE 0 END) >= 3;
        """
    elif criteria == "all":
        # to select records where all emotional variables are not null 
        query = """
        SELECT medical_record_id, patient_id
        FROM medical_records_colclean
        WHERE hunger IS NOT NULL
          AND satiety IS NOT NULL
          AND emotional_eating IS NOT NULL
          AND emotional_eating_value IS NOT NULL
          AND quantity_control IS NOT NULL
          AND impulse_control IS NOT NULL;
        """
    
    # Get the relevant records, and extract their medical record and patient IDs
    relevant_records = pd.read_sql_query(query, conn)
    relevant_medical_record_ids = tuple(relevant_records['medical_record_id'])
    relevant_patient_ids = tuple(relevant_records['patient_id'])
    
    # Create a new database in the output directory
    output_db_path = os.path.join(paper1_directory, output_filename)
    filtered_conn = sqlite3.connect(output_db_path)
    
    # Filter each table in the source SQl to only contain the records that comply the criteria of the given scenario; 
    # ie. they have records where any/3+/all emotional variables are available
    for table_name in table_names:
        if table_name.startswith("sqlite_"):
            # Skip any SQLite system tables
            continue 
        # In case of tables that may contain several medical records from the same patient, 
        # filter by medical_record_id
        if table_name == "medical_records_colclean" or table_name == "prescriptions_colclean":
            query = f"""
            SELECT * 
            FROM {table_name}
            WHERE medical_record_id IN {relevant_medical_record_ids}
            """
        else:
            # For all other tables, filter by patient_id only, as medical record ID is not available in those
            query_check_column = f"PRAGMA table_info({table_name});"
            columns = pd.read_sql_query(query_check_column, conn)
            if 'patient_id' not in columns['name'].values:
                continue  # Skip tables without patient_id
            query = f"""
            SELECT *
            FROM {table_name}
            WHERE patient_id IN {relevant_patient_ids}
            """
        
        # Execute the given query and save the result in a new SQLite database
        filtered_data = pd.read_sql_query(query, conn)
        filtered_data.to_sql(table_name, filtered_conn, index=False, if_exists="replace")
    
    filtered_conn.close()
    return len(relevant_records), len(set(relevant_records['patient_id']))

# Create and save the three databases for the three scenarios - any/3+/all emotional variables available
any_count, any_patients = create_filtered_database("any", "emotional_any_notna.sqlite")
three_plus_count, three_plus_patients = create_filtered_database("3plus", "emotional_3plus_notna.sqlite")
all_count, all_patients = create_filtered_database("all", "emotional_all_notna.sqlite")

# Close the connection
conn.close()

# Print summary
print(f"Any emotional data points are available in {any_count} records from {any_patients} patients")
print(f"At least 3 emotional data points are available in {three_plus_count} records from {three_plus_patients} patients")
print(f"All emotional data points are available in {all_count} records from {all_patients} patients")

Any emotional data points are available in 2482 records from 2437 patients
At least 3 emotional data points are available in 2169 records from 2132 patients
All emotional data points are available in 1853 records from 1826 patients


#### Clean and link measurements to prescriptions and medical records

Based on the patient ID and the date of a given measurements, look for prescriptions with the same patient ID that cover the range of time in which the measurement was taken. This way, measurements can be linked to important metadata, such as the prescription and medical record they belong to, the step of the programme they were taken in, etc. 

In summary, this is a key step in the research, without which data on any measurement's identity would be insufficient, and measurements from different prescriptions of the same individual could be mixed, for example. In a previous attempt, I tried identifying blocks of measurements as those that are taken within two months of each other, but I consider this a much more solid approach. 

It is important to note that some patients may take repeated measurements on the same occasion. These duplicates need to be removed, as they inflate the dataset. 

After removing the duplicates, measurements and prescriptions are linked in a two-step process. 

First, measurements are linked to all possible prescriptions that can belong to them based on the shared patient ID (this scenario where every option is linked to every option is called a Cartesian product). 

After, these possible links are filtered by date: a measurement belongs to a prescription if it is within its validity period, or is 5 days within its start or end dates. In the latter case, a measurement may be assigned to multiple prescriptions; if this happens, it is assigned to the one it is closer to in time. 

If a measurement is not succesfully linked to any prescription, it is lost. 

In [6]:
"""
Remove duplicate measurements before doing any data frame merging. 
Any measurement from the same patient on the same day (ignoring time) with the same weight should be considered a duplicate.
"""

import pandas as pd
import sqlite3
import os

# Connect to the database, load the measurements table
conn = sqlite3.connect(os.path.join(paper1_directory, "emotional_all_notna.sqlite"))
measurements = pd.read_sql_query("SELECT * FROM measurements_colclean", conn)

# Convert measurement_date to datetime, if not already in that format. 
# Add a temporary column with the measurement date only; time is ignored, 
# as repeated measurements are at least a few seconds or minutes apart. 
measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'])
measurements['measurement_date_date'] = measurements['measurement_date'].dt.date
# After, remove duplicates based on patient id, date, and weight. 
# Drop the temporary column. 
measurements_rowclean = measurements.drop_duplicates(subset=['patient_id', 'measurement_date_date', 'weight_kg'])
measurements_rowclean = measurements_rowclean.drop(columns=['measurement_date_date'])
# Save the cleaned measurements back to the database with the _rowclean name code
measurements_rowclean.to_sql("measurements_rowclean", conn, if_exists="replace", index=False)

print(f"Duplicate measurements removed. There are {len(measurements_rowclean)} measurements from {measurements_rowclean['patient_id'].nunique()} patients.")

Duplicate measurements removed. There are 35709 measurements from 1826 patients.


In [7]:
"""
Link metadata from the prescriptions table to measurements. 

The two dataframes are merged based on patient_id, creating a Cartesian product of the two tables, 
where every measurement from one patient is linked to every possible prescription from that patient.

This Cartesian product is then filtered based on the dates of both the measurement and the prescription, 
in order to, preferably, only consider a prescription being linked to a given measurement
if the measurement date is between the prescription's start and end dates. 

If a measurement is not within any prescription's validity period, 
there is a permissivity of 5 days, meaning that a measurement can be linked to a prescription if
it is within 5 days from the start or end date of the prescription.
If this allows a measurement to be linked to multiple prescriptions,
it is linked to the one it is closest to in date. 

If a measurement is not linked to any valid prescription, 
it is excluded from the outuput. 
"""

# Connect to the paper-specific database, load the prescriptions table, and make sure its date values are in datetime format
conn = sqlite3.connect(os.path.join(paper1_directory, "emotional_all_notna.sqlite"))
prescriptions = pd.read_sql_query("SELECT * FROM prescriptions_colclean", conn)
prescriptions['prescription_creation_date'] = pd.to_datetime(prescriptions['prescription_creation_date'])
prescriptions['prescription_validity_end_date'] = pd.to_datetime(prescriptions['prescription_validity_end_date'])

# Merge the measurements and prescriptions data frames on patient ID,
# creating the Cartesian product that needs further date-based filtering
merged = pd.merge(measurements_rowclean, prescriptions, on="patient_id", how="left", suffixes=('_meas', '_presc'))

# To execute date-based filtering: 
# First, define those measurements that are within the range of a prescription. 
# If any measurement can be assigned to a prescription based on this criteria, it will be. 
merged['measurement_in_prescription_range'] = (
    (merged['measurement_date'] >= merged['prescription_creation_date']) &
    (merged['measurement_date'] <= merged['prescription_validity_end_date'])
)
# If after this, a measurement is still not linked to any prescription due to not being in the range of any, 
# it will be linked to the prescription it is closest to, within a 5-day permissivity range. 
# For these out-of-range measurements, first, the distance from the start/end dates of any prescription is calculated. 
merged['days_before_prescription_start'] = (merged['prescription_creation_date'] - merged['measurement_date']).dt.days
merged['days_after_prescription_end'] = (merged['measurement_date'] - merged['prescription_validity_end_date']).dt.days
# After, near-range measurements are defined, 
# as measurements that are NOT within the range of any prescription, 
# AND they are at within 5 days before the start/after the end of any prescription. 
merged['measurement_near_prescription_range'] = (
    (~merged['measurement_in_prescription_range']) &
    (
        ((merged['days_before_prescription_start'] <= 5) & (merged['days_before_prescription_start'] > 0)) |
        ((merged['days_after_prescription_end'] <= 5) & (merged['days_after_prescription_end'] > 0))
    )
)
# After, a distance metric calculation determines how far a given measurement is from a prescription. 
# In-range measurements get a distance metric of 0,
# while out-of-range measurements get the minimum distance to any boundary they are close to. 
merged['measurement_distance_from_prescription_range'] = merged.apply(
    lambda row: 0 if row['measurement_in_prescription_range'] else min(max(row['days_before_prescription_start'], 0), max(row['days_after_prescription_end'], 0)),
    axis=1
)
# After defining the in-range and near-range logics, the database (currently containing Cartesian products) 
# is filtered to keep only in-or near-range measurements. 
# Any measurements not assigned to a prescription is lost. 
measurements_with_metadata = merged[merged['measurement_in_prescription_range'] | merged['measurement_near_prescription_range']].copy()
# In edge cases where multiple prescriptions are linked to a single measurement, only the closest match is kept. 
# This is done by sorting the data frame by patient id, measurement date and distance from range, 
# and if multiple measurement-prescription pairs from the same patient on the same date are found, 
# duplicates are removed and only the row with the smallest distance from range is kept. 
measurements_with_metadata = measurements_with_metadata.sort_values(['patient_id', 'measurement_date', 'measurement_distance_from_prescription_range'])
measurements_with_metadata = measurements_with_metadata.drop_duplicates(['patient_id', 'measurement_date'])

# After filtering the data frame, columns are reordered, and any irrelevant ones, like prescribed supplements, are dropped. 
column_order = [
    'patient_id',
    'medical_record_id',
    'prescription_id',
    'measurement_date',
    'prescription_creation_date',
    'prescription_validity_end_date',
    'prescription_validity_days',
    'method',
    'step',
    'weight_kg',
    'bmi',
    'bmr_kcal',
    'fat_%',
    'vat_%',
    'muscle_%',
    'water_%',
    'measurement_in_prescription_range',
    'days_before_prescription_start',
    'days_after_prescription_end',
    'measurement_near_prescription_range',
    'measurement_distance_from_prescription_range'
]
measurements_with_metadata = measurements_with_metadata[column_order]

# The measurements_with_metadata data frame is saved within the SQL database, and some summary info is printed. 
measurements_with_metadata.to_sql("measurements_with_metadata", conn, if_exists="replace", index=False)

print(f"Measurements are linked to their corresponding prescriptions and medical records. \n"
    f"There are a total of {measurements_with_metadata.shape[0]} measurements "
    f"from {measurements_with_metadata['medical_record_id'].nunique()} medical records "
    f"of {measurements_with_metadata['patient_id'].nunique()} patients.")

Measurements are linked to their corresponding prescriptions and medical records. 
There are a total of 20976 measurements from 1678 medical records of 1664 patients.


#### Add sex, genomics ID and baseline/final weight data to medical records

In an effort to create data frames containing the most possible information in one place, the medical records data frame is completed with the sex (originally stored in Patients) as well as the baseline and final weight data (measurements linked to medical records stored in measurements_with_metadata) of patients. Genomics sample IDs are also fetched for patients that have it available. 

Categorical values (gender and 3/6 eating behavior values) are converted to boolean 0/1 integers instead of str values. 1 means Yes or Female. Accordingly, the sex column is named sex_f downstream to account for the fact that 1 = Female. The eating behavior values have a corresponding _yn or _likert appended to their name. 

Besides executing these merge operations, the code checks the time passed between baseline and final measurements in each medical record, along with whether the measurements are close to the beginning/end date of the medical record they belong to or not. This helps checking whether the length of the actual followup is similar to that of the medical record or not. 

Any medical record that has no associated measurements is lost here. 

In [ ]:
"""
Complete Medical records by adding sex, baseline/final weight data and genomics sample IDs to it. 

Sex and genomics sample IDs are fetched from the Patients table, based on the patient_id.

Baseline and final weight measurements are obtained from the measurements_with_metadata table created in the previous step. 
The logic is the following: 
Measurements are grouped by patient and medical record ID, and the first and last measurements of each group are assigned
to the medical records table as baseline and final measurements, respectively.
Delta weight is calculated as the difference between final and baseline weights, to obtain negative results. 

Measurement dates are added and it is checked if they are within the medical record creation and closing dates.

If a medical record has no measurements linked to it, it is dropped. 

Additionally, the 'days_between_measurements' column is added to calculate the number of days between the baseline and final measurements.

Sex and categorical eating behavior variables are converted to boolean values (0/1 integers) for easier downstream analysis. 
1 = yes/female, 0 = no/male. Accordingly, the sex column is renamed to sex_f, and eating behavior values are renamed to [value]_yn or [value]_likert.

Finally, the columns are reordered to match the desired order.
"""

import pandas as pd
import sqlite3

# Connect to the database, load relevant tables
conn = sqlite3.connect(os.path.join(paper1_directory, "emotional_all_notna.sqlite"))
medical_records = pd.read_sql_query("SELECT * FROM medical_records_colclean", conn)
patients = pd.read_sql_query("SELECT * FROM patients_colclean", conn)
measurements_with_metadata = pd.read_sql_query("SELECT * FROM measurements_with_metadata", conn)

# The following functions complete the original medical records data frame with research-relevant variables.
# First, add the sex variable to medical_records_complete by merging patients' sex into medical_records_complete based on patient_id
"""
Adding sex data and genomics sample IDs to medical records
"""
medical_records_complete = pd.merge(
    medical_records,
    patients[['patient_id', 'sex', 'genomics_sample_id']],
    on='patient_id',
    how='left'
)

# After, add baseline and final measurements to medical_records_complete
# Treat measurements coming from a given medical record as units
# by grouping measurements_with_metadata by patient_id and medical_record_id 
"""
Adding weight data to medical records
"""
grouped_measurements = measurements_with_metadata.groupby(['patient_id', 'medical_record_id'])
# Extract the first (baseline) and last (final) measurement for each group
baseline = grouped_measurements.first().reset_index()
final = grouped_measurements.last().reset_index()
# Insert baseline and final measurements into medical_records_complete
medical_records_complete = pd.merge(
    medical_records_complete,
    baseline[['patient_id', 'medical_record_id', 'measurement_date', 'weight_kg', 'bmi']],
    on=['patient_id', 'medical_record_id'],
    how='left'
)
medical_records_complete = pd.merge(
    medical_records_complete,
    final[['patient_id', 'medical_record_id', 'measurement_date', 'weight_kg', 'bmi']],
    on=['patient_id', 'medical_record_id'],
    how='left',
    suffixes=('_baseline', '_final')
)
# Make sure all dates are in datetime format for further operations, 
# and calculate delta weight and delta BMI values (final - baseline, so the resulting weight loss value is negative)
medical_records_complete['medical_record_creation_date'] = pd.to_datetime(medical_records_complete['medical_record_creation_date'])
medical_records_complete['medical_record_closing_date'] = pd.to_datetime(medical_records_complete['medical_record_closing_date'])
medical_records_complete['measurement_date_baseline'] = pd.to_datetime(medical_records_complete['measurement_date_baseline'])
medical_records_complete['measurement_date_final'] = pd.to_datetime(medical_records_complete['measurement_date_final'])
medical_records_complete['delta_weight_kg'] = medical_records_complete['weight_kg_final'] - medical_records_complete['weight_kg_baseline']
medical_records_complete['delta_bmi'] = medical_records_complete['bmi_final'] - medical_records_complete['bmi_baseline']
# Check if the baseline and final measurements are close to the starting/closing date of the medical record they belong to or not (within a 10-day window). 
# In some cases, the first measurement is recorded weeks after opening the medical record, or the last one is taken long before closing it. 
# In other cases, the medical record's closing date is absent, if this happens, the last measurement will be considered as out of range. 
# This is supposed to help identify cases where the followup has some imperfections. 
window_days = 10
medical_records_complete['baseline_measurement_inrange'] = (
    (medical_records_complete['measurement_date_baseline'] >= 
     medical_records_complete['medical_record_creation_date'] - pd.Timedelta(days=window_days)) &
    (medical_records_complete['measurement_date_baseline'] <=
     medical_records_complete['measurement_date_baseline'] + pd.Timedelta(days=window_days))
)
medical_records_complete['final_measurement_inrange'] = (
    (medical_records_complete['measurement_date_final'] >= 
     medical_records_complete['medical_record_closing_date'] - pd.Timedelta(days=window_days)) &
    (medical_records_complete['measurement_date_final'] <= 
     medical_records_complete['medical_record_closing_date'] + pd.Timedelta(days=window_days))
)
# Add a column that calculates the days passed between baseline and final measurements
# This also helps identify cases where the medical record's duration and the actual followup time are very different
medical_records_complete['days_between_measurements'] = (
    (medical_records_complete['measurement_date_final'] - medical_records_complete['measurement_date_baseline']).dt.days
)

"""
Convert categorical columns (sex, hunger, satiety, emotional_eating) 
to boolean values (0/1 integers) for downstream calculations. 
In case of gender, female = 0, and the column name is renamed accordingly (sex_f). 
In case of eating behavior datapoints, yes = 1, no = 0.
"""
print("Converting categorical columns (sex, hunger, satiety, emotional_eating) to 0/1 integers...")
# Define mappings
sex_map = {'female': 1, 'male': 0}
yes_no_map = {'yes': 1, 'no': 0}

# Apply mappings - use .get() on map to handle potential unexpected values gracefully (assigns NaN)
if 'sex' in medical_records_complete.columns:
    original_sex_nan_count = medical_records_complete['sex'].isnull().sum()
    medical_records_complete['sex'] = medical_records_complete['sex'].map(sex_map)
    new_sex_nan_count = medical_records_complete['sex'].isnull().sum()
    if new_sex_nan_count > original_sex_nan_count:
        print("  Warning: Mapping 'sex' introduced NaNs. Check source data for values other than 'Female'/'Male'.")
    
# """CHECK MAYBE DELETE"""
    
    # # Rename sex column AFTER conversion
    # medical_records_complete = medical_records_complete.rename(columns={'sex': 'sex_f'})
    # print("  Renamed 'sex' column to 'sex_f'.")


for col in ['hunger', 'satiety', 'emotional_eating']:
    if col in medical_records_complete.columns:
        original_nan_count = medical_records_complete[col].isnull().sum()
        # Ensure column is string before mapping, handle potential non-string values
        medical_records_complete[col] = medical_records_complete[col].astype(str).map(yes_no_map)
        new_nan_count = medical_records_complete[col].isnull().sum()
        if new_nan_count > original_nan_count:
             print(f"  Warning: Mapping '{col}' introduced NaNs. Check source data for values other than 'Yes'/'No'.")

"""
Removing medical records with no associated measurements
"""
# As for some reason (unidentified as of 16Apr25) many medical records have no available measurements associated to them, 
# any such instances are dropped from the data frame. 
medical_records_complete = medical_records_complete.dropna(subset=['weight_kg_baseline', 'weight_kg_final'])

"""
Presenting and saving the output
"""
# Rename and reorder columns for better clarity and interpretability
medical_records_complete = medical_records_complete.rename(columns={
    'measurement_date_baseline': 'baseline_measurement_date',
    'measurement_date_final': 'final_measurement_date',
    'weight_kg_baseline': 'baseline_weight_kg',
    'weight_kg_final': 'final_weight_kg', 
    'bmi_baseline': 'baseline_bmi',
    'bmi_final': 'final_bmi',
    'sex': 'sex_f', 
    'hunger': 'hunger_yn',
    'satiety': 'satiety_yn',
    'emotional_eating': 'emotional_eating_yn',
    'emotional_eating_value': 'emotional_eating_value_likert',
    'quantity_control': 'quantity_control_likert',
    'impulse_control': 'impulse_control_likert',
})
desired_column_order = [
    'patient_id',
    'medical_record_id',
    'genomics_sample_id',
    'medical_record_creation_date',
    'medical_record_closing_date',
    'intervention_duration_days',
    'baseline_measurement_date',
    'final_measurement_date',
    'days_between_measurements',
    'baseline_measurement_inrange',
    'final_measurement_inrange',
    'birth_date',
    'age',
    'age_when_creating_record',
    'sex_f',
    'height_m',
    'baseline_weight_kg',
    'final_weight_kg',
    'delta_weight_kg',
    'baseline_bmi',
    'final_bmi',
    'delta_bmi',
    'wc_cm_confirm_time',
    'pnk_method',
    'orders_in_medical_record',
    'dietitian_visits',
    'physical_activity',
    'physical_activity_frequency',
    'physical_inactivity_cause',
    'weight_gain_cause',
    'smoking',
    'medications',
    'hunger_yn',
    'satiety_yn',
    'emotional_eating_yn',
    'emotional_eating_value_likert',
    'quantity_control_likert',
    'impulse_control_likert'
]
medical_records_complete = medical_records_complete[desired_column_order]
# Save the complete medical records to the SQL database, and print a summary statement
medical_records_complete.to_sql("medical_records_complete", conn, if_exists="replace", index=False)
print(f"Medical records table completed with sex and baseline/final weight data. \n" 
      f"There are {len(medical_records_complete)} records available from {medical_records_complete['patient_id'].nunique()} patients.")

Converting categorical columns (sex, hunger, satiety, emotional_eating) to 0/1 integers...
Medical records table completed with sex and baseline/final weight data. 
There are 1678 records available from 1664 patients.


#### Create the base input for survival analysis - v0

Here, data frames specifically prepared for survival analysis are created. Time-to-event (days) of achieving 3 different weight loss targets (5-10-15%) in 3 different time frames (40-60-80 days) is analyzed. Relevant demographic, anthropometric and eating behavior variables are added to each analyzed medical record. 

In [15]:
import pandas as pd
import os
import sqlite3
from datetime import timedelta
# Removed: import logging

"""
CONFIGURATION
"""
# Define directories and database paths - paper1_directory should be defined 
# in the first cell of this notebook chapter
input_db_path = os.path.join(paper1_directory, 'emotional_all_notna.sqlite')
input_measurements = "measurements_with_metadata"
input_medical_records = "medical_records_complete"
output_db_path = os.path.join(paper1_directory, 'survival_analysis.sqlite')

# Define analysis parameters
weight_loss_targets = [5, 10, 15]     # Weight loss target percentages
time_windows = [40, 60, 80]       # Time windows (centers) in days
window_span = 10                   # Permissible span around windows (+/- days)

# Define the variables stored in medical_records_complete that are relevant for the analysis. 
# These include basic metadata like patient and record ID,
# basic factors such as age and sex, 
# as well as the emotional and eating behavior variables pivotal to the research question. 
# The list can be amended on demand - 
# for example, right now it does not include medical record creating and closing dates. 
relevant_medical_values = ['patient_id', 'medical_record_id', 'sex', 'age',
                             'height_m', 'baseline_bmi', 'hunger', 'satiety', 'emotional_eating',
                             'emotional_eating_value', 'quantity_control', 'impulse_control']

"""
DATA LOADING & PREPARATION
"""

def load_measurements(connection):
    """
    Load measurements from the measurement_with_metadata table; 
    make sure key values are in the correct format. 
    """
    query = f"SELECT * FROM {input_measurements}"
    measurements = pd.read_sql_query(query, connection)
    measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'], errors='coerce')
    measurements['weight_kg'] = pd.to_numeric(measurements['weight_kg'], errors='coerce')
    return measurements

def load_medical_records(connection):
    """
    Load medical records from the medical_records_complete table;
    make sure date values are in datetime format. 
    The exact columns to be used are defined in the prepare_patient_data function.
    """
    query = f"SELECT * FROM {input_medical_records}"
    medical_records = pd.read_sql_query(query, connection)
    medical_records['medical_record_creation_date'] = pd.to_datetime(medical_records['medical_record_creation_date'], errors='coerce')
    return medical_records

def prepare_patient_data(measurements, medical_records):
    """
    Filter measurements to only include those from the earliest medical record for each patient.
    Merge measurements with relevant medical record data, including the pivotal eating behavior scores. 
    """
    # Filter measurements to only include those from the first treatment record of each patient
    earliest_records_with_data = measurements.sort_values('measurement_date')\
        .groupby('patient_id')['medical_record_id']\
        .first()\
        .reset_index()
    filtered_measurements = pd.merge(
        measurements,
        earliest_records_with_data,
        on=['patient_id', 'medical_record_id'],
        how='inner'
    )
    # Identify the baseline measurement in each record
    baseline_data = filtered_measurements.sort_values('measurement_date')\
                                       .groupby(['patient_id', 'medical_record_id'])\
                                       .first()\
                                       .reset_index()

    cols_to_select = [col for col in relevant_medical_values if col in medical_records.columns]
    medical_record_data = medical_records[cols_to_select]
    # Merge baseline measurements with relevant medical record data
    prepared_data = pd.merge(
        baseline_data,
        medical_record_data,
        on=['patient_id', 'medical_record_id'],
        how='left' # Keep all baseline data
    )
    return prepared_data, filtered_measurements

"""
CALCULATE WEIGHT LOSS OUTCOMES
"""

def _get_patient_baseline(patient_data, patient_id, medical_record_id):
    """
    Get the baseline data for each patient's corresponding medical record.
    """
    patient_baseline = patient_data[
        (patient_data['patient_id'] == patient_id) &
        (patient_data['medical_record_id'] == medical_record_id)
    ]
    if len(patient_baseline) == 0:
        print(f"WARN: No baseline data found for patient {patient_id}, record {medical_record_id}. Skipping.")
        return None
    return patient_baseline.iloc[0]

def _check_target_achievement(measurements_within_window, baseline_weight, weight_loss_target):
    """
    Check if the weight loss target was achieved in some of the given measurements.
    """
    # Set default to False/None
    target_achieved = False
    first_success_measurement = None
    # Calculate weight loss percentage for each measurement in the window, 
    # and check if it meets the target
    for _, row in measurements_within_window.iterrows():
        current_weight = row['weight_kg']
        if baseline_weight is not None and baseline_weight > 0:
            current_weight_loss = ((baseline_weight - current_weight) / baseline_weight) * 100
            if round(current_weight_loss, 2) >= weight_loss_target:
                target_achieved = True
                first_success_measurement = row
                break # Stop at the first success; if that is not identified, target_achieved remains False as by default
    return target_achieved, first_success_measurement

def _determine_final_measurement(target_achieved, first_success_row, measurements_around_cutoff,
                                measurements_within_window, baseline_date, window_center):
    """
    Determine the final measurement based on success or censoring (ie. completion without success) rules.
    """
    # Set final measurement to None by default
    final_measurement = None
    # Set target final date based on the given time window
    target_date = baseline_date + timedelta(days=window_center)
    # If weight loss target was achieved at any point of the followup time window,
    # use the first success measurement as the final measurement.  
    if target_achieved:
        final_measurement = first_success_row
    # In case of no success, the date closest to the target date is used as the final measurement. 
    elif not measurements_around_cutoff.empty:
        measurements_around_cutoff = measurements_around_cutoff.copy()
        measurements_around_cutoff['distance_to_center'] = abs(
            (measurements_around_cutoff['measurement_date'] - target_date).dt.days
        )
        closest_measurement_idx = measurements_around_cutoff['distance_to_center'].idxmin()
        final_measurement = measurements_around_cutoff.loc[closest_measurement_idx]
    # In case of no success nor completion (delayed dropout), use the last available measurement as the final measurement
    elif not measurements_within_window.empty:
        final_measurement = measurements_within_window.sort_values('measurement_date').iloc[-1]
    # Else: Instant dropout, final_measurement remains None, 
    # and is set to the baseline measurementin the calculate_outcome_metrics function.
    return final_measurement

def _calculate_outcome_metrics(baseline_row, final_measurement_row):
    """
    Calculate follow-up lenght and weight loss percentage based on baseline and final measurement.
    """
    # Identify the baseline measurement
    baseline_date = baseline_row['measurement_date']
    baseline_weight = baseline_row['weight_kg']
    # In patients that have at least one followup measurement, identify the end date and final weight, 
    # to calculate followup length and weight loss in kg and %
    if final_measurement_row is not None:
        end_date = final_measurement_row['measurement_date']
        final_weight = final_measurement_row['weight_kg']
        followup_period = (end_date - baseline_date).days
        weight_loss_kg = baseline_weight - final_weight
        weight_loss_pct = ((baseline_weight - final_weight) / baseline_weight) * 100
    # In patients that have no followup measurement (instant dropouts), 
    # the end date and final weight are set to the baseline values, 
    # and followup length and weight loss are set to 0. 
    else: 
        end_date = baseline_date
        final_weight = baseline_weight
        followup_period = 0
        weight_loss_kg = 0
        weight_loss_pct = 0
    return {
        'end_date': end_date,
        'final_weight': final_weight,
        'followup_period': followup_period,
        'weight_loss_kg': weight_loss_kg,
        'weight_loss_pct': round(weight_loss_pct, 2)
    }

"""
CORE ANALYSIS FUNCTION
"""

def calculate_weight_loss_outcome(patient_data, filtered_measurements, weight_loss_target, window_center, window_span):
    """
    Calculate weight loss outcomes for each patient in a survival analysis-ready format. 
    """
    # Initialize an empty list to store results, and group measurements by patient and medical record ID
    results = []
    grouped_measurements = filtered_measurements.groupby(['patient_id', 'medical_record_id'])
    # Iterate through each group within measurements. 
    for (patient_id, medical_record_id), group in grouped_measurements:
        # 1. Identify baseline measurement date and weight
        baseline_row = _get_patient_baseline(patient_data, patient_id, medical_record_id)
        if baseline_row is None: continue
        baseline_date = baseline_row['measurement_date']
        baseline_weight = baseline_row['weight_kg']
        # 2. Define observation time windows and group measurements within the defined window
        # Calculations are done for both the complete observation period, 
        # as well as the period strictry around the cutoff date, within the defined permissivity window. 
        min_window_date = baseline_date + timedelta(days=(window_center - window_span))
        max_window_date = baseline_date + timedelta(days=(window_center + window_span))
        measurements_within_window = group[
            (group['measurement_date'] > baseline_date) &
            (group['measurement_date'] <= max_window_date)
        ].sort_values('measurement_date')
        measurements_around_cutoff = group[
            (group['measurement_date'] >= min_window_date) &
            (group['measurement_date'] <= max_window_date)
        ]
        # 3. Check whether target weight loss was achieved in the defined time window
        target_achieved, first_success_row = _check_target_achievement(
            measurements_within_window, baseline_weight, weight_loss_target
        )
        # 4. Identify the last measurement date within the time window,
        # whether based on target achievment or followup completion
        final_measurement_row = _determine_final_measurement(
            target_achieved, first_success_row, measurements_around_cutoff,
            measurements_within_window, baseline_date, window_center
        )
        # 5. Check for dropout status - instant dropouts are those who have no second measurement, 
        # while delayed dropouts are those who have not reached target, 
        # and their final measurement is before the cutoff window. 
        is_instant_dropout = final_measurement_row is None
        is_delayed_dropout = (not target_achieved and
                              final_measurement_row is not None and
                              final_measurement_row['measurement_date'] < min_window_date)
        dropout = is_instant_dropout or is_delayed_dropout
        success = target_achieved
        # 6. Calculate metrics like final date and weight, followup length and weight lost. 
        outcome_metrics = _calculate_outcome_metrics(baseline_row, final_measurement_row)

        """ARE WE GOING TO MODIFY AVG CALCS?"""

        # # 7. 
        # # --- NEW: Calculate metrics based *always* on the last measurement within the window ---
        # actual_last_measurement_row = None
        # if not measurements_within_window.empty:
        #     actual_last_measurement_row = measurements_within_window.iloc[-1]

        # # Use the same helper, but pass the actual last measurement row
        # actual_end_metrics = _calculate_outcome_metrics(baseline_row, actual_last_measurement_row)
        # actual_wl_pct_at_window_end = actual_end_metrics['weight_loss_pct']
        # # --- End NEW ---



        # 8. Assemble the result - this is where the output tables' columns are defined. 
        # If additional variables are inserted at an earlier part of the code, 
        # they need to be mentioned here as well. 
        result = {
            'patient_id': patient_id,
            'medical_record_id': medical_record_id,
            'baseline_date': baseline_date,
            'end_date': outcome_metrics['end_date'],
            'followup_period': outcome_metrics['followup_period'],
            'baseline_weight': baseline_weight,
            'final_weight': outcome_metrics['final_weight'],
            'weight_loss_kg': outcome_metrics['weight_loss_kg'],
            'weight_loss_pct': outcome_metrics['weight_loss_pct'],
            # 
            f'{weight_loss_target}pct_achieved': success,
            'dropout': dropout,
            # Add baseline characteristics safely using .get()
            'sex': baseline_row.get('sex'),
            'age': baseline_row.get('age'),
            'height_m': baseline_row.get('height_m'),
            'baseline_bmi': baseline_row.get('bmi'),
            'hunger': baseline_row.get('hunger'),
            'satiety': baseline_row.get('satiety'),
            'emotional_eating': baseline_row.get('emotional_eating'),
            'emotional_eating_value': baseline_row.get('emotional_eating_value'),
            'quantity_control': baseline_row.get('quantity_control'),
            'impulse_control': baseline_row.get('impulse_control')
        }
        results.append(result)
    return pd.DataFrame(results)


"""
MAIN ORCHESTRATION FUNCTION
"""

def generate_survival_analysis_datasets(input_connection, output_connection, weight_loss_targets, time_windows, window_span=10):
    """
    The main function to orchestrate the survival analysis process, calling all previously defined functions in an organized manner. 
    Generate survival analysis datasets for multiple weight loss targets and observation time windows.
    Targets and timeframes are defined in the configuration section at the beginning of the code module.
    Save data to a separate SQLite database. 
    """
    # 1. Load and prepare input data
    measurements = load_measurements(input_connection)
    medical_records = load_medical_records(input_connection)
    patient_data, filtered_measurements = prepare_patient_data(measurements, medical_records)
    if patient_data.empty:
        print("ERROR: Prepared patient data is empty. Cannot proceed.")
        return {}, pd.DataFrame()
    # 2. Calculate weight loss outcomes for each target-timeframe combination. 
    # Targets and timeframes are defined in the config section of the script. 
    # Initialize a results dictionary and a list for summary statistics. 
    results = {}
    summary_list = []
    for window in sorted(time_windows):
        for target in sorted(weight_loss_targets):
            # Name each instance accordingly, where sa stands for survival analysis, 
            # and the numbers indicate the time window and target percentage.
            name = f"sa_{window}d_{target}p"
            print(f"--- Processing: {name} ---") # Minimal progress indication
            result_df = calculate_weight_loss_outcome(
                patient_data,
                filtered_measurements,
                target,
                window,
                window_span # Defined in config - the permissivity window around the followup cutoff time
            )
            results[name] = result_df
            # Add the calculated instances to the summary statistics list. 
            if not result_df.empty:
                summary_row = {
                    'analysis_name': name,
                    'weight_loss_target': target,
                    'time_window': window,
                    'total_patients': len(result_df),
                    'achieved_target': int(result_df[f'{target}pct_achieved'].sum()),
                    'dropout_count': int(result_df['dropout'].sum()),
                    'avg_weight_loss_pct': result_df['weight_loss_pct'].mean() if not result_df['weight_loss_pct'].isnull().all() else 0
                }
                summary_list.append(summary_row)
            else:
                 print(f"WARN: No results generated for {name}. Skipping summary entry.")
    # Turn the summary statistics list into a data frame
    summary = pd.DataFrame(summary_list)

    # 3. Save the analysis results (9 tables by default) to the SQLite database defined in the config section
    print(f"--- Saving results to output database: {output_db_path} ---")
    # Save individual tables
    for name, df in results.items():
        print(f"Saving table: {name} ({len(df)} rows)")
        df.to_sql(name, output_connection, if_exists='replace', index=False)
    # Save the summary stats table in the database as well
    print(f"Saving summary table: survival_analysis_summary ({len(summary)} rows)")
    summary.to_sql('survival_analysis_summary', output_connection, if_exists='replace', index=False)
    output_connection.commit() # Ensure changes are saved
    print("--- All results saved successfully ---")
    return results, summary

"""
EXECUTION BLOCK
"""

"""
This part of the code calls all the functions and executes the code. 
Currently it has a lot of debug messages and error handling, which might be an overkill, 
but overall, it should not affect transparency of the code.
"""

if __name__ == "__main__":
    print("========== Starting Survival Analysis Script ==========")
    # By default, connections are set to None, and will be established in the try block.
    input_conn = None
    output_conn = None
    try:
        # Connect to in-and output databases
        print(f"Connecting to input database: {input_db_path}")
        if not os.path.exists(input_db_path):
             raise FileNotFoundError(f"Input database not found at {input_db_path}")
        input_conn = sqlite3.connect(input_db_path)
        print(f"Connecting to output database: {output_db_path}")
        output_conn = sqlite3.connect(output_db_path)
        # Run the main analysis function
        results, summary = generate_survival_analysis_datasets(
            input_conn,
            output_conn,
            weight_loss_targets,
            time_windows,
            window_span
        )
        # Display summary if successful
        if not summary.empty:
            print("\n--- Survival Analysis Summary ---")
            print(summary.to_string()) # Use print for console display
            print("--- End Summary ---")
        else:
            print("WARN: Analysis completed, but the summary table is empty.")
        print(f"Analysis data successfully generated and saved to {output_db_path}")

    # Minimal error handling for critical failures
    except FileNotFoundError as e:
        print(f"ERROR: Database file not found - {e}")
    except sqlite3.Error as e:
        print(f"ERROR: SQLite database error - {e}")
    except ValueError as e:
        print(f"ERROR: Data processing error - {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred - {e}")
        # Consider adding traceback for debugging complex errors:
        # import traceback
        # print(traceback.format_exc())
    finally:
        # Ensure connections are closed
        print("Closing database connections...")
        if input_conn:
            input_conn.close()
        if output_conn:
            output_conn.close()
        print("========== Survival Analysis Script Finished ==========")


========== Starting Survival Analysis Script ==========
Connecting to input database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\emotional_all_notna.sqlite
Connecting to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite
--- Processing: sa_40d_5p ---
--- Processing: sa_40d_10p ---
--- Processing: sa_40d_15p ---
--- Processing: sa_60d_5p ---
--- Processing: sa_60d_10p ---
--- Processing: sa_60d_15p ---
--- Processing: sa_80d_5p ---
--- Processing: sa_80d_10p ---
--- Processing: sa_80d_15p ---
--- Saving results to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite ---
Saving table: sa_40d_5p (1664 rows)
Saving table: sa_40d_10p (1664 rows)
Saving table: sa_40d_15p (1664 rows)
Saving table: sa_60d_5p (1664 rows)
Saving table: sa_60d_10p (1664 rows)
Saving table: sa_60d_15p (1664 rows)
Saving table: sa_80d_5p (1664 rows)
Saving table: sa_80d_10p (1664 rows)
Savin

#### Reconsidered input structure for SA 18-19 Apr v1 - this is still not perfect

In [ ]:
import pandas as pd
import os
import sqlite3
from datetime import timedelta
import numpy as np # Import numpy for NaN

"""
CONFIGURATION
"""
# Define directories and database paths - paper1_directory should be defined
# in the first cell of this notebook chapter
input_db_path = os.path.join(paper1_directory, 'emotional_all_notna.sqlite')
input_measurements = "measurements_with_metadata"
input_medical_records = "medical_records_complete"
# Modified: Output database path remains the same, but will contain a different table structure
output_db_path = os.path.join(paper1_directory, 'shukishukishuuu.sqlite')
output_table_name = "shukishukishuuu" # Define the name for the single wide table

# Define analysis parameters
weight_loss_targets = [5, 10, 15]     # Weight loss target percentages
time_windows = [40, 60, 80]       # Time windows (centers) in days
window_span = 10                   # Permissible span around windows (+/- days)

# Define the variables stored in medical_records_complete that are relevant for the analysis.
# These include basic metadata like patient and record ID,
# basic factors such as age and sex,
# as well as the emotional and eating behavior variables pivotal to the research question.
# The list can be amended on demand -
# for example, right now it does not include medical record creating and closing dates.
relevant_medical_values = ['patient_id', 'medical_record_id', 'sex', 'age',
                             'height_m', 'baseline_bmi', 'hunger', 'satiety', 'emotional_eating',
                             'emotional_eating_value', 'quantity_control', 'impulse_control', 'genomics_sample_id']

"""
DATA LOADING & PREPARATION
"""

def load_measurements(connection):
    """
    Load measurements from the measurement_with_metadata table;
    make sure key values are in the correct format.
    """
    query = f"SELECT * FROM {input_measurements}"
    measurements = pd.read_sql_query(query, connection)
    measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'], errors='coerce')
    measurements['weight_kg'] = pd.to_numeric(measurements['weight_kg'], errors='coerce')
    # Ensure sorting for consistent 'first'/'last' operations later
    measurements = measurements.sort_values(['patient_id', 'medical_record_id', 'measurement_date'])
    return measurements

def load_medical_records(connection):
    """
    Load medical records from the medical_records_complete table;
    make sure date values are in datetime format.
    The exact columns to be used are defined in the prepare_patient_data function.
    """
    query = f"SELECT * FROM {input_medical_records}"
    medical_records = pd.read_sql_query(query, connection)
    # Ensure relevant date columns are datetime
    for col in ['medical_record_creation_date', 'baseline_measurement_date', 'final_measurement_date']:
         if col in medical_records.columns:
             medical_records[col] = pd.to_datetime(medical_records[col], errors='coerce')
    return medical_records

def prepare_patient_data(measurements, medical_records):
    """
    Filter measurements to only include those from the earliest medical record for each patient.
    Merge measurements with relevant medical record data, including the pivotal eating behavior scores.
    """
    # Filter measurements to only include those from the first treatment record of each patient
    # Ensure measurements are sorted before grouping
    measurements = measurements.sort_values(['patient_id', 'measurement_date'])
    earliest_records_with_data = measurements.groupby('patient_id')['medical_record_id'].first().reset_index()

    filtered_measurements = pd.merge(
        measurements,
        earliest_records_with_data,
        on=['patient_id', 'medical_record_id'],
        how='inner'
    )
    # Ensure filtered_measurements are sorted for baseline identification
    filtered_measurements = filtered_measurements.sort_values(['patient_id', 'medical_record_id', 'measurement_date'])

    # Identify the baseline measurement row for each record (the first measurement in the filtered set)
    baseline_data_rows = filtered_measurements.groupby(['patient_id', 'medical_record_id']).first().reset_index()

    # Select only relevant columns from medical_records to merge
    cols_to_select = [col for col in relevant_medical_values if col in medical_records.columns]
    medical_record_subset = medical_records[cols_to_select]

    # Merge baseline measurement info with the selected medical record data
    # Use baseline_data_rows which contains the actual first measurement details
    prepared_data = pd.merge(
        baseline_data_rows[['patient_id', 'medical_record_id', 'measurement_date', 'weight_kg']], # Get baseline date/weight from actual first measurement
        medical_record_subset,
        on=['patient_id', 'medical_record_id'],
        how='left' # Keep all baseline measurements
    )
    # Rename columns for clarity before returning
    prepared_data = prepared_data.rename(columns={'measurement_date': 'baseline_date', 'weight_kg': 'baseline_weight_kg'})

    # Add baseline_bmi from medical_records if available and not already present from measurement merge
    if 'baseline_bmi' in medical_record_subset.columns and 'baseline_bmi' not in prepared_data.columns:
         prepared_data = pd.merge(
              prepared_data,
              medical_record_subset[['patient_id', 'medical_record_id', 'baseline_bmi']],
              on=['patient_id', 'medical_record_id'],
              how='left'
         )

    return prepared_data, filtered_measurements


"""
MODIFIED: CALCULATE WEIGHT LOSS OUTCOMES FOR WIDE TABLE
"""

def _get_patient_baseline(patient_data, patient_id, medical_record_id):
    """
    Get the prepared baseline data for a patient's specific medical record.
    (Function remains largely the same, but operates on the prepared_data structure)
    """
    patient_baseline = patient_data[
        (patient_data['patient_id'] == patient_id) &
        (patient_data['medical_record_id'] == medical_record_id)
    ]
    if patient_baseline.empty:
        print(f"WARN: No baseline data found for patient {patient_id}, record {medical_record_id}. Skipping.")
        return None
    # Ensure we return a Series for consistent access
    return patient_baseline.iloc[0]

def _calculate_wl_metrics(baseline_weight, current_weight):
    """ Helper to calculate weight loss kg and % """
    if pd.isna(baseline_weight) or pd.isna(current_weight) or baseline_weight == 0:
        return np.nan, np.nan
    wl_kg = baseline_weight - current_weight
    wl_pct = (wl_kg / baseline_weight) * 100
    return wl_kg, round(wl_pct, 2)

# Removed _check_target_achievement, _determine_final_measurement, _calculate_outcome_metrics
# Their logic will be integrated into the main calculation function.

def calculate_wide_patient_outcomes(prepared_patient_data, filtered_measurements, weight_loss_targets, time_windows, window_span):
    """
    Calculate all required outcomes (fixed-time and time-to-event) for each patient
    and return a list of dictionaries, each representing a row in the wide table.
    """
    results_list = []
    # Group all measurements to only get the relevant (earliest) medical record per patient
    grouped_measurements = filtered_measurements.groupby(['patient_id', 'medical_record_id'])

    for (patient_id, medical_record_id), group in grouped_measurements:
        # 1. Get Baseline Info
        baseline_info = _get_patient_baseline(prepared_patient_data, patient_id, medical_record_id)
        if baseline_info is None:
            continue

        baseline_date = baseline_info['baseline_date']
        baseline_weight = baseline_info['baseline_weight_kg']

        # Initialize result dictionary with baseline info
        result = {
            'patient_ID': patient_id, 
            'medical_record_ID': medical_record_id,
            'baseline_date': baseline_date,
            'baseline_weight_kg': baseline_weight,
            # Add other baseline characteristics safely using .get() or direct access
            'sex': baseline_info.get('sex'),
            'age': baseline_info.get('age'),
            'height_m': baseline_info.get('height_m'),
            'baseline_bmi': baseline_info.get('baseline_bmi'), # Get BMI from prepared data
            'hunger': baseline_info.get('hunger'),
            'satiety': baseline_info.get('satiety'),
            'emotional_eating': baseline_info.get('emotional_eating'),
            'emotional_eating_value': baseline_info.get('emotional_eating_value'),
            'quantity_control': baseline_info.get('quantity_control'),
            'impulse_control': baseline_info.get('impulse_control')
        }

        # Get all measurements *after* baseline for this group
        followup_measurements = group[group['measurement_date'] > baseline_date].sort_values('measurement_date')

        # 2. Calculate Overall Follow-up Metrics
        if not followup_measurements.empty:
            last_measurement = followup_measurements.iloc[-1]
            result['last_aval_date'] = last_measurement['measurement_date']
            result['total_followup_days'] = (last_measurement['measurement_date'] - baseline_date).days
            result['last_aval_weight_kg'] = last_measurement['weight_kg']
            wl_kg, wl_pct = _calculate_wl_metrics(baseline_weight, last_measurement['weight_kg'])
            result['total_wl_kg'] = wl_kg
            result['total_wl_%'] = wl_pct
        else:
            # Handle instant dropouts (only baseline measurement exists)
            result['last_aval_date'] = baseline_date
            result['total_followup_days'] = 0
            result['last_aval_weight_kg'] = baseline_weight
            result['total_wl_kg'] = 0.0
            result['total_wl_%'] = 0.0

        # 3. Calculate Fixed-Timepoint Metrics (for each time window)
        for window_center in time_windows:
            min_window_date = baseline_date + timedelta(days=(window_center - window_span))
            max_window_date = baseline_date + timedelta(days=(window_center + window_span))
            target_date = baseline_date + timedelta(days=window_center)

            # Find measurements around the cutoff window
            measurements_around_cutoff = followup_measurements[
                (followup_measurements['measurement_date'] >= min_window_date) &
                (followup_measurements['measurement_date'] <= max_window_date)
            ]

            measurement_at_window = None
            is_dropout_at_window = True # Assume dropout unless proven otherwise

            if not measurements_around_cutoff.empty:
                # Find measurement closest to the window center
                measurements_around_cutoff = measurements_around_cutoff.copy()
                measurements_around_cutoff['distance_to_center'] = abs(
                    (measurements_around_cutoff['measurement_date'] - target_date).dt.days
                )
                closest_measurement_idx = measurements_around_cutoff['distance_to_center'].idxmin()
                measurement_at_window = measurements_around_cutoff.loc[closest_measurement_idx]
                is_dropout_at_window = False # Measurement found within/around window
            elif not followup_measurements.empty:
                 # No measurement in cutoff window, check if *any* followup exists before the window
                 last_followup_before_window = followup_measurements[followup_measurements['measurement_date'] < min_window_date]
                 if not last_followup_before_window.empty:
                      # Use the latest measurement before the window started
                      measurement_at_window = last_followup_before_window.iloc[-1]
                      # Still considered dropout *for this window* as they didn't reach it
                      is_dropout_at_window = True
                 else:
                      """!!!CHECK this logic, it might get tricky!!!"""
                      # Followup exists, but only *after* the window (unlikely but possible)
                      # Treat as dropout for this window, no relevant measurement
                      measurement_at_window = None
                      is_dropout_at_window = True
            else:
                 # Instant dropout (no followup measurements at all)
                 measurement_at_window = None
                 is_dropout_at_window = True


            # Populate results for this time window
            prefix = f"{window_center}d"
            if measurement_at_window is not None:
                result[f'{prefix}_weight_kg'] = measurement_at_window['weight_kg']
                wl_kg, wl_pct = _calculate_wl_metrics(baseline_weight, measurement_at_window['weight_kg'])
                result[f'wl_{prefix}_kg'] = wl_kg
                result[f'wl_{prefix}_%'] = wl_pct
                result[f'{prefix}_date'] = measurement_at_window['measurement_date']
                result[f'days_to_{prefix}_measurement'] = (measurement_at_window['measurement_date'] - baseline_date).days
            else:
                # No relevant measurement found for this window
                result[f'{prefix}_weight_kg'] = np.nan
                result[f'wl_{prefix}_kg'] = np.nan
                result[f'wl_{prefix}_%'] = np.nan
                result[f'{prefix}_date'] = pd.NaT
                result[f'days_to_{prefix}_measurement'] = np.nan

            result[f'{prefix}_dropout'] = is_dropout_at_window


        # 4. Calculate Time-to-Event Metrics (for each weight loss target)
        for target in weight_loss_targets:
            target_achieved = False
            first_success_measurement = None
            actual_wl_at_success = np.nan

            # Check all followup measurements for the first success
            for _, row in followup_measurements.iterrows():
                current_weight = row['weight_kg']
                if baseline_weight is not None and baseline_weight > 0:
                    current_weight_loss_pct = ((baseline_weight - current_weight) / baseline_weight) * 100
                    if round(current_weight_loss_pct, 2) >= target:
                        target_achieved = True
                        first_success_measurement = row
                        actual_wl_at_success = round(current_weight_loss_pct, 2)
                        break # Stop at the first success

            # Populate results for this target
            prefix = f"{target}%_wl"
            result[f'{prefix}_achieved'] = target_achieved
            if target_achieved and first_success_measurement is not None:
                result[f'{prefix}_%'] = actual_wl_at_success
                result[f'{prefix}_date'] = first_success_measurement['measurement_date']
                result[f'days_to_{prefix}'] = (first_success_measurement['measurement_date'] - baseline_date).days
            else:
                result[f'{prefix}_%'] = np.nan
                result[f'{prefix}_date'] = pd.NaT
                result[f'days_to_{prefix}'] = np.nan # Or perhaps total_followup_days if censored? Check analysis plan needs. NaN is safer.

        results_list.append(result)

    return pd.DataFrame(results_list)


"""
MODIFIED: MAIN ORCHESTRATION FUNCTION FOR WIDE TABLE
"""

def generate_wide_analysis_dataset(input_connection, output_connection, weight_loss_targets, time_windows, window_span=10):
    """
    Orchestrates the process to generate the single wide survival analysis dataset.
    Loads data, prepares patient baseline info, calculates all outcomes per patient,
    and saves the resulting wide DataFrame to the output database.
    """
    # 1. Load and prepare input data
    print("Loading measurements...")
    measurements = load_measurements(input_connection)
    print("Loading medical records...")
    medical_records = load_medical_records(input_connection)
    print("Preparing patient data...")
    prepared_data, filtered_measurements = prepare_patient_data(measurements, medical_records)

    if prepared_data.empty:
        print("ERROR: Prepared patient data is empty. Cannot proceed.")
        return pd.DataFrame() # Return empty DataFrame

    # 2. Calculate wide outcomes for all patients
    print("Calculating wide outcomes for all patients...")
    wide_results_df = calculate_wide_patient_outcomes(
        prepared_data,
        filtered_measurements,
        weight_loss_targets,
        time_windows,
        window_span
    )

    # 3. Save the single wide table
    if not wide_results_df.empty:
        print(f"--- Saving results to output database: {output_db_path} ---")
        print(f"Saving table: {output_table_name} ({len(wide_results_df)} rows)")
        wide_results_df.to_sql(output_table_name, output_connection, if_exists='replace', index=False)
        output_connection.commit() # Ensure changes are saved
        print("--- Wide table saved successfully ---")
    else:
        print("WARN: No results generated. Output table will be empty or not created.")

    # Removed the old summary logic based on multiple tables
    # A new summary could be generated from wide_results_df if needed

    return wide_results_df # Return the generated DataFrame

"""
EXECUTION BLOCK (Modified to call the new main function)
"""

if __name__ == "__main__":
    print("========== Generating Survival Analysis Input Dataset ==========")
    input_conn = None
    output_conn = None
    try:
        # Connect to in-and output databases
        print(f"Connecting to input database: {input_db_path}")
        if not os.path.exists(input_db_path):
             raise FileNotFoundError(f"Input database not found at {input_db_path}")
        input_conn = sqlite3.connect(input_db_path)

        print(f"Connecting to output database: {output_db_path}")
        output_conn = sqlite3.connect(output_db_path)

        # Run the new main analysis function
        wide_df = generate_wide_analysis_dataset(
            input_conn,
            output_conn,
            weight_loss_targets,
            time_windows,
            window_span
        )

        # Display basic info if successful
        if not wide_df.empty:
            print("\n--- Survival Analysis Input Table Generation Summary ---")
            print(f"Generated table '{output_table_name}' with {len(wide_df)} rows and {len(wide_df.columns)} columns.")
            # print(wide_df.head().to_string()) # Optionally print head
            print("--- End Summary ---")
        else:
            print("WARN: Analysis completed, but the resulting DataFrame is empty.")

        print(f"Analysis data saved to {output_db_path}")

    # Error handling remains the same
    except FileNotFoundError as e:
        print(f"ERROR: Database file not found - {e}")
    except sqlite3.Error as e:
        print(f"ERROR: SQLite database error - {e}")
    except ValueError as e:
        print(f"ERROR: Data processing error - {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred - {e}")
        # Consider adding traceback for debugging complex errors:
        # import traceback
        # print(traceback.format_exc())
    finally:
        # Ensure connections are closed
        print("Closing database connections...")
        if input_conn:
            input_conn.close()
        if output_conn:
            output_conn.close()
        print("========== Survival Analysis Input Data GenerationFinished ==========")

========== Starting Wide Survival Analysis Script ==========
Connecting to input database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\emotional_all_notna.sqlite
Connecting to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\shukishukishuuu.sqlite
Loading measurements...
Loading medical records...
Preparing patient data...
Calculating wide outcomes for all patients...
--- Saving results to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\shukishukishuuu.sqlite ---
Saving table: shukishukishuuu (1664 rows)
--- Wide table saved successfully ---

--- Wide Analysis Table Generation Summary ---
Generated table 'shukishukishuuu' with 1664 rows and 49 columns.
--- End Summary ---
Analysis data successfully generated and saved to C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\shukishukishuuu.sqlite
Closing database connections...
========== Wide Survival Analysis Script Finished ==========


#### Reconsidered input structure for SA 18-19 Apr v2 - revise this well, I think this is the one!

In [ ]:
# This code should be placed in a new cell in your Jupyter Notebook.
# It incorporates the requested changes into the previous wide-format script.

import pandas as pd
import os
import sqlite3
from datetime import timedelta
import numpy as np # Import numpy for NaN

"""
CONFIGURATION
"""
# Define directories and database paths - paper1_directory should be defined
# in the first cell of this notebook chapter
input_db_path = os.path.join(paper1_directory, 'emotional_all_notna.sqlite')
input_measurements = "measurements_with_metadata"
input_medical_records = "medical_records_complete"
# Modified: Output database path remains the same, but will contain a different table structure
output_db_path = os.path.join(paper1_directory, 'survival_analysis_wide_v2.sqlite') # Changed filename for new version
output_table_name = "survival_analysis_wide_v2" # Changed table name for new version

# Define analysis parameters
weight_loss_targets = [5, 10, 15]     # Weight loss target percentages
time_windows = [40, 60, 80]       # Time windows (centers) in days
window_span = 10                   # Permissible span around windows (+/- days)

# Define the variables stored in medical_records_complete that are relevant for the analysis.
# These include basic metadata like patient and record ID,
# basic factors such as age and sex,
# as well as the emotional and eating behavior variables pivotal to the research question.
# The list can be amended on demand -
# for example, right now it does not include medical record creating and closing dates.
# --- MODIFIED: Added 'dietitian_visits' ---
relevant_medical_values = ['patient_id', 'medical_record_id', 'dietitian_visits', 'sex', 'age',
                             'height_m', 'baseline_bmi', 'hunger', 'satiety', 'emotional_eating',
                             'emotional_eating_value', 'quantity_control', 'impulse_control', 'weight_gain_cause', 'genomics_sample_id']

# --- NEW: Define the desired final column order ---
# This list determines the order of columns in the final output table.
# It includes baseline info, overall followup, adherence proxies,
# fixed-time metrics, time-to-event metrics, and finally confounders/predictors.
FINAL_COLUMN_ORDER = [
    # IDs
    'patient_ID', 'medical_record_ID',
    # Followup period info and adherence proxies
    'baseline_date', 'last_aval_date', 'total_followup_days', 'nr_visits', 'nr_total_measurements', 'avg_days_between_measurements',
    # Total weight change
    'baseline_weight_kg', 'last_aval_weight_kg', 'total_wl_kg', 'total_wl_%', 'baseline_bmi', 'final_bmi', 'bmi_reduction',
    # Fixed-Timepoint Analysis (Dynamically generated columns will be inserted here by logic below)
    # Time-to-Event Analysis (Dynamically generated columns will be inserted here by logic below)
    # Confounders / Predictors (from medical records)
    'sex', 'age', 'height_m', 'hunger', 'satiety',
    'emotional_eating', 'emotional_eating_value', 'quantity_control', 'impulse_control', 'weight_gain_cause', 'genomics_sample_id'
]

# --- NEW: Dynamically insert fixed-time and time-to-event columns into FINAL_COLUMN_ORDER ---
fixed_time_cols = []
for window in time_windows:
    prefix = f"{window}d"
    fixed_time_cols.extend([
        f'{prefix}_weight_kg', f'wl_{prefix}_kg', f'wl_{prefix}_%',
        f'{prefix}_date', f'days_to_{prefix}_measurement', f'{prefix}_dropout'
    ])

time_to_event_cols = []
for target in weight_loss_targets:
    prefix = f"{target}%_wl"
    time_to_event_cols.extend([
        f'{prefix}_achieved', f'{prefix}_%', f'{prefix}_date', f'days_to_{prefix}'
    ])

# Find the insertion point (after adherence proxies)
insert_point = FINAL_COLUMN_ORDER.index('total_wl_%') + 1
# Insert the dynamic columns
FINAL_COLUMN_ORDER[insert_point:insert_point] = fixed_time_cols + time_to_event_cols


"""
DATA LOADING & PREPARATION (No changes needed here, kept for context)
"""

def load_measurements(connection):
    """
    Load measurements from the measurement_with_metadata table;
    make sure key values are in the correct format.
    """
    query = f"SELECT * FROM {input_measurements}"
    measurements = pd.read_sql_query(query, connection)
    measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'], errors='coerce')
    measurements['weight_kg'] = pd.to_numeric(measurements['weight_kg'], errors='coerce')
    # Ensure sorting for consistent 'first'/'last' operations later
    measurements = measurements.sort_values(['patient_id', 'medical_record_id', 'measurement_date'])
    return measurements

def load_medical_records(connection):
    """
    Load medical records from the medical_records_complete table;
    make sure date values are in datetime format.
    The exact columns to be used are defined in the prepare_patient_data function.
    """
    query = f"SELECT * FROM {input_medical_records}"
    medical_records = pd.read_sql_query(query, connection)
    # Ensure relevant date columns are datetime
    for col in ['medical_record_creation_date', 'baseline_measurement_date', 'final_measurement_date']:
         if col in medical_records.columns:
             medical_records[col] = pd.to_datetime(medical_records[col], errors='coerce')
    # --- NEW: Ensure dietitian_visits is numeric ---
    if 'dietitian_visits' in medical_records.columns:
        medical_records['dietitian_visits'] = pd.to_numeric(medical_records['dietitian_visits'], errors='coerce')
    return medical_records

def prepare_patient_data(measurements, medical_records):
    """
    Filter measurements to only include those from the earliest medical record for each patient.
    Merge measurements with relevant medical record data, including the pivotal eating behavior scores.
    """
    # Filter measurements to only include those from the first treatment record of each patient
    # Ensure measurements are sorted before grouping
    measurements = measurements.sort_values(['patient_id', 'measurement_date'])
    earliest_records_with_data = measurements.groupby('patient_id')['medical_record_id'].first().reset_index()

    filtered_measurements = pd.merge(
        measurements,
        earliest_records_with_data,
        on=['patient_id', 'medical_record_id'],
        how='inner'
    )
    # Ensure filtered_measurements are sorted for baseline identification
    filtered_measurements = filtered_measurements.sort_values(['patient_id', 'medical_record_id', 'measurement_date'])

    # Identify the baseline measurement row for each record (the first measurement in the filtered set)
    baseline_data_rows = filtered_measurements.groupby(['patient_id', 'medical_record_id']).first().reset_index()

    # Select only relevant columns from medical_records to merge
    # --- MODIFIED: Now includes 'dietitian_visits' if added to relevant_medical_values ---
    cols_to_select = [col for col in relevant_medical_values if col in medical_records.columns]
    medical_record_subset = medical_records[cols_to_select]

    # Merge baseline measurement info with the selected medical record data
    # Use baseline_data_rows which contains the actual first measurement details
    prepared_data = pd.merge(
        baseline_data_rows[['patient_id', 'medical_record_id', 'measurement_date', 'weight_kg']], # Get baseline date/weight from actual first measurement
        medical_record_subset,
        on=['patient_id', 'medical_record_id'],
        how='left' # Keep all baseline measurements
    )
    # Rename columns for clarity before returning
    prepared_data = prepared_data.rename(columns={'measurement_date': 'baseline_date', 'weight_kg': 'baseline_weight_kg'})

    # Add baseline_bmi from medical_records if available and not already present from measurement merge
    if 'baseline_bmi' in medical_record_subset.columns and 'baseline_bmi' not in prepared_data.columns:
         prepared_data = pd.merge(
              prepared_data,
              medical_record_subset[['patient_id', 'medical_record_id', 'baseline_bmi']],
              on=['patient_id', 'medical_record_id'],
              how='left'
         )

    return prepared_data, filtered_measurements


"""
MODIFIED: CALCULATE WEIGHT LOSS OUTCOMES FOR WIDE TABLE
"""

def _get_patient_baseline(patient_data, patient_id, medical_record_id):
    """
    Get the prepared baseline data for a patient's specific medical record.
    (Function remains largely the same, but operates on the prepared_data structure)
    """
    patient_baseline = patient_data[
        (patient_data['patient_id'] == patient_id) &
        (patient_data['medical_record_id'] == medical_record_id)
    ]
    if patient_baseline.empty:
        print(f"WARN: No baseline data found for patient {patient_id}, record {medical_record_id}. Skipping.")
        return None
    # Ensure we return a Series for consistent access
    return patient_baseline.iloc[0]

def _calculate_wl_metrics(baseline_weight, current_weight):
    """ Helper to calculate weight loss kg and % """
    if pd.isna(baseline_weight) or pd.isna(current_weight) or baseline_weight == 0:
        return np.nan, np.nan
    wl_kg = baseline_weight - current_weight
    wl_pct = (wl_kg / baseline_weight) * 100
    return wl_kg, round(wl_pct, 2)

# Removed _check_target_achievement, _determine_final_measurement, _calculate_outcome_metrics
# Their logic will be integrated into the main calculation function.

def calculate_wide_patient_outcomes(prepared_patient_data, filtered_measurements, weight_loss_targets, time_windows, window_span):
    """
    Calculate all required outcomes (fixed-time and time-to-event) for each patient
    and return a list of dictionaries, each representing a row in the wide table.
    """
    results_list = []
    # Group all measurements for the relevant (earliest) medical record per patient
    grouped_measurements = filtered_measurements.groupby(['patient_id', 'medical_record_id'])

    for (patient_id, medical_record_id), group in grouped_measurements:
        # --- NEW: Calculate total number of measurements for this group (record) ---
        num_measurements = len(group)

        # 1. Get Baseline Info
        baseline_info = _get_patient_baseline(prepared_patient_data, patient_id, medical_record_id)
        if baseline_info is None:
            continue

        baseline_date = baseline_info['baseline_date']
        baseline_weight = baseline_info['baseline_weight_kg']

        # Initialize result dictionary with baseline info AND adherence proxies
        result = {
            # IDs
            'patient_ID': patient_id, # Match Excel header
            'medical_record_ID': medical_record_id, # Match Excel header
            # Baseline
            'baseline_date': baseline_date,
            'baseline_weight_kg': baseline_weight,
            # Adherence Proxies (Initialize early)
            'nr_visits': baseline_info.get('dietitian_visits'), # Get from merged baseline data
            'nr_total_measurements': num_measurements, # Use calculated value
            'avg_days_between_measurements': np.nan, # Initialize, calculated later
            # Confounders / Predictors (Initialize early)
            'sex': baseline_info.get('sex'),
            'age': baseline_info.get('age'),
            'height_m': baseline_info.get('height_m'),
            'baseline_bmi': baseline_info.get('baseline_bmi'), # Get BMI from prepared data
            'hunger': baseline_info.get('hunger'),
            'satiety': baseline_info.get('satiety'),
            'emotional_eating': baseline_info.get('emotional_eating'),
            'emotional_eating_value': baseline_info.get('emotional_eating_value'),
            'quantity_control': baseline_info.get('quantity_control'),
            'impulse_control': baseline_info.get('impulse_control'), 
            'weight_gain_cause': baseline_info.get('weight_gain_cause'),
            'genomics_sample_id': baseline_info.get('genomics_sample_id')
        }

        # Get all measurements *after* baseline for this group
        followup_measurements = group[group['measurement_date'] > baseline_date].sort_values('measurement_date')

        # 2. Calculate Overall Follow-up Metrics
        if not followup_measurements.empty:
            last_measurement = followup_measurements.iloc[-1]
            result['last_aval_date'] = last_measurement['measurement_date']
            # --- MODIFIED: Calculate inclusive total_followup_days (+1) ---
            result['total_followup_days'] = (last_measurement['measurement_date'] - baseline_date).days + 1
            result['last_aval_weight_kg'] = last_measurement['weight_kg']
            wl_kg, wl_pct = _calculate_wl_metrics(baseline_weight, last_measurement['weight_kg'])
            result['total_wl_kg'] = wl_kg
            result['total_wl_%'] = wl_pct
        else:
            # Handle instant dropouts (only baseline measurement exists)
            result['last_aval_date'] = baseline_date
            # --- MODIFIED: Set total_followup_days to 1 for instant dropouts ---
            result['total_followup_days'] = 1
            result['last_aval_weight_kg'] = baseline_weight
            result['total_wl_kg'] = 0.0
            result['total_wl_%'] = 0.0

        # --- NEW: Calculate avg_days_between_measurements ---
        # Requires total_followup_days and nr_total_measurements
        if result['nr_total_measurements'] is not None and result['nr_total_measurements'] > 1:
             # Use the calculated inclusive total_followup_days
             total_days = result['total_followup_days']
             # Calculate average based on number of intervals (N measurements = N-1 intervals)
             # Ensure total_days is treated as the span covering N points (so N-1 intervals)
             # If total_days is 1 (instant dropout), num_measurements is 1, this condition isn't met.
             # If total_days > 1, num_measurements must be >= 2.
             result['avg_days_between_measurements'] = round( (total_days -1) / (result['nr_total_measurements'] - 1) , 2) if (result['nr_total_measurements'] - 1) > 0 else np.nan
        else:
             # If 0 or 1 measurements, average days between is undefined
             result['avg_days_between_measurements'] = np.nan


        # 3. Calculate Fixed-Timepoint Metrics (for each time window)
        for window_center in time_windows:
            min_window_date = baseline_date + timedelta(days=(window_center - window_span))
            max_window_date = baseline_date + timedelta(days=(window_center + window_span))
            target_date = baseline_date + timedelta(days=window_center)

            # Find measurements strictly *within* the cutoff window span
            measurements_around_cutoff = followup_measurements[
                (followup_measurements['measurement_date'] >= min_window_date) &
                (followup_measurements['measurement_date'] <= max_window_date)
            ]

            measurement_for_window = None # The measurement to use for this window's stats
            is_dropout_at_window = True # Assume dropout unless a measurement is found *in* the window

            if not measurements_around_cutoff.empty:
                # Measurement exists within the window span. Find the closest one.
                is_dropout_at_window = False # Found measurement in window
                measurements_around_cutoff = measurements_around_cutoff.copy()
                measurements_around_cutoff['distance_to_center'] = abs(
                    (measurements_around_cutoff['measurement_date'] - target_date).dt.days
                )
                closest_measurement_idx = measurements_around_cutoff['distance_to_center'].idxmin()
                measurement_for_window = measurements_around_cutoff.loc[closest_measurement_idx]
            # else: If measurements_around_cutoff is empty, is_dropout_at_window remains True.
            # No need to check for measurements *before* the window for populating data,
            # as per the requirement to leave fields blank for dropouts.

            # --- MODIFIED: Populate results based *strictly* on dropout status for the window ---
            prefix = f"{window_center}d"
            result[f'{prefix}_dropout'] = is_dropout_at_window # Set dropout status first

            if is_dropout_at_window:
                # If dropout for this window, set all related metrics to NaN/NaT
                result[f'{prefix}_weight_kg'] = np.nan
                result[f'wl_{prefix}_kg'] = np.nan
                result[f'wl_{prefix}_%'] = np.nan
                result[f'{prefix}_date'] = pd.NaT
                result[f'days_to_{prefix}_measurement'] = np.nan
            else:
                # If NOT dropout, populate metrics using the found measurement_for_window
                # (This block only runs if measurement_for_window is not None)
                result[f'{prefix}_weight_kg'] = measurement_for_window['weight_kg']
                wl_kg, wl_pct = _calculate_wl_metrics(baseline_weight, measurement_for_window['weight_kg'])
                result[f'wl_{prefix}_kg'] = wl_kg
                result[f'wl_{prefix}_%'] = wl_pct
                result[f'{prefix}_date'] = measurement_for_window['measurement_date']
                # Calculate days from baseline to this specific measurement
                result[f'days_to_{prefix}_measurement'] = (measurement_for_window['measurement_date'] - baseline_date).days + 1 # Inclusive days


        # 4. Calculate Time-to-Event Metrics (for each weight loss target)
        # (No changes needed in this section based on discussion)
        for target in weight_loss_targets:
            target_achieved = False
            first_success_measurement = None
            actual_wl_at_success = np.nan

            # Check all followup measurements for the first success
            for _, row in followup_measurements.iterrows():
                current_weight = row['weight_kg']
                if baseline_weight is not None and baseline_weight > 0:
                    current_weight_loss_pct = ((baseline_weight - current_weight) / baseline_weight) * 100
                    if round(current_weight_loss_pct, 2) >= target:
                        target_achieved = True
                        first_success_measurement = row
                        actual_wl_at_success = round(current_weight_loss_pct, 2)
                        break # Stop at the first success

            # Populate results for this target
            prefix = f"{target}%_wl"
            result[f'{prefix}_achieved'] = target_achieved
            if target_achieved and first_success_measurement is not None:
                result[f'{prefix}_%'] = actual_wl_at_success
                result[f'{prefix}_date'] = first_success_measurement['measurement_date']
                # Calculate inclusive days to achieve target
                result[f'days_to_{prefix}'] = (first_success_measurement['measurement_date'] - baseline_date).days + 1
            else:
                result[f'{prefix}_%'] = np.nan
                result[f'{prefix}_date'] = pd.NaT
                result[f'days_to_{prefix}'] = np.nan

        results_list.append(result)

    return pd.DataFrame(results_list)


"""
MODIFIED: MAIN ORCHESTRATION FUNCTION FOR WIDE TABLE
"""

def generate_wide_analysis_dataset(input_connection, output_connection, weight_loss_targets, time_windows, window_span=10):
    """
    Orchestrates the process to generate the single wide survival analysis dataset.
    Loads data, prepares patient baseline info, calculates all outcomes per patient,
    reorders columns, and saves the resulting wide DataFrame to the output database.
    """
    # 1. Load and prepare input data
    print("Loading measurements...")
    measurements = load_measurements(input_connection)
    print("Loading medical records...")
    medical_records = load_medical_records(input_connection)
    print("Preparing patient data...")
    prepared_data, filtered_measurements = prepare_patient_data(measurements, medical_records)

    if prepared_data.empty:
        print("ERROR: Prepared patient data is empty. Cannot proceed.")
        return pd.DataFrame() # Return empty DataFrame

    # 2. Calculate wide outcomes for all patients
    print("Calculating wide outcomes for all patients...")
    wide_results_df = calculate_wide_patient_outcomes(
        prepared_data,
        filtered_measurements,
        weight_loss_targets,
        time_windows,
        window_span
    )

    # --- NEW: Reorder columns according to FINAL_COLUMN_ORDER defined in config ---
    if not wide_results_df.empty:
        print("Reordering columns...")
        # Ensure all columns in FINAL_COLUMN_ORDER exist in the DataFrame, handle potential missing ones
        final_columns_present = [col for col in FINAL_COLUMN_ORDER if col in wide_results_df.columns]
        missing_cols = [col for col in FINAL_COLUMN_ORDER if col not in wide_results_df.columns]
        if missing_cols:
             print(f"WARN: The following columns defined in FINAL_COLUMN_ORDER were not found in the generated data and will be skipped: {missing_cols}")
        # Add any columns present in DataFrame but not in FINAL_COLUMN_ORDER to the end, just in case
        extra_cols = [col for col in wide_results_df.columns if col not in final_columns_present]
        if extra_cols:
             print(f"WARN: The following columns were generated but not included in FINAL_COLUMN_ORDER; they will be added to the end: {extra_cols}")

        wide_results_df = wide_results_df[final_columns_present + extra_cols]


    # 3. Save the single wide table
    if not wide_results_df.empty:
        print(f"--- Saving results to output database: {output_db_path} ---")
        print(f"Saving table: {output_table_name} ({len(wide_results_df)} rows)")
        wide_results_df.to_sql(output_table_name, output_connection, if_exists='replace', index=False)
        output_connection.commit() # Ensure changes are saved
        print("--- Wide table saved successfully ---")
    else:
        print("WARN: No results generated. Output table will be empty or not created.")

    # Removed the old summary logic based on multiple tables
    # A new summary could be generated from wide_results_df if needed

    return wide_results_df # Return the generated DataFrame

"""
EXECUTION BLOCK (Modified to call the new main function and use new output names)
"""

if __name__ == "__main__":
    print("========== Generating Survival Analysis Input Dataset (Wide Format v2) ==========") # Updated title
    input_conn = None
    output_conn = None
    try:
        # Connect to in-and output databases
        print(f"Connecting to input database: {input_db_path}")
        if not os.path.exists(input_db_path):
             raise FileNotFoundError(f"Input database not found at {input_db_path}")
        input_conn = sqlite3.connect(input_db_path)

        print(f"Connecting to output database: {output_db_path}")
        output_conn = sqlite3.connect(output_db_path)

        # Run the new main analysis function
        wide_df = generate_wide_analysis_dataset(
            input_conn,
            output_conn,
            weight_loss_targets,
            time_windows,
            window_span
        )

        # Display basic info if successful
        if not wide_df.empty:
            print("\n--- Survival Analysis Input Table Generation Summary ---")
            print(f"Generated table '{output_table_name}' with {len(wide_df)} rows and {len(wide_df.columns)} columns.")
            # print(wide_df.head().to_string()) # Optionally print head
            print("--- End Summary ---")
        else:
            print("WARN: Analysis completed, but the resulting DataFrame is empty.")

        print(f"Analysis data saved to {output_db_path}")

    # Error handling remains the same
    except FileNotFoundError as e:
        print(f"ERROR: Database file not found - {e}")
    except sqlite3.Error as e:
        print(f"ERROR: SQLite database error - {e}")
    except ValueError as e:
        print(f"ERROR: Data processing error - {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred - {e}")
        # Consider adding traceback for debugging complex errors:
        # import traceback
        # print(traceback.format_exc())
    finally:
        # Ensure connections are closed
        print("Closing database connections...")
        if input_conn:
            input_conn.close()
        if output_conn:
            output_conn.close()
        print("========== Survival Analysis Input Data Generation Finished (Wide Format v2) ==========") # Updated title


========== Generating Survival Analysis Input Dataset (Wide Format v3) ==========
Connecting to input database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\emotional_all_notna.sqlite
Connecting to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis_wide_v3.sqlite
Loading measurements...
Loading medical records...
Preparing patient data...
Calculating wide outcomes for all patients...
Reordering columns...
--- Saving results to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis_wide_v3.sqlite ---
Saving table: survival_analysis_wide_v3 (1664 rows)
--- Wide table saved successfully ---

--- Survival Analysis Input Table Generation Summary ---
Generated table 'survival_analysis_wide_v3' with 1664 rows and 56 columns.
--- End Summary ---
Analysis data saved to C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis_wide_v3.sqlite
Closing database connecti

#### Actually, this may be the one, Apr 20 v3 - added final BMI too, and fetching BMI from measurements instead of medical records

In [18]:
####
# filepath: c:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\paper1_script.ipynb
# This code should be placed in a new cell in your Jupyter Notebook.
# It incorporates the requested changes for BMI handling into the wide-format script.

import pandas as pd
import os
import sqlite3
from datetime import timedelta
import numpy as np # Import numpy for NaN

"""
CONFIGURATION
"""
# Define directories and database paths - paper1_directory should be defined
# in the first cell of this notebook chapter
input_db_path = os.path.join(paper1_directory, 'emotional_all_notna.sqlite')
input_measurements = "measurements_with_metadata"
input_medical_records = "medical_records_complete"
# Output database path and table name
output_db_path = os.path.join(paper1_directory, 'survival_analysis.sqlite') # Changed filename for new version
output_table_name = "sa_input_table" # Changed table name for new version

# Define analysis parameters
weight_loss_targets = [5, 10, 15]     # Weight loss target percentages
time_windows = [40, 60, 80]       # Time windows (centers) in days
window_span = 10                   # Permissible span around windows (+/- days)

# Define the variables stored in medical_records_complete that are relevant for the analysis.
# --- MODIFIED: Removed 'baseline_bmi' as it will be sourced from measurements ---
relevant_medical_values = ['patient_id', 'medical_record_id', 'dietitian_visits', 'sex_f', 'age_when_creating_record',
                             'height_m', 'hunger_yn', 'satiety_yn', 'emotional_eating_yn', 
                             'emotional_eating_value_likert', 'quantity_control_likert', 'impulse_control_likert', 'weight_gain_cause', 'genomics_sample_id']

# --- MODIFIED: Renamed to output_column_order and updated for BMI ---
# This list determines the order of columns in the final output table.
output_column_order = [
    # IDs
    'patient_ID', 'medical_record_ID',
    # Followup period info and adherence proxies
    'baseline_date', 'last_aval_date', 'total_followup_days', 'nr_visits', 'nr_total_measurements', 'avg_days_between_measurements',
    # Total weight change & BMI change
    'baseline_weight_kg', 'last_aval_weight_kg', 'total_wl_kg', 'total_wl_%',
    'baseline_bmi', 'final_bmi', 'bmi_reduction', # Added BMI columns here
    # Fixed-Timepoint Analysis (Dynamically generated columns will be inserted here by logic below)
    # Time-to-Event Analysis (Dynamically generated columns will be inserted here by logic below)
    # Confounders / Predictors (from medical records)
    'sex_f', 'age', 'height_m', 'hunger_yn', 'satiety_yn', 'emotional_eating_yn', 
    'emotional_eating_value_likert', 'quantity_control_likert', 'impulse_control_likert', 'weight_gain_cause', 'genomics_sample_id'
]

# --- MODIFIED: Dynamically insert fixed-time and time-to-event columns into output_column_order ---
fixed_time_cols = []
for window in time_windows:
    prefix = f"{window}d"
    # --- MODIFIED: Excluded BMI calculation for fixed timepoints ---
    fixed_time_cols.extend([
        f'{prefix}_weight_kg', f'wl_{prefix}_kg', f'wl_{prefix}_%',
        f'{prefix}_date', f'days_to_{prefix}_measurement', f'{prefix}_dropout'
        # Removed: f'{prefix}_bmi'
    ])

time_to_event_cols = []
for target in weight_loss_targets:
    prefix = f"{target}%_wl"
    time_to_event_cols.extend([
        f'{prefix}_achieved', f'{prefix}_%', f'{prefix}_date', f'days_to_{prefix}'
    ])

# Find the insertion point (after BMI reduction)
# --- MODIFIED: Insertion point updated ---
insert_point = output_column_order.index('bmi_reduction') + 1
# Insert the dynamic columns
output_column_order[insert_point:insert_point] = fixed_time_cols + time_to_event_cols


"""
DATA LOADING & PREPARATION
"""

def load_measurements(connection):
    """
    Load measurements from the measurement_with_metadata table;
    make sure key values are in the correct format.
    --- MODIFIED: Ensure BMI is numeric ---
    """
    query = f"SELECT * FROM {input_measurements}"
    measurements = pd.read_sql_query(query, connection)
    measurements['measurement_date'] = pd.to_datetime(measurements['measurement_date'], errors='coerce')
    measurements['weight_kg'] = pd.to_numeric(measurements['weight_kg'], errors='coerce')
    # --- NEW: Ensure BMI is numeric ---
    measurements['bmi'] = pd.to_numeric(measurements['bmi'], errors='coerce')
    # Ensure sorting for consistent 'first'/'last' operations later
    measurements = measurements.sort_values(['patient_id', 'medical_record_id', 'measurement_date'])
    return measurements

def load_medical_records(connection):
    """
    Load medical records from the medical_records_complete table;
    make sure date values are in datetime format.
    The exact columns to be used are defined in the prepare_patient_data function.
    """
    query = f"SELECT * FROM {input_medical_records}"
    medical_records = pd.read_sql_query(query, connection)
    # Ensure relevant date columns are datetime
    for col in ['medical_record_creation_date', 'baseline_measurement_date', 'final_measurement_date']:
         if col in medical_records.columns:
             medical_records[col] = pd.to_datetime(medical_records[col], errors='coerce')
    # Ensure dietitian_visits is numeric
    if 'dietitian_visits' in medical_records.columns:
        medical_records['dietitian_visits'] = pd.to_numeric(medical_records['dietitian_visits'], errors='coerce')
    return medical_records

def prepare_patient_data(measurements, medical_records):
    """
    Filter measurements to only include those from the earliest medical record for each patient.
    Merge measurements with relevant medical record data, including the pivotal eating behavior scores.
    --- MODIFIED: Fetch baseline BMI from measurements ---
    """
    # Filter measurements to only include those from the first treatment record of each patient
    measurements = measurements.sort_values(['patient_id', 'measurement_date'])
    earliest_records_with_data = measurements.groupby('patient_id')['medical_record_id'].first().reset_index()

    filtered_measurements = pd.merge(
        measurements,
        earliest_records_with_data,
        on=['patient_id', 'medical_record_id'],
        how='inner'
    )
    filtered_measurements = filtered_measurements.sort_values(['patient_id', 'medical_record_id', 'measurement_date'])

    # Identify the baseline measurement row for each record (the first measurement in the filtered set)
    # --- MODIFIED: Include 'bmi' ---
    baseline_data_rows = filtered_measurements.groupby(['patient_id', 'medical_record_id']).first().reset_index()

    # Select only relevant columns from medical_records to merge
    cols_to_select = [col for col in relevant_medical_values if col in medical_records.columns]
    medical_record_subset = medical_records[cols_to_select]

    # Merge baseline measurement info with the selected medical record data
    # --- MODIFIED: Include 'bmi' from baseline_data_rows ---
    prepared_data = pd.merge(
        baseline_data_rows[['patient_id', 'medical_record_id', 'measurement_date', 'weight_kg', 'bmi']], # Added 'bmi'
        medical_record_subset,
        on=['patient_id', 'medical_record_id'],
        how='left' # Keep all baseline measurements
    )
    # Rename columns for clarity before returning
    # --- MODIFIED: Rename 'bmi' to 'baseline_bmi' ---
    prepared_data = prepared_data.rename(columns={
        'measurement_date': 'baseline_date',
        'weight_kg': 'baseline_weight_kg',
        'bmi': 'baseline_bmi' # Rename BMI from measurement
    })

    # --- REMOVED: Separate merge for baseline_bmi from medical_records is no longer needed ---
    # if 'baseline_bmi' in medical_record_subset.columns and 'baseline_bmi' not in prepared_data.columns:
    #      ... (old merge logic removed) ...

    return prepared_data, filtered_measurements


"""
MODIFIED: CALCULATE WEIGHT LOSS OUTCOMES FOR WIDE TABLE
"""

def _get_patient_baseline(patient_data, patient_id, medical_record_id):
    """
    Get the prepared baseline data for a patient's specific medical record.
    """
    patient_baseline = patient_data[
        (patient_data['patient_id'] == patient_id) &
        (patient_data['medical_record_id'] == medical_record_id)
    ]
    if patient_baseline.empty:
        print(f"WARN: No baseline data found for patient {patient_id}, record {medical_record_id}. Skipping.")
        return None
    return patient_baseline.iloc[0]

def _calculate_wl_metrics(baseline_weight, current_weight):
    """ Helper to calculate weight loss kg and % """
    if pd.isna(baseline_weight) or pd.isna(current_weight) or baseline_weight == 0:
        return np.nan, np.nan
    wl_kg = baseline_weight - current_weight
    wl_pct = (wl_kg / baseline_weight) * 100
    return wl_kg, round(wl_pct, 2)

# --- NEW: Helper to calculate BMI reduction ---
def _calculate_bmi_reduction(baseline_bmi, final_bmi):
    """ Helper to calculate BMI reduction (final - baseline) """
    if pd.isna(baseline_bmi) or pd.isna(final_bmi):
        return np.nan
    return round(final_bmi - baseline_bmi, 2)


def calculate_wide_patient_outcomes(prepared_patient_data, filtered_measurements, weight_loss_targets, time_windows, window_span):
    """
    Calculate all required outcomes (fixed-time and time-to-event) for each patient
    and return a list of dictionaries, each representing a row in the wide table.
    --- MODIFIED: Incorporates BMI from measurements and BMI reduction ---
    """
    results_list = []
    grouped_measurements = filtered_measurements.groupby(['patient_id', 'medical_record_id'])

    for (patient_id, medical_record_id), group in grouped_measurements:
        num_measurements = len(group)

        # 1. Get Baseline Info
        baseline_info = _get_patient_baseline(prepared_patient_data, patient_id, medical_record_id)
        if baseline_info is None:
            continue

        baseline_date = baseline_info['baseline_date']
        baseline_weight = baseline_info['baseline_weight_kg']
        # --- MODIFIED: Get baseline_bmi from prepared_data (sourced from measurement) ---
        baseline_bmi = baseline_info.get('baseline_bmi') # Use .get() for safety

        # Initialize result dictionary
        result = {
            # IDs
            'patient_ID': patient_id,
            'medical_record_ID': medical_record_id,
            # Baseline
            'baseline_date': baseline_date,
            'baseline_weight_kg': baseline_weight,
            'baseline_bmi': baseline_bmi, # Add baseline BMI here
            # Adherence Proxies
            'nr_visits': baseline_info.get('dietitian_visits'),
            'nr_total_measurements': num_measurements,
            'avg_days_between_measurements': np.nan,
            # Confounders / Predictors
            'sex_f': baseline_info.get('sex_f'),
            'age': baseline_info.get('age_when_creating_record'),
            'height_m': baseline_info.get('height_m'),
            'hunger_yn': baseline_info.get('hunger_yn'),
            'satiety_yn': baseline_info.get('satiety_yn'),
            'emotional_eating_yn': baseline_info.get('emotional_eating_yn'),
            'emotional_eating_value_likert': baseline_info.get('emotional_eating_value_likert'),
            'quantity_control_likert': baseline_info.get('quantity_control_likert'),
            'impulse_control_likert': baseline_info.get('impulse_control_likert'),
            'weight_gain_cause': baseline_info.get('weight_gain_cause'),
            'genomics_sample_id': baseline_info.get('genomics_sample_id')
        }

        followup_measurements = group[group['measurement_date'] > baseline_date].sort_values('measurement_date')

        # 2. Calculate Overall Follow-up Metrics
        if not followup_measurements.empty:
            last_measurement = followup_measurements.iloc[-1]
            result['last_aval_date'] = last_measurement['measurement_date']
            result['total_followup_days'] = (last_measurement['measurement_date'] - baseline_date).days + 1
            result['last_aval_weight_kg'] = last_measurement['weight_kg']
            wl_kg, wl_pct = _calculate_wl_metrics(baseline_weight, last_measurement['weight_kg'])
            result['total_wl_kg'] = wl_kg
            result['total_wl_%'] = wl_pct
            # --- NEW: Get final BMI and calculate reduction ---
            result['final_bmi'] = last_measurement.get('bmi') # Get BMI from last measurement
            result['bmi_reduction'] = _calculate_bmi_reduction(result['baseline_bmi'], result['final_bmi'])
        else:
            # Handle instant dropouts
            result['last_aval_date'] = baseline_date
            result['total_followup_days'] = 1
            result['last_aval_weight_kg'] = baseline_weight
            result['total_wl_kg'] = 0.0
            result['total_wl_%'] = 0.0
            # --- NEW: Set final BMI and reduction for instant dropouts ---
            result['final_bmi'] = result['baseline_bmi'] # Final BMI is baseline BMI
            result['bmi_reduction'] = 0.0 # No change

        # Calculate avg_days_between_measurements
        if result['nr_total_measurements'] is not None and result['nr_total_measurements'] > 1:
             total_days = result['total_followup_days']
             result['avg_days_between_measurements'] = round( (total_days -1) / (result['nr_total_measurements'] - 1) , 2) if (result['nr_total_measurements'] - 1) > 0 else np.nan
        else:
             result['avg_days_between_measurements'] = np.nan


        # 3. Calculate Fixed-Timepoint Metrics (for each time window)
        # --- MODIFIED: Excluded BMI calculation ---
        for window_center in time_windows:
            min_window_date = baseline_date + timedelta(days=(window_center - window_span))
            max_window_date = baseline_date + timedelta(days=(window_center + window_span))
            target_date = baseline_date + timedelta(days=window_center)

            measurements_around_cutoff = followup_measurements[
                (followup_measurements['measurement_date'] >= min_window_date) &
                (followup_measurements['measurement_date'] <= max_window_date)
            ]

            measurement_for_window = None
            is_dropout_at_window = True

            if not measurements_around_cutoff.empty:
                is_dropout_at_window = False
                measurements_around_cutoff = measurements_around_cutoff.copy()
                measurements_around_cutoff['distance_to_center'] = abs(
                    (measurements_around_cutoff['measurement_date'] - target_date).dt.days
                )
                closest_measurement_idx = measurements_around_cutoff['distance_to_center'].idxmin()
                measurement_for_window = measurements_around_cutoff.loc[closest_measurement_idx]

            prefix = f"{window_center}d"
            result[f'{prefix}_dropout'] = is_dropout_at_window

            if is_dropout_at_window:
                result[f'{prefix}_weight_kg'] = np.nan
                result[f'wl_{prefix}_kg'] = np.nan
                result[f'wl_{prefix}_%'] = np.nan
                result[f'{prefix}_date'] = pd.NaT
                result[f'days_to_{prefix}_measurement'] = np.nan
                # Removed: result[f'{prefix}_bmi'] = np.nan
            else:
                result[f'{prefix}_weight_kg'] = measurement_for_window['weight_kg']
                wl_kg, wl_pct = _calculate_wl_metrics(baseline_weight, measurement_for_window['weight_kg'])
                result[f'wl_{prefix}_kg'] = wl_kg
                result[f'wl_{prefix}_%'] = wl_pct
                result[f'{prefix}_date'] = measurement_for_window['measurement_date']
                result[f'days_to_{prefix}_measurement'] = (measurement_for_window['measurement_date'] - baseline_date).days + 1
                # Removed: result[f'{prefix}_bmi'] = measurement_for_window.get('bmi')


        # 4. Calculate Time-to-Event Metrics (for each weight loss target)
        # (No changes needed in this section)
        for target in weight_loss_targets:
            target_achieved = False
            first_success_measurement = None
            actual_wl_at_success = np.nan

            for _, row in followup_measurements.iterrows():
                current_weight = row['weight_kg']
                if baseline_weight is not None and baseline_weight > 0:
                    current_weight_loss_pct = ((baseline_weight - current_weight) / baseline_weight) * 100
                    if round(current_weight_loss_pct, 2) >= target:
                        target_achieved = True
                        first_success_measurement = row
                        actual_wl_at_success = round(current_weight_loss_pct, 2)
                        break

            prefix = f"{target}%_wl"
            result[f'{prefix}_achieved'] = target_achieved
            if target_achieved and first_success_measurement is not None:
                result[f'{prefix}_%'] = actual_wl_at_success
                result[f'{prefix}_date'] = first_success_measurement['measurement_date']
                result[f'days_to_{prefix}'] = (first_success_measurement['measurement_date'] - baseline_date).days + 1
            else:
                result[f'{prefix}_%'] = np.nan
                result[f'{prefix}_date'] = pd.NaT
                result[f'days_to_{prefix}'] = np.nan

        results_list.append(result)

    return pd.DataFrame(results_list)


"""
MODIFIED: MAIN ORCHESTRATION FUNCTION FOR WIDE TABLE
"""

def generate_wide_analysis_dataset(input_connection, output_connection, weight_loss_targets, time_windows, window_span=10):
    """
    Orchestrates the process to generate the single wide survival analysis dataset.
    Loads data, prepares patient baseline info, calculates all outcomes per patient,
    reorders columns, and saves the resulting wide DataFrame to the output database.
    --- MODIFIED: Uses output_column_order ---
    """
    # 1. Load and prepare input data
    print("Loading measurements...")
    measurements = load_measurements(input_connection)
    print("Loading medical records...")
    medical_records = load_medical_records(input_connection)
    print("Preparing patient data...")
    prepared_data, filtered_measurements = prepare_patient_data(measurements, medical_records)

    if prepared_data.empty:
        print("ERROR: Prepared patient data is empty. Cannot proceed.")
        return pd.DataFrame()

    # 2. Calculate wide outcomes for all patients
    print("Calculating wide outcomes for all patients...")
    wide_results_df = calculate_wide_patient_outcomes(
        prepared_data,
        filtered_measurements,
        weight_loss_targets,
        time_windows,
        window_span
    )

    # --- MODIFIED: Reorder columns according to output_column_order ---
    if not wide_results_df.empty:
        print("Reordering columns...")
        # Ensure all columns in output_column_order exist in the DataFrame
        final_columns_present = [col for col in output_column_order if col in wide_results_df.columns]
        missing_cols = [col for col in output_column_order if col not in wide_results_df.columns]
        if missing_cols:
             print(f"WARN: The following columns defined in output_column_order were not found and will be skipped: {missing_cols}")
        # Add any extra columns not in the defined order to the end
        extra_cols = [col for col in wide_results_df.columns if col not in final_columns_present]
        if extra_cols:
             print(f"WARN: The following columns were generated but not in output_column_order; adding to the end: {extra_cols}")

        wide_results_df = wide_results_df[final_columns_present + extra_cols]


    # 3. Save the single wide table
    if not wide_results_df.empty:
        print(f"--- Saving results to output database: {output_db_path} ---")
        print(f"Saving table: {output_table_name} ({len(wide_results_df)} rows)")
        wide_results_df.to_sql(output_table_name, output_connection, if_exists='replace', index=False)
        output_connection.commit()
        print("--- Wide table saved successfully ---")
    else:
        print("WARN: No results generated. Output table will be empty or not created.")

    return wide_results_df


"""
EXECUTION BLOCK (Modified to use new output names)
"""

if __name__ == "__main__":
    print("========== Generating Survival Analysis Input Dataset (Wide Format v3) ==========") # Updated title
    input_conn = None
    output_conn = None
    try:
        print(f"Connecting to input database: {input_db_path}")
        if not os.path.exists(input_db_path):
             raise FileNotFoundError(f"Input database not found at {input_db_path}")
        input_conn = sqlite3.connect(input_db_path)

        print(f"Connecting to output database: {output_db_path}")
        output_conn = sqlite3.connect(output_db_path)

        wide_df = generate_wide_analysis_dataset(
            input_conn,
            output_conn,
            weight_loss_targets,
            time_windows,
            window_span
        )

        if not wide_df.empty:
            print("\n--- Survival Analysis Input Table Generation Summary ---")
            print(f"Generated table '{output_table_name}' with {len(wide_df)} rows and {len(wide_df.columns)} columns.")
            # print(wide_df.head().to_string())
            print("--- End Summary ---")
        else:
            print("WARN: Analysis completed, but the resulting DataFrame is empty.")

        print(f"Analysis data saved to {output_db_path}")

    except FileNotFoundError as e:
        print(f"ERROR: Database file not found - {e}")
    except sqlite3.Error as e:
        print(f"ERROR: SQLite database error - {e}")
    except ValueError as e:
        print(f"ERROR: Data processing error - {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred - {e}")
        # import traceback
        # print(traceback.format_exc())
    finally:
        print("Closing database connections...")
        if input_conn:
            input_conn.close()
        if output_conn:
            output_conn.close()
        print("========== Survival Analysis Input Data Generation Finished (Wide Format v3) ==========") # Updated title


========== Generating Survival Analysis Input Dataset (Wide Format v3) ==========
Connecting to input database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\emotional_all_notna.sqlite
Connecting to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite
Loading measurements...
Loading medical records...
Preparing patient data...
Calculating wide outcomes for all patients...
Reordering columns...
--- Saving results to output database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite ---
Saving table: sa_input_table (1664 rows)
--- Wide table saved successfully ---

--- Survival Analysis Input Table Generation Summary ---
Generated table 'sa_input_table' with 1664 rows and 56 columns.
--- End Summary ---
Analysis data saved to C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite
Closing database connections...
========== Survival Analysis Input Data

### Data analysis

#### Generation of summary stats tables in the database - the code is not revised, the output is draft-level, but it is already insightful and seems correct. 

In [19]:
# This code calculates detailed population and outcome summary statistics,
# ensures variables are columns and stats are rows in the final output,
# fixes categorical counts (both population and outcome),
# and saves them to the specified table names in the SQLite database.

import pandas as pd
import sqlite3
import os
import numpy as np

"""
CONFIGURATION
"""
# Define database path, input table name, and output table names
# Ensure 'paper1_directory' is defined in your notebook environment before running this cell
if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
     raise NameError("'paper1_directory' is not defined. Please define it in a previous cell.")

db_path = os.path.join(paper1_directory, 'survival_analysis.sqlite') # DB with the final wide table
input_table_name = "sa_input_table" # The wide table created previously
output_pop_summary_table = "population_summary"
output_outcome_summary_table = "outcome_summary"

# Define dynamic parameters (should match those used to create sa_input_table)
weight_loss_targets = [5, 10, 15]
time_windows = [40, 60, 80]

"""
HELPER FUNCTIONS
"""

def format_mean_sd(series, decimals=1):
    """Calculates mean and SD, returns formatted string 'mean ± SD'."""
    numeric_series = pd.to_numeric(series, errors='coerce')
    if numeric_series.isnull().all() or numeric_series.empty:
        return np.nan
    mean = numeric_series.mean()
    std = numeric_series.std()
    if pd.isna(mean) or pd.isna(std):
         return np.nan
    return f"{mean:.{decimals}f} ± {std:.{decimals}f}"

def format_n_percent(series, condition_value, total_n, decimals=1):
    """Calculates N and % matching a condition (primarily for strings),
       returns formatted string 'N (X.X%)'. Case-insensitive for strings.
    """
    if total_n == 0:
        return "0 (NaN%)"

    # Primarily designed for string comparison now
    condition_str = str(condition_value).lower()
    try:
        # Convert series to string, strip whitespace, convert to lower case, and compare
        condition_mask = series.astype(str).str.strip().str.lower().eq(condition_str)
        n = condition_mask.sum()
    except Exception as e: # Broad exception catch if string methods fail
        print(f"  Warning: Could not perform string comparison for series '{series.name}' with value '{condition_value}'. Error: {e}. Falling back to direct comparison.")
        # Fallback for non-string types or errors during string conversion
        try:
            condition_mask = (series == condition_value)
            n = condition_mask.sum()
        except TypeError:
             print(f"  Error: Type error during fallback comparison for series '{series.name}' with value '{condition_value}'. Setting N to 0.")
             n = 0
        except Exception as e_fallback:
             print(f"  Error: Unexpected error during fallback comparison for series '{series.name}' with value '{condition_value}'. Error: {e_fallback}. Setting N to 0.")
             n = 0

    percent = (n / total_n) * 100 if total_n > 0 else 0
    return f"{int(n)} ({percent:.{decimals}f}%)" # Ensure n is int for formatting


def get_describe_stats(df, columns):
    """Runs describe() and extracts key stats for specified columns."""
    actual_cols = [col for col in columns if col in df.columns]
    if not actual_cols:
        return pd.DataFrame()
    numeric_subset = df[actual_cols].select_dtypes(include=np.number)
    if numeric_subset.empty:
        # print(f"  Warning: No numeric columns found among {actual_cols} for describe.") # Less verbose
        return pd.DataFrame()
    described = numeric_subset.describe(percentiles=[.25, .5, .75]).transpose()
    stats_to_keep = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
    described = described[[col for col in stats_to_keep if col in described.columns]]
    return described

"""
SUMMARY TABLE GENERATION FUNCTIONS
"""

def generate_population_summary(df):
    """Generates the population summary data (Variables as index, Stats as columns)."""
    # NOTE: This function returns the data BEFORE transposition.
    if df.empty:
        print("Input DataFrame is empty. Cannot generate population summary.")
        return pd.DataFrame()

    total_n = len(df)
    summary_data_by_var = {}
    print(f"Generating population summary data (N={total_n})...")

    def add_stats(var_name, stats_dict):
        if var_name not in summary_data_by_var:
            summary_data_by_var[var_name] = {}
        summary_data_by_var[var_name].update(stats_dict)

    add_stats('Total Population', {'N': total_n})

    # --- Numerical Summaries ---
    numeric_cols_pop = [
        'age', 'height_m', 'baseline_weight_kg', 'last_aval_weight_kg',
        'baseline_bmi', 'final_bmi',
        'emotional_eating_value_likert', 'quantity_control_likert', 'impulse_control_likert'
    ]
    pop_described = get_describe_stats(df, numeric_cols_pop)
    for col in pop_described.index:
        stats_for_col = {}
        stats_for_col['N'] = int(pop_described.loc[col, 'count']) if 'count' in pop_described.columns else np.nan
        stats_for_col['Mean (SD)'] = format_mean_sd(df[col])
        if '50%' in pop_described.columns and '25%' in pop_described.columns and '75%' in pop_described.columns:
             median, iqr_25, iqr_75 = pop_described.loc[col, ['50%', '25%', '75%']]
             if not pd.isna([median, iqr_25, iqr_75]).any():
                  stats_for_col['Median (IQR)'] = f"{median:.1f} ({iqr_25:.1f}-{iqr_75:.1f})"
             else: stats_for_col['Median (IQR)'] = np.nan
        else: stats_for_col['Median (IQR)'] = np.nan
        if 'min' in pop_described.columns and 'max' in pop_described.columns:
             min_val, max_val = pop_described.loc[col, ['min', 'max']]
             if not pd.isna([min_val, max_val]).any():
                  stats_for_col['Min - Max'] = f"{min_val:.1f} - {max_val:.1f}"
             else: stats_for_col['Min - Max'] = np.nan
        else: stats_for_col['Min - Max'] = np.nan
        add_stats(col, stats_for_col)

    # --- Categorical/Boolean Summaries ---
    # Sex ('Female') - Use format_n_percent for string check
    if 'sex_f' in df.columns:
        add_stats('Sex: Female', {'N (%)': format_n_percent(df['sex_f'], '1', total_n)})
    else: print("  Warning: 'sex_f' column not found.")

    # Yes/No Questions - Use format_n_percent for string check
    for col_name, display_name in [('hunger_yn', 'Hunger: Yes'), ('satiety_yn', 'Satiety: Yes'), ('emotional_eating_yn', 'Emotional Eating: Yes')]:
        if col_name in df.columns:
             add_stats(display_name, {'N (%)': format_n_percent(df[col_name], '1', total_n)})
        else: print(f"  Warning: '{col_name}' column not found.")

    # Availability Checks (Not NULL) - Direct calculation
    for col_name, display_name in [('weight_gain_cause', 'Weight Gain Cause Available'), ('genomics_sample_id', 'Genomics Sample ID Available')]:
        if col_name in df.columns:
            n_not_null = df[col_name].notna().sum()
            percent_not_null = (n_not_null / total_n) * 100 if total_n > 0 else 0
            add_stats(display_name, {'N (%)': f"{n_not_null} ({percent_not_null:.1f}%)"})
        else: print(f"  Warning: '{col_name}' column not found.")

    # Convert dictionary to DataFrame (Variables as index, Stats as columns)
    summary_df = pd.DataFrame.from_dict(summary_data_by_var, orient='index')
    summary_df.index.name = 'Variable'

    # Reorder STAT columns (optional, done before transpose)
    desired_col_order = ['N', 'N (%)', 'Mean (SD)', 'Median (IQR)', 'Min - Max']
    existing_cols_ordered = [col for col in desired_col_order if col in summary_df.columns]
    remaining_cols = [col for col in summary_df.columns if col not in existing_cols_ordered]
    final_col_order = existing_cols_ordered + remaining_cols
    summary_df = summary_df[final_col_order]

    return summary_df


def generate_outcome_summary(df, weight_targets, time_windows_list):
    """Generates the outcome summary data (Variables as index, Stats as columns)."""
    # NOTE: This function returns the data BEFORE transposition.
    if df.empty:
        print("Input DataFrame is empty. Cannot generate outcome summary.")
        return pd.DataFrame()

    total_n = len(df)
    summary_data_by_var = {}
    print(f"Generating outcome summary data (N={total_n})...")

    def add_stats(var_name, stats_dict):
        if var_name not in summary_data_by_var:
            summary_data_by_var[var_name] = {}
        for key, value in stats_dict.items():
            summary_data_by_var[var_name][key] = value

    add_stats('Total Population', {'N': total_n})

    # --- Overall Adherence & Outcome Summaries ---
    numeric_cols_outcome = [
        'total_followup_days', 'nr_visits', 'nr_total_measurements', 'avg_days_between_measurements',
        'total_wl_kg', 'total_wl_%', 'bmi_reduction'
    ]
    outcome_described = get_describe_stats(df, numeric_cols_outcome)
    for col in outcome_described.index:
        stats_for_col = {}
        stats_for_col['N'] = int(outcome_described.loc[col, 'count']) if 'count' in outcome_described.columns else np.nan
        stats_for_col['Mean (SD)'] = format_mean_sd(df[col])
        if '50%' in outcome_described.columns and '25%' in outcome_described.columns and '75%' in outcome_described.columns:
             median, iqr_25, iqr_75 = outcome_described.loc[col, ['50%', '25%', '75%']]
             if not pd.isna([median, iqr_25, iqr_75]).any():
                  stats_for_col['Median (IQR)'] = f"{median:.1f} ({iqr_25:.1f}-{iqr_75:.1f})"
             else: stats_for_col['Median (IQR)'] = np.nan
        else: stats_for_col['Median (IQR)'] = np.nan
        if 'min' in outcome_described.columns and 'max' in outcome_described.columns:
             min_val, max_val = outcome_described.loc[col, ['min', 'max']]
             if not pd.isna([min_val, max_val]).any():
                  stats_for_col['Min - Max'] = f"{min_val:.1f} - {max_val:.1f}"
             else: stats_for_col['Min - Max'] = np.nan
        else: stats_for_col['Min - Max'] = np.nan
        add_stats(col, stats_for_col)

    # --- Specific Outcome Metrics ---
    # Instant Dropouts (total_followup_days == 1) - Direct calculation assuming numeric/bool
    if 'total_followup_days' in df.columns:
        try:
            # Attempt direct comparison (works for numbers, might work for bools if 1 used)
            instant_dropout_mask = (df['total_followup_days'] == 1)
            n_instant_dropout = instant_dropout_mask.sum()
            percent_instant = (n_instant_dropout / total_n) * 100 if total_n > 0 else 0
            add_stats('Instant Dropouts', {'N (%)': f"{n_instant_dropout} ({percent_instant:.1f}%)"})
        except Exception as e:
            print(f"  Warning: Could not calculate Instant Dropouts directly. Error: {e}. Trying format_n_percent.")
            # Fallback to string comparison if direct fails
            add_stats('Instant Dropouts', {'N (%)': format_n_percent(df['total_followup_days'], 1, total_n)})
    else: print("  Warning: 'total_followup_days' column not found for Instant Dropout calculation.")

    # --- Dynamic Time Window Metrics ---
    for window in time_windows_list:
        dropout_col = f'{window}d_dropout'
        wl_kg_col = f'wl_{window}d_kg'
        wl_pct_col = f'wl_{window}d_%'
        completer_var_name = f'Completers at {window}d'
        wl_kg_var_name = f'WL (kg) at {window}d [Completers]'
        wl_pct_var_name = f'WL (%) at {window}d [Completers]'

        if dropout_col in df.columns:
            # N (%) Completers (Not Dropout) - Direct calculation assuming boolean False
            try:
                completers_mask = (df[dropout_col] == False) # Explicitly check for boolean False
                n_completers = completers_mask.sum()
                percent_completers = (n_completers / total_n) * 100 if total_n > 0 else 0
                add_stats(completer_var_name, {'N (%)': f"{n_completers} ({percent_completers:.1f}%)"})

                # Mean (SD) Weight Loss for Completers
                completers_df = df.loc[completers_mask].copy()
                if not completers_df.empty:
                    if wl_kg_col in completers_df.columns:
                        add_stats(wl_kg_var_name, {
                            'Mean (SD)': format_mean_sd(completers_df[wl_kg_col]),
                            'N': len(completers_df[wl_kg_col].dropna())
                        })
                    else:
                        print(f"  Warning: '{wl_kg_col}' column not found.")
                        add_stats(wl_kg_var_name, {'Mean (SD)': np.nan, 'N': 0})
                    if wl_pct_col in completers_df.columns:
                        add_stats(wl_pct_var_name, {
                            'Mean (SD)': format_mean_sd(completers_df[wl_pct_col]),
                            'N': len(completers_df[wl_pct_col].dropna())
                        })
                    else:
                        print(f"  Warning: '{wl_pct_col}' column not found.")
                        add_stats(wl_pct_var_name, {'Mean (SD)': np.nan, 'N': 0})
                else:
                    # print(f"  Note: No completers found for {window}d window to calculate WL stats.") # Less verbose
                    add_stats(wl_kg_var_name, {'Mean (SD)': np.nan, 'N': 0})
                    add_stats(wl_pct_var_name, {'Mean (SD)': np.nan, 'N': 0})

            except Exception as e:
                 print(f"  Error calculating completer stats for {window}d. Column '{dropout_col}' might not be boolean. Error: {e}")
                 add_stats(completer_var_name, {'N (%)': 'Error'})
                 add_stats(wl_kg_var_name, {'Mean (SD)': 'Error', 'N': 'Error'})
                 add_stats(wl_pct_var_name, {'Mean (SD)': 'Error', 'N': 'Error'})

        else:
            print(f"  Warning: Dropout column '{dropout_col}' not found for window {window}d.")
            add_stats(completer_var_name, {'N (%)': np.nan})
            add_stats(wl_kg_var_name, {'Mean (SD)': np.nan, 'N': np.nan})
            add_stats(wl_pct_var_name, {'Mean (SD)': np.nan, 'N': np.nan})


    # --- Dynamic Weight Loss Target Metrics ---
    for target in weight_targets:
        achieved_col = f'{target}%_wl_achieved'
        days_col = f'days_to_{target}%_wl'
        achiever_var_name = f'Achieved {target}% WL'
        days_var_name = f'Days to {target}% WL [Achievers]'

        if achieved_col in df.columns:
            # N (%) Achievers - Direct calculation assuming boolean True
            try:
                achievers_mask = (df[achieved_col] == True) # Explicitly check for boolean True
                n_achievers = achievers_mask.sum()
                percent_achievers = (n_achievers / total_n) * 100 if total_n > 0 else 0
                add_stats(achiever_var_name, {'N (%)': f"{n_achievers} ({percent_achievers:.1f}%)"})

                # Mean (SD) Days to Achievement (for Achievers)
                if days_col in df.columns:
                    achievers_df = df.loc[achievers_mask].copy()
                    if not achievers_df.empty:
                        add_stats(days_var_name, {
                            'Mean (SD)': format_mean_sd(achievers_df[days_col]),
                            'N': len(achievers_df[days_col].dropna())
                        })
                    else:
                        # print(f"  Note: No achievers found for {target}% target to calculate days.") # Less verbose
                        add_stats(days_var_name, {'Mean (SD)': np.nan, 'N': 0})
                else:
                    print(f"  Warning: Days column '{days_col}' not found for target {target}%.")
                    add_stats(days_var_name, {'Mean (SD)': np.nan, 'N': np.nan})

            except Exception as e:
                 print(f"  Error calculating achiever stats for {target}%. Column '{achieved_col}' might not be boolean. Error: {e}")
                 add_stats(achiever_var_name, {'N (%)': 'Error'})
                 add_stats(days_var_name, {'Mean (SD)': 'Error', 'N': 'Error'})

        else:
            print(f"  Warning: Achievement column '{achieved_col}' not found for target {target}%.")
            add_stats(achiever_var_name, {'N (%)': np.nan})
            add_stats(days_var_name, {'Mean (SD)': np.nan, 'N': np.nan})


    # Convert dictionary to DataFrame (Variables as index, Stats as columns)
    summary_df = pd.DataFrame.from_dict(summary_data_by_var, orient='index')
    summary_df.index.name = 'Variable'

    # Reorder STAT columns (optional, done before transpose)
    desired_col_order = ['N', 'N (%)', 'Mean (SD)', 'Median (IQR)', 'Min - Max']
    existing_cols_ordered = [col for col in desired_col_order if col in summary_df.columns]
    remaining_cols = [col for col in summary_df.columns if col not in existing_cols_ordered]
    final_col_order = existing_cols_ordered + remaining_cols
    summary_df = summary_df[final_col_order]

    return summary_df


"""
MAIN ORCHESTRATION FUNCTION
"""

def create_and_save_summary_tables(db_path, input_table, pop_table_out, outcome_table_out, weight_targets, time_windows_list):
    """Loads data, generates both summary tables, transposes for final output, and saves them."""
    conn = None
    try:
        print(f"\nConnecting to database: {db_path}")
        if not os.path.exists(db_path):
            print(f"ERROR: Database file not found at {db_path}")
            return

        conn = sqlite3.connect(db_path)
        print(f"Loading input table: {input_table}")
        df = pd.read_sql_query(f"SELECT * FROM {input_table}", conn)

        if df.empty:
            print(f"Input table '{input_table}' is empty. Cannot generate summaries.")
            return

        # --- Generate Population Summary ---
        pop_summary_raw = generate_population_summary(df.copy())
        if not pop_summary_raw.empty:
            print(f"\n--- Population Summary ({pop_table_out}) ---")
            # Transpose for final output (Stats as index/rows, Variables as columns)
            pop_summary_final = pop_summary_raw #.transpose() OR MAYBE DON'T, Not as nice!
            pop_summary_final.index.name = 'Statistic' # Index is now stats
            print(pop_summary_final.to_string())
            print(f"\nSaving population summary to table: {pop_table_out}")
            # Save the transposed DataFrame, index=True saves 'Statistic' column
            pop_summary_final.to_sql(pop_table_out, conn, if_exists='replace', index=True)
        else:
            print("Population summary generation failed or resulted in an empty table.")

        # --- Generate Outcome Summary ---
        outcome_summary_raw = generate_outcome_summary(df.copy(), weight_targets, time_windows_list)
        if not outcome_summary_raw.empty:
            print(f"\n--- Outcome Summary ({outcome_table_out}) ---")
            # Transpose for final output (Stats as index/rows, Variables as columns)

            outcome_summary_final = outcome_summary_raw # .transpose() OR MAYBE DON'T, it is not as nice
            outcome_summary_final.index.name = 'Statistic' # Index is now stats
            print(outcome_summary_final.to_string())
            print(f"\nSaving outcome summary to table: {outcome_table_out}")
            # Save the transposed DataFrame, index=True saves 'Statistic' column
            outcome_summary_final.to_sql(outcome_table_out, conn, if_exists='replace', index=True)
        else:
            print("Outcome summary generation failed or resulted in an empty table.")

        conn.commit()
        print("\nSummary tables saved successfully.")

    except sqlite3.Error as e:
        print(f"ERROR: SQLite error - {e}")
        if conn: conn.rollback()
    except pd.errors.DatabaseError as e:
         print(f"ERROR: Pandas/Database error during SQL operation - {e}")
         if conn: conn.rollback()
    except KeyError as e:
        print(f"ERROR: A required column name was not found in the DataFrame: {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred - {e}")
        import traceback
        print(traceback.format_exc())
        if conn: conn.rollback()
    finally:
        if conn:
            conn.close()
            print("Database connection closed.")

"""
EXECUTION
"""
# Make sure 'paper1_directory' is defined before this point!
if 'paper1_directory' in locals() or 'paper1_directory' in globals():
    create_and_save_summary_tables(
        db_path,
        input_table_name,
        output_pop_summary_table,
        output_outcome_summary_table,
        weight_loss_targets,
        time_windows
    )
else:
    print("ERROR: 'paper1_directory' variable is not defined. Please define it before running this cell.")



Connecting to database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite
Loading input table: sa_input_table
Generating population summary data (N=1664)...

--- Population Summary (population_summary) ---
                                    N         N (%)    Mean (SD)      Median (IQR)     Min - Max
Statistic                                                                                       
Total Population               1664.0           NaN          NaN               NaN           NaN
age                            1664.0           NaN  47.2 ± 10.2  47.0 (41.0-54.0)   18.0 - 83.0
height_m                       1664.0           NaN    1.7 ± 0.1     1.6 (1.6-1.7)     1.4 - 2.1
baseline_weight_kg             1664.0           NaN  82.8 ± 12.1  82.0 (74.0-89.1)  57.5 - 131.2
last_aval_weight_kg            1664.0           NaN  76.3 ± 10.8  74.7 (68.4-82.4)  53.7 - 130.5
baseline_bmi                   1664.0           NaN   30.2 ± 3.0  30.1 (27.7

#### To-do: 2-split segmented population descriptions; multiple linear regressions to predict WL from EE; multiple logistic regressions to predict WL target success from EE; KM plots stratified by EE groups; maybe more? 

#### Comparative summary stats

##### Explanation of the first draft pipeline

Explanation:

Configuration:

rows_config: Defines each variable (row) and specifies if it's continuous or categorical, and how it should be formatted (mean_sd or n_perc). The special N row uses n_perc_pop.
strata_config: Defines each stratification (column group). It specifies the column name in the DataFrame, the type of stratification (median_split, value_split, binary, isna), the cutoff value (for value_split), and the desired labels for the two resulting groups.
Helper Functions:

format_mean_sd: Calculates and formats mean ± standard deviation.
format_n_perc: Calculates N and % for the positive class (1 or True) in binary/boolean columns.
format_n_perc_pop: Calculates N and % relative to the total input population size (for the 'N' row).
calculate_p_value: Performs Welch's t-test (equal_var=False) for continuous variables or Chi-squared test for categorical variables. It checks the expected frequencies from chi2_contingency and issues a warnings.warn if any expected count is < 5.
format_p_value: Formats the p-value for display (e.g., "<0.001").
generate_comparative_summary Function:

Takes the input DataFrame (df), rows_config, and strata_config.
Calculates total_population_n.
Pre-calculates Stratification Columns: It iterates through strata_config first to create temporary binary (0/1) columns in the DataFrame (_strata_...) based on the specified type (median split, value split, binary check, isna check). This avoids recalculating medians repeatedly. It stores the mapping between the stratification name and the temporary column name.
Iterates through Stratifications: For each defined stratification:
It retrieves the pre-calculated stratification column and labels.
It splits the DataFrame into group0_df and group1_df based on the stratification column, dropping rows where the stratification value is missing.
Iterates through Row Variables: For each variable defined in rows_config:
It calculates the statistic (Mean±SD or N(%)) for both group0_df and group1_df using the appropriate helper function.
It calculates the p-value comparing the two groups using calculate_p_value.
It stores the formatted stats and p-value in the summary_results dictionary.
Formats Output: Converts the summary_results dictionary into a pandas DataFrame, reorders the columns logically (Group 0, Group 1, p-value for each stratification), ensures the row order matches rows_config, and returns the final table.
Example Usage (if __name__ == "__main__":)

LOAD YOUR DATA: Replace the example SQLite loading with how you actually load your sa_input_table DataFrame.
Preprocessing: Includes crucial steps to:
Ensure boolean-like columns are consistently represented as numeric (0.0/1.0) to handle potential NaNs. You might need to adjust the .map() dictionary if your string representations are different.
Create the genomics_available column based on whether genomics_sample_id is null/NaN.
Create the instant_dropout column.
Ensure continuous columns are numeric, coercing errors to NaN.
Calls generate_comparative_summary.
Prints the resulting DataFrame (using options to show all rows/columns).

##### Code of the first draft pipeline

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
import sqlite3 # Assuming data is loaded from the SQLite DB
import os # Added for path joining

# --- Configuration ---

# Define the variables to include as rows in the table
# Format: 'variable_name': {'type': 'continuous'/'categorical', 'format': 'mean_sd'/'n_perc'/'n_perc_pop'}
# 'n_perc_pop' is special for the first 'N' row
# 'n_perc' for categorical assumes 0/1 or False/True, calculates N(%) for 1/True
rows_config = {
    'N': {'type': 'categorical', 'format': 'n_perc_pop'},
    'age': {'type': 'continuous', 'format': 'mean_sd'}, # Using corrected age column
    'sex_f': {'type': 'categorical', 'format': 'n_perc'},
    'height_m': {'type': 'continuous', 'format': 'mean_sd'},
    'baseline_weight_kg': {'type': 'continuous', 'format': 'mean_sd'},
    'baseline_bmi': {'type': 'continuous', 'format': 'mean_sd'},
    'hunger_yn': {'type': 'categorical', 'format': 'n_perc'},
    'satiety_yn': {'type': 'categorical', 'format': 'n_perc'},
    'emotional_eating_yn': {'type': 'categorical', 'format': 'n_perc'},
    'emotional_eating_value_likert': {'type': 'continuous', 'format': 'mean_sd'},
    'quantity_control_likert': {'type': 'continuous', 'format': 'mean_sd'},
    'impulse_control_likert': {'type': 'continuous', 'format': 'mean_sd'},
    'total_followup_days': {'type': 'continuous', 'format': 'mean_sd'},
    'avg_days_between_measurements': {'type': 'continuous', 'format': 'mean_sd'},
    'genomics_available': {'type': 'categorical', 'format': 'n_perc'}, # Assuming a boolean column is created
    'total_wl_%': {'type': 'continuous', 'format': 'mean_sd'},
    'bmi_reduction': {'type': 'continuous', 'format': 'mean_sd'},
    'instant_dropout': {'type': 'categorical', 'format': 'n_perc'}, # Assuming boolean: total_followup_days == 1
    '40d_dropout': {'type': 'categorical', 'format': 'n_perc'},
    'wl_40d_%': {'type': 'continuous', 'format': 'mean_sd'},
    '60d_dropout': {'type': 'categorical', 'format': 'n_perc'},
    'wl_60d_%': {'type': 'continuous', 'format': 'mean_sd'},
    '80d_dropout': {'type': 'categorical', 'format': 'n_perc'},
    'wl_80d_%': {'type': 'continuous', 'format': 'mean_sd'},
    '5%_wl_achieved': {'type': 'categorical', 'format': 'n_perc'},
    'days_to_5%_wl': {'type': 'continuous', 'format': 'mean_sd'},
    '10%_wl_achieved': {'type': 'categorical', 'format': 'n_perc'},
    'days_to_10%_wl': {'type': 'continuous', 'format': 'mean_sd'},
    '15%_wl_achieved': {'type': 'categorical', 'format': 'n_perc'},
    'days_to_15%_wl': {'type': 'continuous', 'format': 'mean_sd'},
}

# Define the stratification criteria for columns
# Format: {'name': 'Stratification Name', 'column': 'df_column', 'type': 'median_split'/'value_split'/'binary'/'isna', 'cutoff': value_for_split, 'labels': ['Group 0 Label', 'Group 1 Label']}
strata_config = [
    {'name': 'Age', 'column': 'age', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Sex', 'column': 'sex_f', 'type': 'binary', 'cutoff': None, 'labels': ['Male (0)', 'Female (1)']}, # Assumes 0/1
    {'name': 'Height', 'column': 'height_m', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Baseline Weight', 'column': 'baseline_weight_kg', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Baseline BMI', 'column': 'baseline_bmi', 'type': 'value_split', 'cutoff': 30, 'labels': ['< 30', '>= 30']},
    {'name': 'Total WL%', 'column': 'total_wl_%', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'BMI Reduction', 'column': 'bmi_reduction', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Hunger', 'column': 'hunger_yn', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': 'Satiety', 'column': 'satiety_yn', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': 'Emotional Eating YN', 'column': 'emotional_eating_yn', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': 'Emotional Eating Likert', 'column': 'emotional_eating_value_likert', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Quantity Control Likert', 'column': 'quantity_control_likert', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Impulse Control Likert', 'column': 'impulse_control_likert', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Followup Duration', 'column': 'total_followup_days', 'type': 'value_split', 'cutoff': 10, 'labels': ['<= 10 Days', '> 10 Days']}, # Note: Cutoff is > 10, so split is <=10 vs >10
    {'name': 'Measurement Frequency', 'column': 'avg_days_between_measurements', 'type': 'median_split', 'cutoff': None, 'labels': ['< Median', '>= Median']},
    {'name': 'Genomics', 'column': 'genomics_sample_id', 'type': 'isna', 'cutoff': None, 'labels': ['Not Available', 'Available']}, # Special type 'isna'
    {'name': '40d Dropout', 'column': '40d_dropout', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': '60d Dropout', 'column': '60d_dropout', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': '80d Dropout', 'column': '80d_dropout', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': '5% WL Achieved', 'column': '5%_wl_achieved', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': '10% WL Achieved', 'column': '10%_wl_achieved', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
    {'name': '15% WL Achieved', 'column': '15%_wl_achieved', 'type': 'binary', 'cutoff': None, 'labels': ['No (0)', 'Yes (1)']}, # Assumes 0/1
]

# --- Helper Functions ---

def format_mean_sd(series):
    """Formats mean and standard deviation."""
    if series.isnull().all() or len(series.dropna()) < 1:
        return "N/A"
    mean = series.mean()
    sd = series.std()
    return f"{mean:.2f} ± {sd:.2f}"

def format_n_perc(series):
    """Formats N and percentage for categorical (0/1 or T/F) variables, focusing on the '1' or 'True' category."""
    if series.isnull().all():
        return "0 (0.0%)"
    n_total = len(series.dropna())
    if n_total == 0:
        return "0 (0.0%)"
    # Ensure boolean or 0/1 interpretation
    series_bool = series.dropna().astype(bool)
    n_positive = series_bool.sum()
    perc_positive = (n_positive / n_total) * 100
    return f"{n_positive} ({perc_positive:.1f}%)"

def format_n_perc_pop(series, total_pop_n):
    """Formats N and percentage relative to the total population."""
    n = len(series) # Count all in the group, including potential NaNs if they weren't dropped before calling
    perc_pop = (n / total_pop_n) * 100 if total_pop_n > 0 else 0
    return f"{n} ({perc_pop:.1f}%)"

def calculate_p_value(series1, series2, var_type):
    """Calculates p-value using Welch's t-test or Chi-squared test."""
    series1_clean = series1.dropna()
    series2_clean = series2.dropna()

    # Check if either group is empty after dropping NaNs
    if series1_clean.empty or series2_clean.empty:
        # print(f"  Debug: Empty series after dropna. S1 len: {len(series1_clean)}, S2 len: {len(series2_clean)}") # Optional debug
        return np.nan

    try:
        if var_type == 'continuous':
            # Need at least 2 observations in each group for t-test
            if len(series1_clean) < 2 or len(series2_clean) < 2:
                # print(f"  Debug: Insufficient data for t-test. S1 len: {len(series1_clean)}, S2 len: {len(series2_clean)}") # Optional debug
                return np.nan
            # Welch's t-test (does not assume equal variance)
            stat, p_val = stats.ttest_ind(series1_clean, series2_clean, equal_var=False, nan_policy='omit')
            return p_val
        elif var_type == 'categorical':
            # --- MODIFIED CONTINGENCY TABLE CREATION ---
            # Create a temporary DataFrame combining the data and group indicators
            group_indicator = np.concatenate([np.zeros(len(series1_clean)), np.ones(len(series2_clean))])
            data_values = pd.concat([series1_clean, series2_clean], ignore_index=True)

            # Check if there's any data left after concatenation (shouldn't happen if checks above passed)
            if data_values.empty:
                 # print("  Debug: data_values empty in categorical p-value calc.") # Optional debug
                 return np.nan

            # Create contingency table using crosstab on the temporary DataFrame
            try:
                 contingency_table = pd.crosstab(index=group_indicator, columns=data_values)
            except Exception as e_crosstab:
                 warnings.warn(f"Error during pd.crosstab for categorical variable: {e_crosstab}. S1 unique: {series1_clean.unique()}, S2 unique: {series2_clean.unique()}", UserWarning)
                 return np.nan
            # --- END MODIFICATION ---

            # Ensure the table has the expected shape (at least 2 rows (groups) and >= 1 column (category))
            # Chi2 requires at least 1 degree of freedom (e.g., 2x2 table).
            # If only one category exists overall, p-value is meaningless (1.0 or NaN).
            if contingency_table.shape[0] < 2 or contingency_table.shape[1] < 1:
                 # print(f"  Debug: Contingency table shape invalid: {contingency_table.shape}") # Optional debug
                 return np.nan # Or 1.0 if shape[1] == 1? NaN is safer.

            # Handle cases where a group might have zero counts for all categories
            if (contingency_table.sum(axis=1) == 0).any():
                 # print(f"  Debug: Zero counts for one group in contingency table:\n{contingency_table}") # Optional debug
                 return np.nan # Chi2 will likely fail or give NaN

            # Chi-squared test
            try:
                chi2, p_val, dof, expected = stats.chi2_contingency(contingency_table)

                # Check expected frequencies only if chi2 ran successfully
                if dof > 0 and (expected < 5).any(): # Check only if dof > 0 (i.e., test is meaningful)
                    warnings.warn(
                        f"Chi-squared test assumption violated (expected count < 5) for variable comparison. "
                        f"Consider Fisher's Exact Test. Contingency Table:\\n{contingency_table}\\nExpected:\\n{expected}",
                        UserWarning
                    )
                # If dof is 0 (e.g., only one category after all), p-value is often NaN or 1.
                if dof == 0:
                    return 1.0 # Or np.nan? Let's return 1.0 indicating no difference detectable.

                return p_val
            except ValueError as e_chi2:
                 # Catch specific errors from chi2_contingency (e.g., sum of frequencies is zero)
                 warnings.warn(f"Error during chi2_contingency: {e_chi2}. Contingency Table:\\n{contingency_table}", UserWarning)
                 return np.nan

        else:
            return np.nan
    except Exception as e:
        warnings.warn(f"Could not calculate p-value: {e}", UserWarning)
        return np.nan

def format_p_value(p_val):
    """Formats p-value."""
    if pd.isna(p_val):
        return "N/A"
    elif p_val < 0.001:
        return "<0.001"
    else:
        return f"{p_val:.3f}"

# --- Main Function ---

def generate_comparative_summary(df, rows_config, strata_config):
    """
    Generates a comparative summary table based on defined row variables and stratification criteria.

    Args:
        df (pd.DataFrame): The input dataframe (e.g., sa_input_table).
        rows_config (dict): Configuration for table rows.
        strata_config (list): Configuration for table columns (stratifications).

    Returns:
        pd.DataFrame: The comparative summary table.
    """
    summary_results = {}
    total_population_n = len(df)
    df_analysis = df.copy() # Work on a copy

    # Pre-calculate stratification columns where needed
    strata_cols_mapping = {}
    for strat_info in strata_config:
        strat_col_name = f"_strata_{strat_info['column']}"
        original_col = strat_info['column']

        # Check if original column exists
        if original_col not in df_analysis.columns:
            warnings.warn(f"Stratification column '{original_col}' not found in DataFrame. Skipping stratification '{strat_info['name']}'.", UserWarning)
            df_analysis[strat_col_name] = pd.NA # Mark as unusable
            labels = ['Group 0', 'Group 1'] # Placeholder labels
        elif strat_info['type'] == 'median_split':
            # Ensure column is numeric before calculating median
            numeric_col = pd.to_numeric(df_analysis[original_col], errors='coerce')
            if numeric_col.isnull().all():
                 warnings.warn(f"Stratification column '{original_col}' for median split is all NaN or non-numeric. Skipping stratification '{strat_info['name']}'.", UserWarning)
                 df_analysis[strat_col_name] = pd.NA
                 labels = ['Group 0', 'Group 1']
            else:
                 median_val = numeric_col.median()
                 df_analysis[strat_col_name] = (numeric_col >= median_val).astype(int)
                 labels = strat_info['labels']
        elif strat_info['type'] == 'value_split':
            cutoff_val = strat_info['cutoff']
            # Ensure column is numeric before comparison
            numeric_col = pd.to_numeric(df_analysis[original_col], errors='coerce')
            if numeric_col.isnull().all():
                 warnings.warn(f"Stratification column '{original_col}' for value split is all NaN or non-numeric. Skipping stratification '{strat_info['name']}'.", UserWarning)
                 df_analysis[strat_col_name] = pd.NA
                 labels = ['Group 0', 'Group 1']
            else:
                # Handle <= vs < based on common interpretation (e.g., BMI >= 30)
                if strat_info['name'] == 'Followup Duration': # Special case <= 10 vs > 10
                     df_analysis[strat_col_name] = (numeric_col > cutoff_val).astype(int)
                     labels = strat_info['labels'] # ['<= 10 Days', '> 10 Days'] -> 0, 1
                else: # Default is < cutoff vs >= cutoff
                    df_analysis[strat_col_name] = (numeric_col >= cutoff_val).astype(int)
                    labels = strat_info['labels'] # ['< Cutoff', '>= Cutoff'] -> 0, 1
        elif strat_info['type'] == 'binary':
             # Assuming column is already 0/1 or T/F. Convert T/F to 0/1 if necessary
             col_series = df_analysis[original_col]
             if col_series.dtype == bool:
                 df_analysis[strat_col_name] = col_series.astype(int)
             elif pd.api.types.is_numeric_dtype(col_series):
                 # Check if it's only 0s and 1s (and NaNs)
                 unique_vals = col_series.dropna().unique()
                 if set(unique_vals).issubset({0, 1}):
                      df_analysis[strat_col_name] = col_series
                 else:
                     warnings.warn(f"Binary column {original_col} contains values other than 0/1. Stratification might be incorrect.", UserWarning)
                     df_analysis[strat_col_name] = pd.NA # Mark as unusable
             else:
                 # Handle potential 'yes'/'no' strings if needed (add conversion logic here if required)
                 warnings.warn(f"Binary column {original_col} is not numeric or boolean. Stratification might be incorrect.", UserWarning)
                 df_analysis[strat_col_name] = pd.NA # Mark as unusable
             labels = strat_info['labels']
        elif strat_info['type'] == 'isna':
            df_analysis[strat_col_name] = df_analysis[original_col].notna().astype(int)
            labels = strat_info['labels'] # ['Not Available', 'Available'] -> 0, 1
        else:
             warnings.warn(f"Unknown stratification type: {strat_info['type']} for column {original_col}", UserWarning)
             df_analysis[strat_col_name] = pd.NA
             labels = ['Group 0', 'Group 1']

        strata_cols_mapping[strat_info['name']] = {'strat_col': strat_col_name, 'labels': labels}


    # --- Iterate through Stratifications (Columns) ---
    for strat_info in strata_config:
        strat_name = strat_info['name']
        # Check if mapping exists (might not if original column was missing)
        if strat_name not in strata_cols_mapping:
            continue
        strat_details = strata_cols_mapping[strat_name]
        strat_col = strat_details['strat_col']
        labels = strat_details['labels']

        if pd.isna(df_analysis[strat_col]).all():
            warnings.warn(f"Stratification column '{strat_col}' for '{strat_name}' could not be created or is all NA. Skipping this stratification.", UserWarning)
            continue

        # Ensure labels list has at least two elements
        if len(labels) < 2:
            warnings.warn(f"Insufficient labels provided for stratification '{strat_name}'. Skipping.", UserWarning)
            continue

        group0_label = labels[0]
        group1_label = labels[1]

        # Create group subsets, dropping rows where the stratification variable is NA
        df_strat_clean = df_analysis.dropna(subset=[strat_col])
        group0_df = df_strat_clean[df_strat_clean[strat_col] == 0]
        group1_df = df_strat_clean[df_strat_clean[strat_col] == 1]

        if group0_df.empty or group1_df.empty:
             warnings.warn(f"One or both groups are empty for stratification '{strat_name}' after dropping NAs. Skipping.", UserWarning)
             continue

        # --- Iterate through Variables (Rows) ---
        for row_var, config in rows_config.items():
            var_type = config['type']
            var_format = config['format']

            if row_var not in summary_results:
                summary_results[row_var] = {}

            # Handle the special 'N' row
            if row_var == 'N':
                stat_g0 = format_n_perc_pop(group0_df, total_population_n)
                stat_g1 = format_n_perc_pop(group1_df, total_population_n)
                p_val_formatted = "N/A" # No p-value for N counts
            else:
                # Check if row_var exists in the dataframe
                if row_var not in df_analysis.columns:
                     warnings.warn(f"Row variable '{row_var}' not found in DataFrame. Skipping for stratification '{strat_name}'.", UserWarning)
                     stat_g0 = "Var Missing"
                     stat_g1 = "Var Missing"
                     p_val_formatted = "N/A"
                else:
                    series_g0 = group0_df[row_var]
                    series_g1 = group1_df[row_var]

                    # Calculate stats for each group
                    if var_format == 'mean_sd':
                        stat_g0 = format_mean_sd(series_g0)
                        stat_g1 = format_mean_sd(series_g1)
                    elif var_format == 'n_perc':
                        stat_g0 = format_n_perc(series_g0)
                        stat_g1 = format_n_perc(series_g1)
                    else:
                        stat_g0 = "Invalid Format"
                        stat_g1 = "Invalid Format"

                    # Calculate p-value
                    p_val = calculate_p_value(series_g0, series_g1, var_type)
                    p_val_formatted = format_p_value(p_val)

            # Store results
            summary_results[row_var][f"{strat_name}: {group0_label}"] = stat_g0
            summary_results[row_var][f"{strat_name}: {group1_label}"] = stat_g1
            summary_results[row_var][f"{strat_name}: p-value"] = p_val_formatted

    # Convert results to DataFrame
    summary_df = pd.DataFrame.from_dict(summary_results, orient='index')
    summary_df.index.name = 'Variable' # Name the index

    # Reorder columns to group by stratification
    ordered_columns = []
    for strat_info in strata_config:
         strat_name = strat_info['name']
         # Check if columns were actually created for this strat_name
         col_prefix = f"{strat_name}: "
         if any(col.startswith(col_prefix) for col in summary_df.columns):
             # Check if mapping exists (might not if original column was missing)
             if strat_name in strata_cols_mapping:
                 labels = strata_cols_mapping[strat_name]['labels']
                 if len(labels) >= 2:
                     ordered_columns.append(f"{strat_name}: {labels[0]}")
                     ordered_columns.append(f"{strat_name}: {labels[1]}")
                     ordered_columns.append(f"{strat_name}: p-value")

    # Ensure only existing columns are selected
    final_columns = [col for col in ordered_columns if col in summary_df.columns]
    summary_df = summary_df[final_columns]

    # Ensure row order matches rows_config
    summary_df = summary_df.reindex(rows_config.keys())

    return summary_df

# --- Example Usage ---
if __name__ == "__main__":
    # ** IMPORTANT: Load your actual DataFrame here **
    # Example: Assuming you have 'survival_analysis.sqlite' and 'sa_input_table'

    # --- MODIFIED: Define output DB path and table name ---
    # Assuming paper1_directory is defined in a previous cell
    if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
        # Define a default or raise an error if paper1_directory is crucial
        # Example: Use current directory if not defined
        paper1_directory = '.'
        print("Warning: 'paper1_directory' not defined. Using current directory for output.")
        # Or raise NameError("'paper1_directory' is not defined. Please define it.")

    output_db_path = os.path.join(paper1_directory, 'survival_analysis.sqlite')
    input_table_name = 'sa_input_table'
    output_table_name = 'comparative_summary'
    # --- END MODIFICATION ---

    input_conn = None
    output_conn = None

    try:
        print(f"Connecting to input database: {output_db_path}") # Use output_db_path as it contains the input table
        if not os.path.exists(output_db_path):
            raise FileNotFoundError(f"Database file not found at {output_db_path}")
        input_conn = sqlite3.connect(output_db_path)
        print(f"Loading input table: {input_table_name}")
        input_df = pd.read_sql_query(f"SELECT * FROM {input_table_name}", input_conn)
        input_conn.close() # Close input connection after loading

        if input_df.empty:
             print(f"Warning: Input table '{input_table_name}' is empty. Cannot generate summary.")
        else:
            # --- Data Preprocessing (Crucial Steps - Adapt as needed) ---
            print("Starting data preprocessing...")
            # 1. Convert boolean-like columns if they are not already bool or 0/1
            bool_cols = ['sex_f', 'hunger_yn', 'satiety_yn', 'emotional_eating_yn',
                         '40d_dropout', '60d_dropout', '80d_dropout',
                         '5%_wl_achieved', '10%_wl_achieved', '15%_wl_achieved']
            for col in bool_cols:
                if col in input_df.columns:
                     if input_df[col].dtype == object: # If it's string 'yes'/'no' or similar
                          # Example conversion (adjust based on your actual string values)
                          input_df[col] = input_df[col].map({'yes': 1, 'no': 0, 'Yes': 1, 'No': 0, 'True': 1, 'False': 0, True: 1, False: 0, 1:1, 0:0}).astype(float) # Use float to allow NAs
                     elif pd.api.types.is_bool_dtype(input_df[col].dtype):
                          input_df[col] = input_df[col].astype(float) # Convert bool to float (0.0/1.0) to allow NAs if any exist
                     elif pd.api.types.is_numeric_dtype(input_df[col].dtype):
                          # Ensure it's float if it might contain NAs, otherwise check if 0/1
                          unique_vals = input_df[col].dropna().unique()
                          if not set(unique_vals).issubset({0, 1}):
                               warnings.warn(f"Column {col} intended as binary contains values other than 0/1.", UserWarning)
                          input_df[col] = input_df[col].astype(float) # Ensure float for consistency


            # 2. Create 'genomics_available' column (based on 'genomics_sample_id')
            if 'genomics_sample_id' in input_df.columns:
                input_df['genomics_available'] = input_df['genomics_sample_id'].notna() # True if ID exists, False if NaN/None
                input_df['genomics_available'] = input_df['genomics_available'].astype(float) # Convert to 0.0 / 1.0
            else:
                warnings.warn("'genomics_sample_id' column not found. Genomics stratification will be skipped.", UserWarning)
                # Remove genomics from strata_config if column doesn't exist
                strata_config = [s for s in strata_config if s['column'] != 'genomics_sample_id']


            # 3. Create 'instant_dropout' column
            if 'total_followup_days' in input_df.columns:
                 input_df['instant_dropout'] = (input_df['total_followup_days'] == 1) # Or <= 1 depending on definition
                 input_df['instant_dropout'] = input_df['instant_dropout'].astype(float)
            else:
                 warnings.warn("'total_followup_days' column not found. 'instant_dropout' row cannot be calculated.", UserWarning)
                 # Remove instant_dropout from rows_config if column doesn't exist
                 if 'instant_dropout' in rows_config: del rows_config['instant_dropout']


            # 4. Ensure numeric types for continuous columns (handle potential errors)
            num_cols = [r for r, c in rows_config.items() if c['type'] == 'continuous' and r != 'N']
            for col in num_cols:
                 if col in input_df.columns:
                      input_df[col] = pd.to_numeric(input_df[col], errors='coerce')

            print("Preprocessing finished.")
            # --- Generate the Summary Table ---
            print("Generating comparative summary table...")
            comparative_summary_df = generate_comparative_summary(input_df, rows_config, strata_config)

            # --- Display or Save the Table ---
            print("Comparative Summary Table generated.")
            # Display the full table (might be very wide)
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            print(comparative_summary_df)

            # --- MODIFIED: Save to SQLite database ---
            if not comparative_summary_df.empty:
                print(f"\nConnecting to output database: {output_db_path}")
                output_conn = sqlite3.connect(output_db_path)
                try:
                    print(f"Saving comparative summary to table: {output_table_name}")
                    # Save DataFrame to SQL table. Use index=True to save the 'Variable' index as a column.
                    comparative_summary_df.to_sql(output_table_name, output_conn, if_exists='replace', index=True)
                    output_conn.commit()
                    print(f"Table '{output_table_name}' saved successfully to {output_db_path}.")
                except sqlite3.Error as e:
                    print(f"SQLite error saving table '{output_table_name}': {e}")
                    if output_conn: output_conn.rollback()
                except Exception as e:
                    print(f"An error occurred saving table '{output_table_name}': {e}")
                    if output_conn: output_conn.rollback()
                finally:
                    if output_conn:
                        output_conn.close()
                        print("Output database connection closed.")
            else:
                print("Comparative summary DataFrame is empty. Nothing to save.")
            # --- END MODIFICATION ---

            # Optionally save to CSV or Excel
            # comparative_summary_df.to_csv("comparative_summary_table.csv")
            # comparative_summary_df.to_excel("comparative_summary_table.xlsx")

    except FileNotFoundError as e:
        print(f"Error: {e}")
    except sqlite3.Error as e:
        print(f"SQLite error during connection or loading: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
    finally:
        # Ensure input connection is closed if it was opened and not closed yet
        if input_conn:
            try:
                input_conn.close()
                print("Input database connection closed.")
            except: # Handle cases where connection might already be closed or invalid
                pass
        # Ensure output connection is closed if it was opened and not closed yet
        if output_conn:
            try:
                output_conn.close()
                print("Output database connection closed.")
            except:
                pass
    print("Script finished.")

Connecting to input database: .\survival_analysis.sqlite
Loading input table: sa_input_table
Starting data preprocessing...
Preprocessing finished.
Generating comparative summary table...
Comparative Summary Table generated.
                               Age: < Median Age: >= Median Age: p-value  \
Variable                                                                   
N                                762 (45.8%)    902 (54.2%)          N/A   
age                             38.52 ± 6.38   54.54 ± 6.22       <0.001   
sex_f                            628 (82.4%)    740 (82.0%)        0.893   
height_m                         1.67 ± 0.08    1.64 ± 0.08       <0.001   
baseline_weight_kg             84.01 ± 11.91  81.86 ± 12.27       <0.001   
baseline_bmi                    30.13 ± 3.00   30.21 ± 3.05        0.609   
hunger_yn                        501 (65.7%)    597 (66.2%)        0.892   
satiety_yn                       456 (59.8%)    553 (61.3%)        0.576   
emotional_eatin

#### Linear regressions

##### Explanation of the first draft pipeline

##### Define scenarios for the analysis

##### Code of the first draft pipeline

In [13]:
####
# Linear Regression Pipeline Module
####

import pandas as pd
import sqlite3
import os
import statsmodels.api as sm
import numpy as np
import warnings
import traceback # Keep traceback for detailed error reporting

# Suppress potential warnings (optional, adjust as needed)
# warnings.simplefilter('ignore', SomeWarningCategory)

# Ensure paper1_directory is defined in a previous cell
if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
    # Try to get it from the environment if running as a script, or raise error
    paper1_directory = os.getenv('PAPER1_DIRECTORY')
    if paper1_directory is None:
        raise NameError("'paper1_directory' variable not defined. Please ensure it is defined or set as an environment variable 'PAPER1_DIRECTORY'.")
    else:
        print(f"Using paper1_directory from environment variable: {paper1_directory}")

# --- CONFIGURATION ---

# Adjust these paths as needed
DB_PATH = os.path.join(paper1_directory, 'survival_analysis.sqlite') # Path to the SQLite database
INPUT_TABLE_NAME = "sa_input_table" # Your wide input table
OUTPUT_TABLE_NAME = "lin_regression_results" # Table to store summary results

# --- Define Scenarios (Example - Adapt as needed) ---
# The structure is similar to logistic regression, but 'outcome_type' implicitly means predicting wl_%
# 'target_perc' is not used here.
# 'time_window' determines which wl_% column to use ('total', 40, 60, etc.)
LINREG_SCENARIOS = [
    # Example: Predict total weight loss % using hunger (unadjusted)
    {
        'name': 'total_wl_hunger_unadjusted',
        'time_window': 'total', # Use 'total_wl_%'
        'predictors': ['hunger_yn'],
        'covariates': []
    },
    # Example: Predict 60-day weight loss % using emotional eating value (adjusted)
    {
        'name': '60d_wl_emot_val_adjusted',
        'time_window': 60, # Use 'wl_60d_%'
        'predictors': ['emotional_eating_value_likert'],
        'covariates': ['age', 'sex_f', 'baseline_bmi']
    },
    # Example: Predict 60-day weight loss % using quantity control (adjusted)
    {
        'name': '60d_wl_quant_ctrl_adjusted',
        'time_window': 60, # Use 'wl_60d_%'
        'predictors': ['quantity_control_likert'],
        'covariates': ['age', 'sex_f', 'baseline_bmi']
    },
    # Add more scenarios following this pattern
]

# --- HELPERS ---

def load_data(db_path, table_name):
    """Loads data from the specified SQLite database table."""
    print(f"Loading data from {db_path}, table {table_name}...")
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file not found at {db_path}")
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        print(f"Loaded {len(df)} rows and {len(df.columns)} columns.")
        if df.empty:
             print(f"Warning: Loaded DataFrame from {table_name} is empty.")
        return df
    except sqlite3.Error as e:
        print(f"SQLite error loading data: {e}")
        raise
    except Exception as e:
        print(f"An error occurred loading data: {e}")
        raise
    finally:
        if conn:
            conn.close()

def preprocess_data(df, scenarios):
    """Handles basic data preparation for linear models."""
    print("Preprocessing data for Linear models...")
    if df.empty:
         print("  Input DataFrame is empty. Skipping preprocessing.")
         return df

    df_processed = df.copy()

    # Identify all unique predictor/covariate columns needed across scenarios
    all_model_cols = set()
    bool_predictors = set() # Specifically track yes/no predictors
    for scenario in scenarios:
        all_model_cols.update(scenario['predictors'])
        all_model_cols.update(scenario['covariates'])
        # Identify boolean predictors based on common naming or explicit list
        for pred in scenario['predictors']:
            if pred.endswith('_yn'): # Assuming '_yn' suffix for boolean predictors
                 bool_predictors.add(pred)

    # Ensure boolean predictors are 0/1 integers
    for col in bool_predictors:
        if col in df_processed.columns:
            if not pd.api.types.is_numeric_dtype(df_processed[col]) or not df_processed[col].dropna().isin([0, 1]).all():
                print(f"  Attempting to convert boolean predictor '{col}' to 0/1 integer.")
                # Handle potential 'yes'/'no' strings or True/False bools
                if df_processed[col].dtype == 'object':
                    df_processed[col] = df_processed[col].str.lower().map({'yes': 1, 'no': 0, 'y': 1, 'n': 0})
                elif df_processed[col].dtype == 'bool':
                     df_processed[col] = df_processed[col].astype(int)

                # Convert to numeric, coercing errors
                df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
                if not df_processed[col].dropna().isin([0, 1]).all():
                     print(f"    Warning: Column '{col}' could not be reliably converted to 0/1.")
            # Convert float 0.0/1.0 to integer
            elif pd.api.types.is_float_dtype(df_processed[col]) and df_processed[col].dropna().isin([0.0, 1.0]).all():
                 df_processed[col] = df_processed[col].astype('Int64') # Use nullable integer
        else:
             print(f"  Warning: Boolean predictor column '{col}' specified in scenarios but not found.")

    # Ensure other predictors/covariates AND potential outcome columns are numeric
    numeric_cols_to_check = all_model_cols - bool_predictors
    # Add potential weight loss outcome columns dynamically based on scenarios
    for scenario in scenarios:
        tw = scenario['time_window']
        wl_col = f"wl_{tw}d_%" if isinstance(tw, int) else "total_wl_%"
        numeric_cols_to_check.add(wl_col)

    for col in numeric_cols_to_check:
        if col in df_processed.columns:
            if not pd.api.types.is_numeric_dtype(df_processed[col]):
                print(f"  Attempting to convert column '{col}' to numeric.")
                original_nan_count = df_processed[col].isnull().sum()
                df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
                if df_processed[col].isnull().sum() > original_nan_count:
                    print(f"    Warning: Conversion of column '{col}' introduced NaNs.")
        # Don't warn if column doesn't exist, it might only be needed for specific scenarios checked later

    print("Preprocessing complete.")
    return df_processed


# --- LINEAR REGRESSION MODULE ---

def run_linear_model(df_subset, outcome_col, predictor_cols, scenario_name):
    """
    Fits an OLS Linear Regression model using statsmodels and extracts results.

    Args:
        df_subset (pd.DataFrame): DataFrame containing only necessary columns and no NaNs for the model.
        outcome_col (str): Name of the continuous outcome column (e.g., 'wl_60d_%').
        predictor_cols (list): List of predictor and covariate column names.
        scenario_name (str): Name of the scenario for labeling outputs.

    Returns:
        pd.DataFrame: DataFrame containing model summary results (Coefficient, CI, p-value per variable),
                      or None if the model fails or data is insufficient.
    """
    print(f"  Running Linear model (OLS) for scenario: {scenario_name}")
    print(f"    Outcome: {outcome_col}")
    print(f"    Predictors: {predictor_cols}")
    print(f"    Data shape: {df_subset.shape}")

    # Check for sufficient data
    min_obs_per_predictor = 10
    required_obs = min_obs_per_predictor * len(predictor_cols) if predictor_cols else 20
    # Need at least k+1 observations for k predictors + intercept
    required_obs = max(required_obs, len(predictor_cols) + 2)
    if df_subset.shape[0] < required_obs:
        print(f"    Warning: Insufficient data ({df_subset.shape[0]} obs) for {len(predictor_cols)} predictors. Need at least {required_obs}. Skipping model.")
        return None
    if not predictor_cols:
        print("    Warning: No predictors specified. Skipping model.")
        return None
    # Check if outcome column has variance
    if outcome_col not in df_subset or df_subset[outcome_col].var() == 0:
        print(f"    Warning: Outcome column '{outcome_col}' has zero variance or is missing. Skipping model.")
        return None

    results_list = []

    try:
        # Prepare data for statsmodels (add constant for intercept)
        X = df_subset[predictor_cols]
        y = df_subset[outcome_col]
        X = sm.add_constant(X, has_constant='add') # Add intercept

        # Fit the model
        ols_model = sm.OLS(y, X)
        result = ols_model.fit()

        # Extract Results
        params = result.params
        conf = result.conf_int()
        conf.columns = ['CI Lower 95%', 'CI Upper 95%']
        p_values = result.pvalues
        t_values = result.tvalues # Get t-statistics

        # Combine results into a DataFrame
        summary_df = pd.DataFrame({
            'Variable': params.index,
            'Coefficient': params.values,
            'Std_Err': result.bse.values, # Standard Error
            'T_Statistic': t_values.values,
            'P_Value': p_values.values,
            'CI Lower 95%': conf['CI Lower 95%'],
            'CI Upper 95%': conf['CI Upper 95%'],
        })

        # Add model-level stats
        model_stats = {
            'scenario_name': scenario_name,
            'outcome_variable': outcome_col,
            'n_observations': int(result.nobs),
            'r_squared': result.rsquared,
            'adj_r_squared': result.rsquared_adj,
            'f_statistic': result.fvalue,
            'f_p_value': result.f_pvalue,
            'aic': result.aic,
            'bic': result.bic,
            'condition_number': result.condition_number, # Indicator for multicollinearity
            # 'converged': True # OLS typically doesn't have convergence issues like iterative methods
        }

        # Add model stats to each row of the summary_df for easy aggregation
        for col, val in model_stats.items():
            summary_df[col] = val

        print("    Model fitting complete.")
        return summary_df

    except np.linalg.LinAlgError as e_linalg:
         print(f"    ERROR: Linear algebra error (likely perfect multicollinearity) for scenario {scenario_name}. Skipping. Error: {e_linalg}")
         return None
    except ValueError as e_val:
         # This might happen if data isn't purely numeric after preprocessing somehow
         print(f"    ERROR: Value error during model fitting (check data types/values) for scenario {scenario_name}. Skipping. Error: {e_val}")
         return None
    except Exception as e_fit:
        print(f"    ERROR: Failed to fit Linear model for scenario {scenario_name}. Error: {e_fit}")
        print(traceback.format_exc()) # Print traceback for debugging
        return None

# --- Scenario Processing Function ---

def process_scenario(scenario, df_analysis):
    """
    Processes a single linear regression scenario.

    Args:
        scenario (dict): Dictionary containing scenario configuration.
        df_analysis (pd.DataFrame): The preprocessed DataFrame.

    Returns:
        pd.DataFrame or None: DataFrame with model results for the scenario,
                              or None if processing fails or is skipped.
    """
    scenario_name = scenario['name']
    print(f"  Starting processing for scenario: {scenario_name}")

    time_window = scenario['time_window']
    predictors = scenario['predictors']
    covariates = scenario['covariates']
    all_predictors = predictors + covariates

    # --- Define Outcome Variable Dynamically ---
    # Determine the source weight loss column based on time_window
    if time_window == 'total':
        outcome_col_name = 'total_wl_%'
    elif isinstance(time_window, int):
        outcome_col_name = f'wl_{time_window}d_%'
    else:
        print(f"    ERROR: Invalid 'time_window' ({time_window}) in scenario '{scenario_name}'. Use 'total' or integer days. Skipping scenario.")
        return None

    print(f"    Outcome variable: '{outcome_col_name}'")

    # --- Prepare Data for Model ---
    print("    Preparing data subset for the model...")
    # Check if all necessary columns exist
    required_cols = [outcome_col_name] + all_predictors
    missing_cols = [col for col in required_cols if col not in df_analysis.columns]
    if missing_cols:
        print(f"    ERROR: Missing required columns for scenario '{scenario_name}': {missing_cols}. Skipping scenario.")
        return None

    # Select relevant columns and drop rows with missing values in predictors/covariates OR outcome
    initial_rows = len(df_analysis)
    # Use a copy to avoid modifying df_analysis if needed elsewhere, though not strictly necessary here
    temp_df = df_analysis.copy()
    df_model_subset = temp_df[required_cols].dropna()
    final_rows = len(df_model_subset)
    print(f"      Selected columns: {required_cols}")
    print(f"      Removed {initial_rows - final_rows} rows with missing values (out of {initial_rows}). Final dataset size for model: {final_rows} rows.")

    if final_rows == 0:
        print(f"    ERROR: No valid data remaining after handling missing values for scenario '{scenario_name}'. Skipping scenario.")
        return None
    if df_model_subset[outcome_col_name].var() == 0:
        print(f"    ERROR: Outcome variable '{outcome_col_name}' has zero variance after filtering. Cannot run linear regression. Skipping scenario.")
        return None

    # --- Run the Linear Model ---
    print(f"    Running linear regression model for scenario '{scenario_name}'...")
    model_output_df = None
    try:
        model_output_df = run_linear_model(
            df_model_subset,
            outcome_col_name,
            all_predictors,
            scenario_name
        )
    except Exception as e_model:
        print(f"    ERROR: An exception occurred during model execution for scenario '{scenario_name}': {e_model}")
        print(traceback.format_exc()) # Print traceback for debugging
        return None # Indicate failure

    if model_output_df is not None and not model_output_df.empty:
        print(f"    Model for scenario '{scenario_name}' ran successfully.")
        return model_output_df
    else:
         print(f"    Model for scenario '{scenario_name}' did not produce valid results (returned None or empty DataFrame).")
         return None

# --- ORCHESTRATION ---

def main():
    """Main function to orchestrate the Linear Regression pipeline."""
    print("========== Starting Linear Regression Pipeline ==========")
    all_results_summary = [] # To store summary stats from each model

    try:
        # 1. Load Data
        print("\n--- Step 1: Loading Data ---")
        df_raw = load_data(DB_PATH, INPUT_TABLE_NAME)
        if df_raw is None: # Assume load_data returns None on error
             print("ERROR: Failed to load data. Stopping pipeline.")
             return # Exit main function
        if df_raw.empty:
             print("ERROR: Input data table is empty. Stopping pipeline.")
             return # Exit main function
        print(f"Data loaded successfully: {df_raw.shape[0]} rows, {df_raw.shape[1]} columns.")

        # 2. Preprocess Data
        print("\n--- Step 2: Preprocessing Data ---")
        # Pass the specific scenarios for this pipeline
        df_analysis = preprocess_data(df_raw, LINREG_SCENARIOS)
        if df_analysis is None: # Assume preprocess_data returns None on error
            print("ERROR: Failed during data preprocessing. Stopping pipeline.")
            return # Exit main function
        if df_analysis.empty:
             print("ERROR: Data preprocessing resulted in an empty DataFrame. Stopping pipeline.")
             return # Exit main function
        print(f"Data preprocessed successfully. Resulting shape: {df_analysis.shape[0]} rows, {df_analysis.shape[1]} columns.")

        # 3. Loop through scenarios and run models
        print("\n--- Step 3: Processing Scenarios ---")
        for scenario in LINREG_SCENARIOS:
            print(f"\n--- Processing Scenario: {scenario.get('name', 'Unnamed Scenario')} ---") # Use .get for safety

            # Call the dedicated function to process this scenario
            scenario_result = process_scenario(scenario, df_analysis)

            # Collect results if model ran successfully
            if scenario_result is not None and not scenario_result.empty:
                all_results_summary.append(scenario_result)
                print(f"  --- Scenario {scenario.get('name', 'Unnamed Scenario')} completed successfully. Results collected. ---")
            else:
                 print(f"  --- Scenario {scenario.get('name', 'Unnamed Scenario')} did not produce valid results or was skipped. ---")

        # 4. Save Aggregated Results
        print("\n--- Step 4: Aggregating and Saving Results ---")
        if all_results_summary:
            # Concatenate all results DataFrames
            results_df = pd.concat(all_results_summary, ignore_index=True)
            print(f"Aggregated results from {len(all_results_summary)} successful scenarios.")

            conn_out = None
            try:
                print(f"Attempting to connect to database: {DB_PATH}")
                conn_out = sqlite3.connect(DB_PATH)
                print(f"Saving aggregated results to table: {OUTPUT_TABLE_NAME}")
                # Use the specific output table name for linear regression
                results_df.to_sql(OUTPUT_TABLE_NAME, conn_out, if_exists='replace', index=False)
                conn_out.commit()
                print("Results saved successfully to the database.")
            except sqlite3.Error as e_sql:
                print(f"ERROR: SQLite error occurred while saving results: {e_sql}")
                if conn_out:
                    try:
                        conn_out.rollback()
                        print("Database transaction rolled back.")
                    except sqlite3.Error as e_rb:
                        print(f"ERROR: Could not rollback transaction: {e_rb}")
            except Exception as e_save:
                print(f"ERROR: An unexpected error occurred saving results: {e_save}")
                if conn_out:
                    try:
                        conn_out.rollback()
                        print("Database transaction rolled back.")
                    except sqlite3.Error as e_rb:
                         print(f"ERROR: Could not rollback transaction: {e_rb}")
                print(traceback.format_exc())
            finally:
                if conn_out:
                    conn_out.close()
                    print("Database connection closed.")
        else:
            print("No valid model results were generated across all scenarios. Nothing to save.")

    except FileNotFoundError as e_fnf:
        print(f"ERROR: File not found during pipeline execution: {e_fnf}")
    except sqlite3.Error as e_sql_main:
        print(f"ERROR: Database error during setup or initial loading: {e_sql_main}")
    except NameError as e_name:
         print(f"ERROR: A required variable or function might be missing or misspelled: {e_name}")
    except KeyError as e_key:
         print(f"ERROR: Missing expected key in scenario configuration or data: {e_key}")
    except Exception as e_main:
        print(f"ERROR: An unexpected error occurred in the main pipeline orchestration: {e_main}")
        print("--- Traceback ---")
        print(traceback.format_exc())
        print("--- End Traceback ---")
    finally:
        print("\n========== Linear Regression Pipeline Finished ==========")


# --- EXECUTION ---
# This block should be outside the main() function definition

if __name__ == "__main__":
     print("Executing script...")
     # Check if running as script vs notebook cell where paper1_directory might be predefined
     if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
          print("'paper1_directory' not found in local/global scope. Checking environment variable...")
          # Attempt to retrieve from environment variable
          paper1_directory = os.getenv('PAPER1_DIRECTORY')
          if paper1_directory is None:
               print("ERROR: 'paper1_directory' variable not defined and 'PAPER1_DIRECTORY' environment variable not set. Cannot run main().")
               # Depending on your setup, you might want to exit here:
               # import sys
               # sys.exit(1)
          else:
               print(f"Using 'paper1_directory' from environment variable: {paper1_directory}")
               main() # Call main only if directory is found
     else:
          # 'paper1_directory' is already defined (e.g., in a notebook cell)
          print(f"Using pre-defined 'paper1_directory': {paper1_directory}")
          main() # Call main

Executing script...
Using pre-defined 'paper1_directory': C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional
========== Starting Linear Regression Pipeline ==========

--- Step 1: Loading Data ---
Loading data from C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite, table sa_input_table...
Loaded 1664 rows and 56 columns.
Data loaded successfully: 1664 rows, 56 columns.

--- Step 2: Preprocessing Data ---
Preprocessing data for Linear models...
Preprocessing complete.
Data preprocessed successfully. Resulting shape: 1664 rows, 56 columns.

--- Step 3: Processing Scenarios ---

--- Processing Scenario: total_wl_hunger_unadjusted ---
  Starting processing for scenario: total_wl_hunger_unadjusted
    Outcome variable: 'total_wl_%'
    Preparing data subset for the model...
      Selected columns: ['total_wl_%', 'hunger_yn']
      Removed 0 rows with missing values (out of 1664). Final dataset size for model: 1664 rows.
    Running linear re

#### Logistic regressions

##### Explanation of the first draft pipeline

Logistic Regression Pipeline Summary

This Python code defines and executes a pipeline for running multiple logistic regression analyses based on a predefined set of scenarios.

**Structure:**

1.  **Configuration:**
    *   Imports necessary libraries (`pandas`, `sqlite3`, `os`, `statsmodels`, `numpy`, `warnings`).
    *   Suppresses `ConvergenceWarning` from `statsmodels`.
    *   Checks for and sets up the `paper1_directory` variable.
    *   Defines constants for the database path (`DB_PATH`), input table name (`INPUT_TABLE_NAME`), and output table name (`OUTPUT_TABLE_NAME`).
    *   Relies on the `LOGREG_SCENARIOS` list (defined in a previous cell) which contains dictionaries, each specifying the parameters for a single logistic regression model (outcome, predictors, covariates, time window, etc.).

2.  **Helper Functions:**
    *   `load_data(db_path, table_name)`: Connects to the specified SQLite database and loads the input table into a pandas DataFrame. Includes error handling for file not found and database errors.
    *   `preprocess_data(df, scenarios)`: Performs initial data cleaning on the loaded DataFrame. It identifies all necessary predictor and covariate columns from the scenarios, attempts to convert boolean-like columns (e.g., ending in `_yn`) to 0/1 integers, and converts other specified columns to numeric types, handling potential errors.

3.  **Core Modeling Function:**
    *   `run_logistic_model(df_subset, outcome_col, predictor_cols, scenario_name)`:
        *   Takes a prepared DataFrame subset, the outcome variable name, and predictor names.
        *   Checks for sufficient data and variance in the outcome variable.
        *   Uses `statsmodels.api.Logit` to fit the logistic regression model (adding a constant for the intercept).
        *   Extracts key results: Coefficients, Odds Ratios (by exponentiating coefficients), 95% Confidence Intervals for ORs, p-values, standard errors, and model-level statistics (N, Log-Likelihood, Pseudo R-squared, AIC, BIC, convergence status).
        *   Returns these results formatted as a pandas DataFrame.
        *   Includes robust error handling for model fitting issues (e.g., `LinAlgError` for multicollinearity, `ValueError`).

4.  **Scenario Processing Function:**
    *   `process_scenario(scenario, df_analysis)`:
        *   Takes a single scenario dictionary and the preprocessed DataFrame.
        *   Dynamically creates the binary outcome variable (e.g., `60d_10p_hunger_unadjusted_outcome`) based on the scenario's `outcome_type`, `target_perc`, and `time_window` specifications (handling weight loss targets and dropout definitions).
        *   Selects the required columns (outcome, predictors, covariates) for the specific scenario.
        *   Handles missing values (`dropna()`) for the selected subset.
        *   Calls `run_logistic_model` with the prepared data for this scenario.
        *   Returns the results DataFrame from `run_logistic_model` or `None` if any step fails.

5.  **Orchestration Function:**
    *   `main()`:
        *   Coordinates the entire pipeline flow.
        *   Calls `load_data` to get the raw data.
        *   Calls `preprocess_data` to clean the data.
        *   Iterates through each `scenario` defined in `LOGREG_SCENARIOS`.
        *   For each scenario, calls `process_scenario`.
        *   Collects the results DataFrames from successfully processed scenarios.
        *   If any results were generated, concatenates them into a single DataFrame.
        *   Connects to the SQLite database and saves the aggregated results DataFrame to the specified `OUTPUT_TABLE_NAME`, replacing it if it exists.
        *   Includes overall error handling for the pipeline steps.

6.  **Execution Block:**
    *   The `if __name__ == "__main__":` block ensures that the `main()` function is called when the code is executed as a script. It also includes logic to handle the definition of `paper1_directory` if it wasn't set previously (e.g., when running in a notebook).

**Workflow:**

The script executes the `main` function, which orchestrates these steps: Load -> Preprocess -> Loop through Scenarios (Define Outcome -> Prepare Subset -> Run Model) -> Aggregate Results -> Save Results. Each scenario defined in `LOGREG_SCENARIOS` results in a separate logistic regression model being fitted and its summary statistics being saved.

##### Define scenarios for the analysis

In [11]:
# Define the scenarios to run for Logistic Regression
# outcome_type: 'target_wl' or 'dropout'
# target_perc: % WL target (used if outcome_type='target_wl')
# time_window: Time window in days ('total', 40, 60, 80, etc.) or 'instant' for dropout
# predictors: List of main predictors (eating behaviors)
# covariates: List of adjustment covariates

LOGREG_SCENARIOS = [
    # 60 days, 10%, eating behavior, UNADJUSTED models
    {
        'name': '60d_10p_hunger_unadjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['hunger_yn'],
        'covariates': []
    },
    {
        'name': '60d_10p_satiety_unadjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['satiety_yn'],
        'covariates': []
    },
    {
        'name': '60d_10p_emotional_eating_unadjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_yn'],
        'covariates': []
    },
    {
        'name': '60d_10p_emotional_eating_value_unadjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_value_likert'],
        'covariates': []
    },
    {
        'name': '60d_10p_quantity_control_unadjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['quantity_control_likert'],
        'covariates': []
    },
    {
        'name': '60d_10p_impulse_control_unadjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['impulse_control_likert'],
        'covariates': []
    },

    # 60 days, 10%, eating behavior, ADJUSTED models
    {
        'name': '60d_10p_hunger_adjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['hunger_yn'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi']
    },
    {
        'name': '60d_10p_satiety_adjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['satiety_yn'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi']
    },
    {
        'name': '60d_10p_emotional_eating_adjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_yn'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi']
    },
    {
        'name': '60d_10p_emotional_eating_value_adjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_value_likert'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi']
    },
    {
        'name': '60d_10p_quantity_control_adjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['quantity_control_likert'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi']
    },
    {
        'name': '60d_10p_impulse_control_adjusted',
        'outcome_type': 'target_wl',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['impulse_control_likert'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi']
    },

    # 60 days, dropout, eating behavior, UNADJUSTED models
    {
        'name': '60d_dropout_hunger_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 60,
        'predictors': ['hunger_yn'],
        'covariates': []
    },
    {
        'name': '60d_dropout_satiety_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 60,
        'predictors': ['satiety_yn'],
        'covariates': []
    },
    {
        'name': '60d_dropout_emotional_eating_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 60,
        'predictors': ['emotional_eating_yn'],
        'covariates': []
    },
    {
        'name': '60d_dropout_emotional_eating_value_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 60,
        'predictors': ['emotional_eating_value_likert'],
        'covariates': []
    },
    {
        'name': '60d_dropout_quantity_control_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 60,
        'predictors': ['quantity_control_likert'],
        'covariates': []
    },
    {
        'name': '60d_dropout_impulse_control_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 60,
        'predictors': ['impulse_control_likert'],
        'covariates': []
    },


    # Predict instant dropout using quantity control (unadjusted)
    {
        'name': 'instant_dropout_quant_ctrl_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 'instant', # Special case using 'nr_visits'
        'predictors': ['quantity_control_likert'],
        'covariates': []
    },

    # Instant, dropout, eating behavior, UNADJUSTED models
    {
        'name': 'instant_dropout_hunger_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 'instant',
        'predictors': ['hunger_yn'],
        'covariates': []
    },
    {
        'name': 'instant_dropout_satiety_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 'instant',
        'predictors': ['satiety_yn'],
        'covariates': []
    },
    {
        'name': 'instant_dropout_emotional_eating_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 'instant',
        'predictors': ['emotional_eating_yn'],
        'covariates': []
    },
    {
        'name': 'instant_dropout_emotional_eating_value_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 'instant',
        'predictors': ['emotional_eating_value_likert'],
        'covariates': []
    },
    {
        'name': 'instant_dropout_quantity_control_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 'instant',
        'predictors': ['quantity_control_likert'],
        'covariates': []
    },
    {
        'name': 'instant_dropout_impulse_control_unadjusted',
        'outcome_type': 'dropout',
        'target_perc': None,
        'time_window': 'instant',
        'predictors': ['impulse_control_likert'],
        'covariates': []
    },


    # # OTHER EXAMPLES AND SETUPS
 
    # # X% achievement during total followup lengths
    # # eg: Predict 10% WL achievement overall (total followup) using emotional eating value (adjusted)
    # {
    #     'name': 'total_10p_wl_emot_val_adjusted',
    #     'outcome_type': 'target_wl',
    #     'target_perc': 10,
    #     'time_window': 'total', # Use 'total_wl_%' column
    #     'predictors': ['emotional_eating_value_likert'],
    #     'covariates': ['age', 'sex_f', 'baseline_bmi']
    # },

    # # NICER WAY OF LISTING SCENARIOS
    # # --- Add your specific scenarios here ---
    # # Example: All 6 variables for 60d/10% WL, adjusted
    # {
    #     'name': '60d_10p_wl_satiety_adjusted',
    #     'outcome_type': 'target_wl', 'target_perc': 10, 'time_window': 60,
    #     'predictors': ['satiety_yn'], 'covariates': ['age', 'sex_f', 'baseline_bmi']
    # },
    # {
    #     'name': '60d_10p_wl_emot_yn_adjusted',
    #     'outcome_type': 'target_wl', 'target_perc': 10, 'time_window': 60,
    #     'predictors': ['emotional_eating_yn'], 'covariates': ['age', 'sex_f', 'baseline_bmi']
    # },
    # {
    #     'name': '60d_10p_wl_emot_val_adjusted',
    #     'outcome_type': 'target_wl', 'target_perc': 10, 'time_window': 60,
    #     'predictors': ['emotional_eating_value_likert'], 'covariates': ['age', 'sex_f', 'baseline_bmi']
    # },
    # {
    #     'name': '60d_10p_wl_quant_ctrl_adjusted',
    #     'outcome_type': 'target_wl', 'target_perc': 10, 'time_window': 60,
    #     'predictors': ['quantity_control_likert'], 'covariates': ['age', 'sex_f', 'baseline_bmi']
    # },
    # {
    #     'name': '60d_10p_wl_impulse_ctrl_adjusted',
    #     'outcome_type': 'target_wl', 'target_perc': 10, 'time_window': 60,
    #     'predictors': ['impulse_control_likert'], 'covariates': ['age', 'sex_f', 'baseline_bmi']
    # },
]

##### Code of the first draft pipeline

In [12]:
####
# Logistic Regression Pipeline Module
####

import pandas as pd
import sqlite3
import os
import statsmodels.api as sm
import numpy as np
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Suppress ConvergenceWarning from statsmodels
warnings.simplefilter('ignore', ConvergenceWarning)

# Ensure paper1_directory is defined in a previous cell
if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
    # Try to get it from the environment if running as a script, or raise error
    paper1_directory = os.getenv('PAPER1_DIRECTORY')
    if paper1_directory is None:
        raise NameError("'paper1_directory' variable not defined. Please ensure it is defined or set as an environment variable 'PAPER1_DIRECTORY'.")
    else:
        print(f"Using paper1_directory from environment variable: {paper1_directory}")

# --- CONFIGURATION ---

# Adjust these paths as needed
DB_PATH = os.path.join(paper1_directory, 'survival_analysis.sqlite') # Path to the SQLite database
INPUT_TABLE_NAME = "sa_input_table" # Your wide input table
OUTPUT_TABLE_NAME = "log_regression_results" # Table to store summary results

# The analysis scenarios are defined in the previous cell

# --- HELPERS ---

def load_data(db_path, table_name):
    """Loads data from the specified SQLite database table."""
    print(f"Loading data from {db_path}, table {table_name}...")
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file not found at {db_path}")
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        print(f"Loaded {len(df)} rows and {len(df.columns)} columns.")
        if df.empty:
             print(f"Warning: Loaded DataFrame from {table_name} is empty.")
        return df
    except sqlite3.Error as e:
        print(f"SQLite error loading data: {e}")
        raise
    except Exception as e:
        print(f"An error occurred loading data: {e}")
        raise
    finally:
        if conn:
            conn.close()

def preprocess_data(df, scenarios):
    """Handles basic data preparation for logistic models."""
    print("Preprocessing data for Logistic models...")
    if df.empty:
         print("  Input DataFrame is empty. Skipping preprocessing.")
         return df

    df_processed = df.copy()

    # Identify all unique predictor/covariate columns needed across scenarios
    all_model_cols = set()
    bool_predictors = set() # Specifically track yes/no predictors
    for scenario in scenarios:
        all_model_cols.update(scenario['predictors'])
        all_model_cols.update(scenario['covariates'])
        # Identify boolean predictors based on common naming or explicit list
        for pred in scenario['predictors']:
            if pred.endswith('_yn'): # Assuming '_yn' suffix for boolean predictors
                 bool_predictors.add(pred)

    # Ensure boolean predictors are 0/1 integers
    for col in bool_predictors:
        if col in df_processed.columns:
            if not pd.api.types.is_numeric_dtype(df_processed[col]) or not df_processed[col].dropna().isin([0, 1]).all():
                print(f"  Attempting to convert boolean predictor '{col}' to 0/1 integer.")
                # Handle potential 'yes'/'no' strings or True/False bools
                if df_processed[col].dtype == 'object':
                    df_processed[col] = df_processed[col].str.lower().map({'yes': 1, 'no': 0, 'y': 1, 'n': 0})
                elif df_processed[col].dtype == 'bool':
                     df_processed[col] = df_processed[col].astype(int)

                # Convert to numeric, coercing errors, then fill NA with a placeholder if needed (e.g., 0 or median)
                # For simplicity here, we'll rely on dropna later, but imputation could be added.
                df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
                if not df_processed[col].dropna().isin([0, 1]).all():
                     print(f"    Warning: Column '{col}' could not be reliably converted to 0/1.")
            # Convert float 0.0/1.0 to integer
            elif pd.api.types.is_float_dtype(df_processed[col]) and df_processed[col].dropna().isin([0.0, 1.0]).all():
                 df_processed[col] = df_processed[col].astype('Int64') # Use nullable integer
        else:
             print(f"  Warning: Boolean predictor column '{col}' specified in scenarios but not found.")

    # Ensure other predictors/covariates are numeric
    numeric_cols_to_check = all_model_cols - bool_predictors
    # Add specific outcome-related columns that need to be numeric
    numeric_cols_to_check.add('nr_visits')
    # Add potential weight loss columns dynamically based on scenarios
    for scenario in scenarios:
        if scenario['outcome_type'] == 'target_wl':
            tw = scenario['time_window']
            wl_col = f"wl_{tw}d_%" if isinstance(tw, int) else "total_wl_%"
            numeric_cols_to_check.add(wl_col)
        elif scenario['outcome_type'] == 'dropout' and isinstance(scenario['time_window'], int):
             dropout_col = f"{scenario['time_window']}d_dropout"
             numeric_cols_to_check.add(dropout_col) # Ensure dropout flags are numeric

    for col in numeric_cols_to_check:
        if col in df_processed.columns:
            if not pd.api.types.is_numeric_dtype(df_processed[col]):
                print(f"  Attempting to convert column '{col}' to numeric.")
                original_nan_count = df_processed[col].isnull().sum()
                df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
                if df_processed[col].isnull().sum() > original_nan_count:
                    print(f"    Warning: Conversion of column '{col}' introduced NaNs.")
        # Don't warn if column doesn't exist, it might only be needed for specific scenarios checked later

    print("Preprocessing complete.")
    return df_processed


# --- LOGISTIC REGRESSION MODULE ---

def run_logistic_model(df_subset, outcome_col, predictor_cols, scenario_name):
    """
    Fits a Logistic Regression model using statsmodels and extracts results.

    Args:
        df_subset (pd.DataFrame): DataFrame containing only necessary columns and no NaNs for the model.
        outcome_col (str): Name of the binary outcome column (0/1).
        predictor_cols (list): List of predictor and covariate column names.
        scenario_name (str): Name of the scenario for labeling outputs.

    Returns:
        pd.DataFrame: DataFrame containing model summary results (OR, CI, p-value per variable),
                      or None if the model fails or data is insufficient.
    """
    print(f"  Running Logistic model for scenario: {scenario_name}")
    print(f"    Outcome: {outcome_col}")
    print(f"    Predictors: {predictor_cols}")
    print(f"    Data shape: {df_subset.shape}")

    # Check for sufficient data
    min_obs_per_predictor = 10
    required_obs = min_obs_per_predictor * len(predictor_cols) if predictor_cols else 20
    if df_subset.shape[0] < max(20, required_obs):
        print(f"    Warning: Insufficient data ({df_subset.shape[0]} obs) for {len(predictor_cols)} predictors. Need at least {max(20, required_obs)}. Skipping model.")
        return None
    if not predictor_cols:
        print("    Warning: No predictors specified. Skipping model.")
        return None
    # Check if outcome column has variance
    if outcome_col not in df_subset or df_subset[outcome_col].nunique() < 2:
        print(f"    Warning: Outcome column '{outcome_col}' has no variation (all 0s or all 1s) or is missing. Skipping model.")
        return None

    results_list = []

    try:
        # Prepare data for statsmodels (add constant for intercept)
        X = df_subset[predictor_cols]
        y = df_subset[outcome_col]
        X = sm.add_constant(X, has_constant='add') # Add intercept

        # Fit the model
        logit_model = sm.Logit(y, X)
        result = logit_model.fit(disp=0) # disp=0 suppresses optimization output

        # Extract Results
        params = result.params
        conf = result.conf_int()
        conf['Odds Ratio'] = params
        conf.columns = ['CI Lower 95%', 'CI Upper 95%', 'Coefficient']
        conf = np.exp(conf) # Exponentiate to get Odds Ratios and their CIs

        p_values = result.pvalues
        # Combine results into a DataFrame
        summary_df = pd.DataFrame({
            'Variable': params.index,
            'Coefficient': params.values,
            'Odds Ratio': conf['Coefficient'],
            'OR CI Lower 95%': conf['CI Lower 95%'],
            'OR CI Upper 95%': conf['CI Upper 95%'],
            'P-Value': p_values.values,
            'Std_Err': result.bse.values # Standard Error
        })

        # Add model-level stats
        model_stats = {
            'scenario_name': scenario_name,
            'outcome_variable': outcome_col,
            'n_observations': int(result.nobs),
            'log_likelihood': result.llf,
            'pseudo_r_squared': result.prsquared,
            'aic': result.aic,
            'bic': result.bic,
            'converged': result.mle_retvals['converged']
        }

        # Add model stats to each row of the summary_df for easy aggregation
        for col, val in model_stats.items():
            summary_df[col] = val

        print("    Model fitting complete.")
        return summary_df

    except np.linalg.LinAlgError as e_linalg:
         print(f"    ERROR: Linear algebra error (likely perfect multicollinearity) for scenario {scenario_name}. Skipping. Error: {e_linalg}")
         return None
    except ValueError as e_val:
         print(f"    ERROR: Value error during model fitting (check data types/values) for scenario {scenario_name}. Skipping. Error: {e_val}")
         return None
    except Exception as e_fit:
        print(f"    ERROR: Failed to fit Logistic model for scenario {scenario_name}. Error: {e_fit}")
        # import traceback # Uncomment for detailed traceback
        # print(traceback.format_exc())
        return None

# --- Scenario Processing Function ---

def process_scenario(scenario, df_analysis):
    """
    Processes a single logistic regression scenario.

    Args:
        scenario (dict): Dictionary containing scenario configuration.
        df_analysis (pd.DataFrame): The preprocessed DataFrame.

    Returns:
        pd.DataFrame or None: DataFrame with model results for the scenario,
                              or None if processing fails or is skipped.
    """
    scenario_name = scenario['name']
    print(f"  Starting processing for scenario: {scenario_name}")

    outcome_type = scenario['outcome_type']
    target_perc = scenario.get('target_perc') # Use .get for safer access
    time_window = scenario['time_window']
    predictors = scenario['predictors']
    covariates = scenario['covariates']
    all_predictors = predictors + covariates

    # --- Define Outcome Variable Dynamically ---
    outcome_col_name = f"{scenario_name}_outcome"
    # Work on a copy to avoid modifying df_analysis across scenarios
    temp_df = df_analysis.copy()
    outcome_defined = False

    try:
        print(f"    Attempting to define outcome variable '{outcome_col_name}'...")
        if outcome_type == 'target_wl':
            if target_perc is None:
                print(f"    ERROR: 'target_perc' must be specified for outcome_type 'target_wl' in scenario '{scenario_name}'. Skipping scenario.")
                return None

            # Determine the source weight loss column
            if time_window == 'total':
                source_wl_col = 'total_wl_%'
            elif isinstance(time_window, int):
                source_wl_col = f'wl_{time_window}d_%'
            else:
                print(f"    ERROR: Invalid 'time_window' ({time_window}) for outcome_type 'target_wl' in scenario '{scenario_name}'. Skipping scenario.")
                return None

            if source_wl_col not in temp_df.columns:
                print(f"    ERROR: Source weight loss column '{source_wl_col}' not found in data for scenario '{scenario_name}'. Skipping scenario.")
                return None

            # Create binary outcome
            print(f"      Defining outcome based on {source_wl_col} >= {target_perc}%")
            temp_df[source_wl_col] = pd.to_numeric(temp_df[source_wl_col], errors='coerce')
            if temp_df[source_wl_col].isnull().all():
                 print(f"      WARNING: Source column '{source_wl_col}' contains only NaN values after numeric conversion.")
            temp_df[outcome_col_name] = (temp_df[source_wl_col] >= target_perc).astype(float) # Use float first
            temp_df[outcome_col_name] = temp_df[outcome_col_name].fillna(0).astype(int) # Fill NaN outcome with 0
            print(f"    Outcome '{outcome_col_name}' created successfully.")
            outcome_defined = True

        elif outcome_type == 'dropout':
            if time_window == 'instant':
                source_dropout_col = 'nr_visits'
                if source_dropout_col not in temp_df.columns:
                     print(f"    ERROR: Column '{source_dropout_col}' needed for instant dropout not found for scenario '{scenario_name}'. Skipping scenario.")
                     return None
                print(f"      Defining outcome based on {source_dropout_col} == 1")
                temp_df[source_dropout_col] = pd.to_numeric(temp_df[source_dropout_col], errors='coerce')
                if temp_df[source_dropout_col].isnull().all():
                     print(f"      WARNING: Source column '{source_dropout_col}' contains only NaN values after numeric conversion.")
                # Handle potential NaNs in nr_visits (treat as non-dropout)
                temp_df[outcome_col_name] = (temp_df[source_dropout_col] == 1).astype(float).fillna(0).astype(int)
                print(f"    Outcome '{outcome_col_name}' created successfully.")
                outcome_defined = True
            elif isinstance(time_window, int):
                source_dropout_col = f'{time_window}d_dropout'
                if source_dropout_col not in temp_df.columns:
                    print(f"    ERROR: Dropout column '{source_dropout_col}' not found for scenario '{scenario_name}'. Skipping scenario.")
                    return None
                print(f"      Defining outcome based on column '{source_dropout_col}' (NaN treated as 0)")
                temp_df[source_dropout_col] = pd.to_numeric(temp_df[source_dropout_col], errors='coerce')
                if temp_df[source_dropout_col].isnull().all():
                     print(f"      WARNING: Source column '{source_dropout_col}' contains only NaN values after numeric conversion.")
                temp_df[outcome_col_name] = temp_df[source_dropout_col].fillna(0).astype(int)
                # Verify it's 0/1
                if not temp_df[outcome_col_name].isin([0, 1]).all():
                     print(f"      WARNING: Dropout column '{source_dropout_col}' contains values other than 0, 1 after cleaning. Check data. Proceeding cautiously.")
                print(f"    Outcome '{outcome_col_name}' created successfully.")
                outcome_defined = True
            else:
                print(f"    ERROR: Invalid 'time_window' ({time_window}) for outcome_type 'dropout' in scenario '{scenario_name}'. Use 'instant' or integer days. Skipping scenario.")
                return None
        else:
            print(f"    ERROR: Invalid outcome_type '{outcome_type}' in scenario '{scenario_name}'. Skipping scenario.")
            return None

    except Exception as e_outcome:
         print(f"    ERROR: Unexpected error creating outcome variable for scenario {scenario_name}: {e_outcome}. Skipping scenario.")
         print(traceback.format_exc()) # Print traceback for debugging
         return None

    if not outcome_defined:
         print(f"    Outcome variable could not be defined for scenario {scenario_name}. Skipping scenario.")
         return None # Should not happen if logic above is correct, but safety check

    # --- Prepare Data for Model ---
    print("    Preparing data subset for the model...")
    # Check if all necessary columns exist
    required_cols = [outcome_col_name] + all_predictors
    missing_cols = [col for col in required_cols if col not in temp_df.columns]
    if missing_cols:
        print(f"    ERROR: Missing required columns for scenario '{scenario_name}': {missing_cols}. Skipping scenario.")
        return None

    # Select relevant columns and drop rows with missing values in predictors/covariates or outcome
    initial_rows = len(temp_df)
    df_model_subset = temp_df[required_cols].dropna()
    final_rows = len(df_model_subset)
    print(f"      Selected columns: {required_cols}")
    print(f"      Removed {initial_rows - final_rows} rows with missing values (out of {initial_rows}). Final dataset size for model: {final_rows} rows.")

    if final_rows == 0:
        print(f"    ERROR: No valid data remaining after handling missing values for scenario '{scenario_name}'. Skipping scenario.")
        return None
    if df_model_subset[outcome_col_name].nunique() < 2:
        print(f"    ERROR: Outcome variable '{outcome_col_name}' has only one unique value ({df_model_subset[outcome_col_name].unique()}) after filtering. Cannot run logistic regression. Skipping scenario.")
        return None

    # --- Run the Logistic Model ---
    print(f"    Running logistic regression model for scenario '{scenario_name}'...")
    model_output_df = None
    try:
        model_output_df = run_logistic_model(
            df_model_subset,
            outcome_col_name,
            all_predictors,
            scenario_name
        )
    except Exception as e_model:
        print(f"    ERROR: An exception occurred during model execution for scenario '{scenario_name}': {e_model}")
        print(traceback.format_exc()) # Print traceback for debugging
        return None # Indicate failure

    if model_output_df is not None and not model_output_df.empty:
        print(f"    Model for scenario '{scenario_name}' ran successfully.")
        return model_output_df
    else:
         print(f"    Model for scenario '{scenario_name}' did not produce valid results (returned None or empty DataFrame).")
         return None

# --- ORCHESTRATION ---

def main():
    """Main function to orchestrate the Logistic Regression pipeline."""
    print("========== Starting Logistic Regression Pipeline ==========")
    all_results_summary = [] # To store summary stats from each model

    try:
        # 1. Load Data
        print("\n--- Step 1: Loading Data ---")
        df_raw = load_data(DB_PATH, INPUT_TABLE_NAME)
        if df_raw is None: # Assume load_data returns None on error
             print("ERROR: Failed to load data. Stopping pipeline.")
             return # Exit main function
        if df_raw.empty:
             print("ERROR: Input data table is empty. Stopping pipeline.")
             return # Exit main function
        print(f"Data loaded successfully: {df_raw.shape[0]} rows, {df_raw.shape[1]} columns.")

        # 2. Preprocess Data
        print("\n--- Step 2: Preprocessing Data ---")
        df_analysis = preprocess_data(df_raw, LOGREG_SCENARIOS)
        if df_analysis is None: # Assume preprocess_data returns None on error
            print("ERROR: Failed during data preprocessing. Stopping pipeline.")
            return # Exit main function
        if df_analysis.empty:
             print("ERROR: Data preprocessing resulted in an empty DataFrame. Stopping pipeline.")
             return # Exit main function
        print(f"Data preprocessed successfully. Resulting shape: {df_analysis.shape[0]} rows, {df_analysis.shape[1]} columns.")

        # 3. Loop through scenarios and run models
        print("\n--- Step 3: Processing Scenarios ---")
        for scenario in LOGREG_SCENARIOS:
            print(f"\n--- Processing Scenario: {scenario.get('name', 'Unnamed Scenario')} ---") # Use .get for safety

            # Call the dedicated function to process this scenario
            scenario_result = process_scenario(scenario, df_analysis)

            # Collect results if model ran successfully
            if scenario_result is not None and not scenario_result.empty:
                all_results_summary.append(scenario_result)
                print(f"  --- Scenario {scenario.get('name', 'Unnamed Scenario')} completed successfully. Results collected. ---")
            else:
                 print(f"  --- Scenario {scenario.get('name', 'Unnamed Scenario')} did not produce valid results or was skipped. ---")

        # 4. Save Aggregated Results
        print("\n--- Step 4: Aggregating and Saving Results ---")
        if all_results_summary:
            # Concatenate all results DataFrames
            results_df = pd.concat(all_results_summary, ignore_index=True)
            print(f"Aggregated results from {len(all_results_summary)} successful scenarios.")

            conn_out = None
            try:
                print(f"Attempting to connect to database: {DB_PATH}")
                conn_out = sqlite3.connect(DB_PATH)
                print(f"Saving aggregated results to table: {OUTPUT_TABLE_NAME}")
                results_df.to_sql(OUTPUT_TABLE_NAME, conn_out, if_exists='replace', index=False)
                conn_out.commit()
                print("Results saved successfully to the database.")
            except sqlite3.Error as e_sql:
                print(f"ERROR: SQLite error occurred while saving results: {e_sql}")
                if conn_out:
                    try:
                        conn_out.rollback()
                        print("Database transaction rolled back.")
                    except sqlite3.Error as e_rb:
                        print(f"ERROR: Could not rollback transaction: {e_rb}")
            except Exception as e_save:
                print(f"ERROR: An unexpected error occurred saving results: {e_save}")
                if conn_out:
                    try:
                        conn_out.rollback()
                        print("Database transaction rolled back.")
                    except sqlite3.Error as e_rb:
                         print(f"ERROR: Could not rollback transaction: {e_rb}")
                print(traceback.format_exc())
            finally:
                if conn_out:
                    conn_out.close()
                    print("Database connection closed.")
        else:
            print("No valid model results were generated across all scenarios. Nothing to save.")

    except FileNotFoundError as e_fnf:
        print(f"ERROR: File not found during pipeline execution: {e_fnf}")
    except sqlite3.Error as e_sql_main:
        print(f"ERROR: Database error during setup or initial loading: {e_sql_main}")
    except NameError as e_name:
         print(f"ERROR: A required variable or function might be missing or misspelled: {e_name}")
    except KeyError as e_key:
         print(f"ERROR: Missing expected key in scenario configuration or data: {e_key}")
    except Exception as e_main:
        print(f"ERROR: An unexpected error occurred in the main pipeline orchestration: {e_main}")
        print("--- Traceback ---")
        print(traceback.format_exc())
        print("--- End Traceback ---")
    finally:
        print("\n========== Logistic Regression Pipeline Finished ==========")


# --- EXECUTION ---
# This block should be outside the main() function definition

if __name__ == "__main__":
     print("Executing script...")
     # Check if running as script vs notebook cell where paper1_directory might be predefined
     if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
          print("'paper1_directory' not found in local/global scope. Checking environment variable...")
          # Attempt to retrieve from environment variable
          paper1_directory = os.getenv('PAPER1_DIRECTORY')
          if paper1_directory is None:
               print("ERROR: 'paper1_directory' variable not defined and 'PAPER1_DIRECTORY' environment variable not set. Cannot run main().")
               # Depending on your setup, you might want to exit here:
               # import sys
               # sys.exit(1)
          else:
               print(f"Using 'paper1_directory' from environment variable: {paper1_directory}")
               # Potentially set DB_PATH or other paths based on paper1_directory here if needed
               # Example: DB_PATH = os.path.join(paper1_directory, 'database', 'my_db.sqlite')
               main() # Call main only if directory is found
     else:
          # 'paper1_directory' is already defined (e.g., in a notebook cell)
          print(f"Using pre-defined 'paper1_directory': {paper1_directory}")
          # Potentially set DB_PATH or other paths based on paper1_directory here if needed
          main() # Call main

Executing script...
Using pre-defined 'paper1_directory': C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional
========== Starting Logistic Regression Pipeline ==========

--- Step 1: Loading Data ---
Loading data from C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite, table sa_input_table...
Loaded 1664 rows and 56 columns.
Data loaded successfully: 1664 rows, 56 columns.

--- Step 2: Preprocessing Data ---
Preprocessing data for Logistic models...
Preprocessing complete.
Data preprocessed successfully. Resulting shape: 1664 rows, 56 columns.

--- Step 3: Processing Scenarios ---

--- Processing Scenario: 60d_10p_hunger_unadjusted ---
  Starting processing for scenario: 60d_10p_hunger_unadjusted
    Attempting to define outcome variable '60d_10p_hunger_unadjusted_outcome'...
      Defining outcome based on wl_60d_% >= 10%
    Outcome '60d_10p_hunger_unadjusted_outcome' created successfully.
    Preparing data subset for the model...
    

#### Kaplan-Meier plots

##### Explanation of the first draft pipeline

Explanation:

plot_kaplan_meier Function:
Takes the DataFrame, column names (duration_col, event_col, strata_col), an optional mapping for legend labels (strata_map), a title prefix, and the output directory.
Creates the output directory if needed.
Crucially, it first selects only the necessary columns and drops rows where any of them are missing (.dropna()). This ensures KM is run on complete data for that specific plot.
It checks if there's enough data and at least two groups to stratify.
It iterates through each unique value in the strata_col.
For each group, it fits the KaplanMeierFitter and plots the survival function on the same axes (ax).
It uses the strata_map to create more readable legend labels if provided.
It performs a logrank_test between the first two groups (for simplicity; extend if needed for >2 groups) to get a p-value comparing the curves.
It sets the title (including the p-value), labels, and saves the plot to a PNG file in the specified directory.
It closes the plot figure (plt.close(fig)) to prevent plots from consuming memory or displaying inline if not desired.
Example Usage:
Preprocessing: It first calculates the median for the Likert scale columns and creates new binary _median_split columns (0 for below median, 1 for median or above). This is done before the loop.
Configuration: Defines the scenarios (targets) and the stratification_vars (including the new median split columns and their label mappings).
Looping: It iterates through each scenario and then through each strata_col.
Column Checks: It checks if the required duration, event, and strata columns exist in the DataFrame before attempting to plot, printing warnings if not.
Function Call: It calls plot_kaplan_meier for each combination.
This setup allows you to easily generate all 18 plots (3 scenarios x 6 stratifications) by running the example usage block after defining your df_analysis DataFrame. Remember to adjust column names (days_to_X%_wl, X%_wl_achieved, etc.) if they differ in your actual DataFrame.

Changes Made:

Clear Config Section: All user-adjustable parameters (paths, scenarios, stratification variables, preprocessing details) are now grouped at the top.
SCENARIOS List: Defines each analysis outcome (5% WL, 10% WL, etc.) with its specific duration and event columns and a readable name.
STRATIFICATION_CONFIG Dictionary: Defines the columns to stratify by and provides mappings for their legend labels. Median split columns are added dynamically during preprocessing.
LIKERT_COLS_FOR_MEDIAN_SPLIT & MEDIAN_SPLIT_MAPPING: Centralizes the logic for creating median split variables.
Dynamic Median Split Addition: The preprocessing step now automatically adds the newly created _median_split columns and their mapping to the STRATIFICATION_CONFIG.
Refactored Main Loop: The loops now iterate through the SCENARIOS list and the STRATIFICATION_CONFIG dictionary, extracting necessary information.
Error Handling: Added checks to ensure required columns (duration, event, strata) exist in the DataFrame before attempting to plot, preventing crashes.
if __name__ == "__main__": block: Encapsulates the execution logic, making the script potentially importable without running the analysis automatically.
Data Loading Example: Included a basic example of loading data from the SQLite database defined in the config.
Filename Sanitization: Added basic sanitization for filenames derived from scenario names and strata columns.
This structure makes it much easier to:

Change the output directory.
Add or remove weight loss targets or dropout scenarios.
Add or remove variables to stratify by.
Adjust the mapping for legend labels.
See all the key parameters in one place.

##### Code of the first draft pipeline

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import os
import sqlite3 # Assuming data is loaded from SQLite

# --- Configuration Section ---

# --- Input/Output ---
# Assuming data is loaded from the same DB as before
DB_PATH = os.path.join(paper1_directory, 'survival_analysis.sqlite') # Path to the SQLite database
INPUT_TABLE_NAME = "sa_input_table" # The table containing all necessary columns
OUTPUT_PLOT_DIR = os.path.join(paper1_directory, 'km_plots')

# --- Analysis Scenarios ---
# Define the different survival outcomes to analyze
# Each dict needs:
#   'name': A short name for the scenario (used in titles/filenames)
#   'duration_col': The column name for time-to-event/censoring
#   'event_col': The column name indicating if the event occurred (1) or not (0)
SCENARIOS = [
    {
        'name': '5% WL',
        'duration_col': 'days_to_5%_wl',
        'event_col': '5%_wl_achieved'
    },
    {
        'name': '10% WL',
        'duration_col': 'days_to_10%_wl',
        'event_col': '10%_wl_achieved'
    },
    {
        'name': '15% WL',
        'duration_col': 'days_to_15%_wl',
        'event_col': '15%_wl_achieved'
    },
    {
        'name': '40d Dropout',
        'duration_col': 'total_followup_days', # Or a specific days_to_60d_measurement if available
        'event_col': '40d_dropout'
    },
    {
        'name': '60d Dropout',
        'duration_col': 'total_followup_days', # Or a specific days_to_60d_measurement if available
        'event_col': '60d_dropout'
    },
    {
        'name': '80d Dropout',
        'duration_col': 'total_followup_days', # Or a specific days_to_60d_measurement if available
        'event_col': '80d_dropout'
    },
]

# --- Stratification Variables ---
# Define the variables to split the population by for the KM plots
# Each key is the column name in the DataFrame.
# The value is a dictionary mapping the column's values to readable labels for the legend.
# Use None or {} for the mapping if the column values are already readable or no mapping is needed.
# Columns requiring median split preprocessing are listed separately below.
STRATIFICATION_CONFIG = {
    'hunger_yn': {0: 'No', 1: 'Yes'},
    'satiety_yn': {0: 'No', 1: 'Yes'},
    'emotional_eating_yn': {0: 'No', 1: 'Yes'},
    # Median split columns will be added here after preprocessing
}

# --- Preprocessing Configuration ---
LIKERT_COLS_FOR_MEDIAN_SPLIT = [
    'emotional_eating_value_likert',
    'quantity_control_likert',
    'impulse_control_likert'
]
MEDIAN_SPLIT_MAPPING = {0: '< Median', 1: '>= Median'} # Common mapping for all median splits

# --- Helper Functions ---

def plot_kaplan_meier(df, duration_col, event_col, strata_col, strata_map, title_prefix, output_dir):
    """
    Generates and saves a Kaplan-Meier plot stratified by a given column.
    (Function definition remains the same as provided previously)

    Args:
        df (pd.DataFrame): DataFrame containing survival data.
        duration_col (str): Name of the column with duration data.
        event_col (str): Name of the column with event status (1=event, 0=censored).
        strata_col (str): Name of the column to stratify by.
        strata_map (dict): Optional dictionary to map strata values to readable labels for the legend.
                           Example: {0: 'No', 1: 'Yes'} or {0: '< Median', 1: '>= Median'}
        title_prefix (str): Prefix for the plot title (e.g., "Time to 10% WL").
        output_dir (str): Directory to save the plot image.
    """
    kmf = KaplanMeierFitter()
    fig, ax = plt.subplots(figsize=(8, 6))
    log_rank_results = None

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # --- Data Preparation for this specific plot ---
    plot_df = df[[duration_col, event_col, strata_col]].dropna()

    if plot_df.empty or plot_df[strata_col].nunique() < 2:
        print(f"Warning: Insufficient data or groups for stratification '{strata_col}'. Skipping plot '{title_prefix} by {strata_col}'.")
        plt.close(fig) # Close the empty figure
        return

    # --- Fit and Plot for each group ---
    all_groups_results = []
    unique_strata = sorted(plot_df[strata_col].unique()) # Ensure consistent order

    for name in unique_strata:
        grouped_df = plot_df[plot_df[strata_col] == name]
        if grouped_df.empty:
            continue # Should not happen with dropna and nunique check, but safety first

        label = strata_map.get(name, name) if strata_map else name # Use mapping if provided
        kmf.fit(grouped_df[duration_col], event_observed=grouped_df[event_col], label=f"{strata_col}: {label}")
        kmf.plot_survival_function(ax=ax)
        # Store data for log-rank test
        all_groups_results.append({
            'durations': grouped_df[duration_col],
            'events': grouped_df[event_col],
            'label': label # Store label for potential error messages
        })

    # --- Perform Log-Rank Test ---
    if len(all_groups_results) >= 2:
        try:
            # Simple pairwise log-rank for the first two groups found
            # For >2 groups, consider pairwise or other methods if needed
            log_rank_results = logrank_test(
                durations_A=all_groups_results[0]['durations'],
                durations_B=all_groups_results[1]['durations'],
                event_observed_A=all_groups_results[0]['events'],
                event_observed_B=all_groups_results[1]['events']
            )
            p_value = log_rank_results.p_value
            p_value_text = f"Log-Rank p={p_value:.3f}"
            if len(all_groups_results) > 2:
                 p_value_text += f" ({all_groups_results[0]['label']} vs {all_groups_results[1]['label']})"

        except Exception as e:
            print(f"Warning: Could not perform log-rank test for {strata_col}. Error: {e}")
            p_value_text = "Log-Rank: Error"
    else:
        p_value_text = "Log-Rank: N/A (<=1 group)"


    # --- Finalize Plot ---
    plt.title(f"{title_prefix} by {strata_col}\n{p_value_text}")
    plt.xlabel("Days")
    plt.ylabel("Survival Probability (Not Achieving Target)")
    plt.ylim(0, 1.05)
    plt.grid(True, linestyle='--', alpha=0.6)
    # Generate a nicer legend title from the column name
    legend_title = strata_col.replace('_', ' ').replace(' yn', '').replace(' median split', '').title()
    plt.legend(title=legend_title)

    # --- Save Plot ---
    # Sanitize scenario name and strata column for filename
    safe_scenario_name = scenario['name'].replace('%', 'perc').replace(' ', '_').lower()
    safe_strata_col = strata_col.replace('%', 'perc').replace(' ', '_').lower()
    filename = f"km_{safe_scenario_name}_by_{safe_strata_col}.png"
    filepath = os.path.join(output_dir, filename)
    try:
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Saved plot: {filepath}")
    except Exception as e:
        print(f"Error saving plot {filepath}: {e}")
    plt.close(fig) # Close the figure to free memory


# --- Main Execution Logic ---

if __name__ == "__main__": # Ensures this runs only when script is executed directly

    # --- 1. Load Data ---
    print(f"Connecting to database: {DB_PATH}")
    try:
        con = sqlite3.connect(DB_PATH)
        print(f"Loading table: {INPUT_TABLE_NAME}")
        df_analysis = pd.read_sql_query(f"SELECT * FROM {INPUT_TABLE_NAME}", con)
        con.close()
        print(f"Data loaded successfully. Shape: {df_analysis.shape}")
    except Exception as e:
        print(f"Error loading data: {e}")
        exit() # Or handle error appropriately

    # --- 2. Preprocessing: Create Median Split Columns ---
    print("\n--- Starting Preprocessing ---")
    df_analysis_processed = df_analysis.copy() # Work on a copy

    for col in LIKERT_COLS_FOR_MEDIAN_SPLIT:
        median_split_col_name = f'{col}_median_split'
        if col in df_analysis_processed.columns:
            # Ensure column is numeric before calculating median
            numeric_col = pd.to_numeric(df_analysis_processed[col], errors='coerce')
            median_val = numeric_col.median()
            if pd.notna(median_val):
                # Create the 0/1 split column
                df_analysis_processed[median_split_col_name] = (numeric_col >= median_val).astype(int)
                # Add this new column to the stratification config
                STRATIFICATION_CONFIG[median_split_col_name] = MEDIAN_SPLIT_MAPPING
                print(f"Created median split for '{col}' as '{median_split_col_name}' (Median: {median_val})")
            else:
                print(f"Warning: Could not calculate median for '{col}' (all NA or non-numeric?). Skipping split.")
                # Optionally create an all-NA column if needed downstream
                # df_analysis_processed[median_split_col_name] = np.nan
        else:
            print(f"Warning: Column '{col}' not found for median split.")
            # Optionally create an all-NA column if needed downstream
            # df_analysis_processed[median_split_col_name] = np.nan

    print("--- Preprocessing Finished ---")


    # --- 3. Generate Plots based on Configuration ---
    print(f"\n--- Generating KM Plots (Output Dir: {OUTPUT_PLOT_DIR}) ---")
    for scenario in SCENARIOS:
        scenario_name = scenario['name']
        duration_col = scenario['duration_col']
        event_col = scenario['event_col']

        # Check if necessary columns exist for the scenario
        if duration_col not in df_analysis_processed.columns or event_col not in df_analysis_processed.columns:
            print(f"\nWarning: Skipping scenario '{scenario_name}' - Columns '{duration_col}' or '{event_col}' not found in DataFrame.")
            continue

        print(f"\n-- Scenario: {scenario_name} --")

        for strata_col, strata_map in STRATIFICATION_CONFIG.items():
            if strata_col not in df_analysis_processed.columns:
                 print(f"Warning: Stratification column '{strata_col}' not found for scenario '{scenario_name}'. Skipping plot.")
                 continue

            # Call the plotting function using configured parameters
            plot_kaplan_meier(
                df=df_analysis_processed,
                duration_col=duration_col,
                event_col=event_col,
                strata_col=strata_col,
                strata_map=strata_map,
                title_prefix=f"Time to {scenario_name}",
                output_dir=OUTPUT_PLOT_DIR
            )

    print("\n--- KM Plot generation finished. ---")


Connecting to database: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite
Loading table: sa_input_table
Data loaded successfully. Shape: (1664, 56)

--- Starting Preprocessing ---
Created median split for 'emotional_eating_value_likert' as 'emotional_eating_value_likert_median_split' (Median: 7.0)
Created median split for 'quantity_control_likert' as 'quantity_control_likert_median_split' (Median: 6.0)
Created median split for 'impulse_control_likert' as 'impulse_control_likert_median_split' (Median: 6.0)
--- Preprocessing Finished ---

--- Generating KM Plots (Output Dir: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\km_plots) ---

-- Scenario: 5% WL --
Saved plot: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\km_plots\km_5perc_wl_by_hunger_yn.png
Saved plot: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\km_plots\km_5perc_wl_by_satiety_yn.png
Saved plot: C:\Users\Felhasználó\Desktop\Projects\PNK_D

#### Cox

##### Summary of the pipeline

COX


Let's refine the pipeline structure specifically for running and evaluating Cox models based on your sa_input_table.

Refined Cox Regression Pipeline Structure:

CONFIG:

DB_PATH: Path to your survival_analysis.sqlite database.
INPUT_TABLE_NAME: "sa_input_table"
OUTPUT_TABLE_NAME: "cox_results" (or similar, for storing model summaries)
ASSUMPTION_PLOT_DIR: Folder path to save PH assumption plots.
SCENARIOS: A list of dictionaries, where each dictionary defines a specific analysis run. This makes it easy to loop through.
HELPERS:

load_data(db_path, table_name): Loads the sa_input_table.
preprocess_data(df): Handles final prep needed specifically for Cox:
Ensure categorical predictors/covariates like sex, hunger, satiety, emotional_eating are numerically encoded (e.g., 0/1). This is where you'd implement the conversion if not done upstream.
Handle any remaining missing values in the columns needed for the models (e.g., using listwise deletion for simplicity initially, df.dropna(subset=columns_for_model)).
Maybe create median split flags here if you want to run models stratified by them (though using continuous is often preferred in Cox).
COX REGRESSION MODULE:

run_cox_model(df, duration_col, event_col, predictor_cols, scenario_name, plot_dir):
Input: DataFrame subset with necessary columns, duration column name, event column name, list of predictor/covariate column names, scenario name (for plot saving), plot directory.
Actions:
Instantiate CoxPHFitter from lifelines.
Fit the model: cph.fit(df, duration_col=duration_col, event_col=event_col, formula=" + ".join(predictor_cols)) (using formula is often cleaner).
Check Proportional Hazards Assumption:
Call cph.check_assumptions(df, show_plots=False, p_value_threshold=0.05). This returns results and can optionally plot.
Save assumption check summary (p-values).
Optionally, generate and save Schoenfeld residual plots for key variables: cph.plot_covariate_groups(...) or manually plot residuals over time. Save plots to plot_dir using scenario_name.
Extract results: cph.summary (Hazard Ratios, CIs, p-values), concordance index (cph.concordance_index_), assumption check results.
Output: A dictionary containing the model summary (as DataFrame/dict), concordance index, and assumption check results.
ORCHESTRATION & EXECUTION:

Load data using load_data.
Preprocess data using preprocess_data.
Initialize an empty list all_results.
Loop through each scenario in SCENARIOS:
Print which scenario is running.
Determine duration_col and event_col based on scenario['event_type']:
If event_type == 'target':
duration_col = f"days_to_{scenario['target_perc']}_perc_wl"
event_col = f"{scenario['target_perc']}_perc_wl_achieved" (ensure this is 0/1 or bool)
If event_type == 'dropout':
duration_col = 'total_followup_days' (or specific window like days_to_60d_measurement if appropriate)
event_col = f"{scenario['time_window']}d_dropout" (ensure this is 0/1 or bool)
Define predictor_cols = scenario['predictors'] + scenario['covariates'].
Select relevant columns and handle missing data for this specific model: df_model = df_analysis[ [duration_col, event_col] + predictor_cols ].dropna()
Check if df_model is empty after dropping NAs. If so, skip.
Call run_cox_model with the appropriate arguments.
Store the returned results dictionary, adding the scenario_name to it. Append to all_results.
Save Results:
Convert all_results (list of dicts, where each dict contains model summary etc.) into a structured format. Saving the cph.summary DataFrame directly might be tricky if combining results. A better approach might be to extract key info (variable, HR, CI_lower, CI_upper, p-value, concordance, assumption_p_value) for each scenario into a new summary DataFrame.
Connect to the SQLite DB and save this summary DataFrame to the OUTPUT_TABLE_NAME (e.g., cox_results), perhaps using df.to_sql(..., if_exists='append').
Key Considerations:

Assumption Checking: Yes, the PH assumption needs to be checked for each model you fit, as it depends on the specific covariates included. lifelines makes this relatively easy. If the assumption is violated for a key predictor:
Report it as a limitation.
Consider stratification (if the violation is due to a categorical variable).
Consider time-varying covariates/coefficients (more advanced).
Consider alternative models (like accelerated failure time models).
Data Types: Ensure your duration columns are numeric (days) and event columns are boolean or 0/1. Categorical predictors must be numerically encoded (dummy variables or 0/1).
Interpretation: Focus on the Hazard Ratios (HRs) and their confidence intervals. An HR > 1 means increased hazard (faster event rate), HR < 1 means decreased hazard (slower event rate) for a one-unit increase in the predictor, holding others constant.
Saving Output: Saving key summary stats (HR, CI, p-value per variable per model) to a single SQL table is efficient for later aggregation and comparison. Saving plots separately is necessary.
This structure provides a modular way to run your exploratory Cox models, check assumptions systematically, and store the results. Remember to handle the upstream data type conversions (like sex to numeric) either in the sa_input_table creation or in the preprocess_data helper function here.

##### Explanation of the 1st draft pipeline

Explanation:

Configuration: Sets up paths, table names, and the SCENARIOS list which drives the analysis.
Helpers:
load_data: Standard function to load a table from SQLite.
preprocess_data: Crucial step. This function needs to be adapted to your specific data. It currently includes examples for converting 'sex', 'hunger', etc., to 0/1. You must review and modify this section to correctly handle all your categorical predictors and ensure event/duration columns are numeric.
Cox Regression Module:
run_cox_model: Takes a prepared DataFrame subset (only relevant columns, no NaNs), fits the CoxPHFitter, checks assumptions using check_assumptions, extracts key results (HR, CI, p-values, concordance), and saves assumption plots if violations occur. It uses a try...except block to handle potential convergence errors during fitting.
Orchestration & Execution (main function):
Loads the raw data.
Calls preprocess_data.
Loops through each scenario dictionary in SCENARIOS.
Determines the correct duration_col and event_col based on event_type.
Selects the necessary columns for the current model.
Uses .dropna() to perform listwise deletion for that specific model.
Calls run_cox_model.
If the model runs successfully, it extracts key summary statistics (HR, CI, p-value for each variable, plus overall concordance, N, etc.) and appends them to the all_results_summary list.
Finally, it converts all_results_summary into a pandas DataFrame and saves it to a new table (cox_regression_results) in your SQLite database.

##### Code of the 1st draft pipeline - REVISE PREPROCESS and FIX LIFELINES INSTALLATION ERROR

In [13]:
####
# Cox Regression Pipeline Module
####

import pandas as pd
import sqlite3
import os
# Ensure paper1_directory is defined in a previous cell
# Example: paper1_directory = r"C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional"
if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
    raise NameError("'paper1_directory' variable not defined. Please ensure it is defined in a previous cell.")

from lifelines import CoxPHFitter
from lifelines.exceptions import ConvergenceError
import matplotlib.pyplot as plt # For saving plots
import numpy as np

# --- CONFIGURATION --

# Adjust these paths as needed
# --- MODIFIED: Use the paper1_directory VARIABLE ---
DB_PATH = os.path.join(paper1_directory, 'survival_analysis.sqlite') # Path to the SQLite database
ASSUMPTION_PLOT_DIR = os.path.join(paper1_directory, 'cox_assumption_plots')

INPUT_TABLE_NAME = "sa_input_table" # Your wide input table
OUTPUT_TABLE_NAME = "cox_regression_results" # Table to store summary results

# Ensure the plot directory exists
os.makedirs(ASSUMPTION_PLOT_DIR, exist_ok=True)

# Define the scenarios to run
# IMPORTANT: Ensure column names for duration/event/predictors/covariates match your sa_input_table EXACTLY.
# IMPORTANT: Ensure categorical variables are numerically encoded (0/1) in sa_input_table.
# --- MODIFIED: Updated variable names ---
SCENARIOS = [
     
    # 60d10p, 6 VARS ONE BY ONE, UNADJUSTED
    
    {
        'name': '60d_10p_hunger_unadjusted', # Descriptive name for the analysis run
        'event_type': 'target',                # 'target' or 'dropout'
        'target_perc': 10,                     # % WL target (used if event_type='target')
        'time_window': 60,                     # Time window in days (used if event_type='dropout')
        'predictors': ['hunger_yn'], # List of main predictors
        'covariates': [] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m' # List of adjustment covariates
    },
    {
        'name': '60d_10p_satiety_unadjusted',
        'event_type': 'target',
        'target_perc': 10, 
        'time_window': 60,
        'predictors': ['satiety_yn'],
        'covariates': [] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_emotional_eating_unadjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_yn'],
        'covariates': [] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_emotional_eating_value_unadjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_value_likert'],
        'covariates': [] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_quantity_control_unadjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['quantity_control_likert'],
        'covariates': [] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_impulse_control_unadjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['impulse_control_likert'],
        'covariates': [] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },

    # 60d10p, 6 VARS ONE BY ONE, ADJUSTED

    {
        'name': '60d_10p_hunger_adjusted', # Descriptive name for the analysis run
        'event_type': 'target',                # 'target' or 'dropout'
        'target_perc': 10,                     # % WL target (used if event_type='target')
        'time_window': 60,                     # Time window in days (used if event_type='dropout')
        'predictors': ['hunger_yn'], # List of main predictors
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m' # List of adjustment covariates
    },
    {
        'name': '60d_10p_satiety_adjusted',
        'event_type': 'target',
        'target_perc': 10, 
        'time_window': 60,
        'predictors': ['satiety_yn'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_emotional_eating_adjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_yn'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_emotional_eating_value_adjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['emotional_eating_value_likert'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_quantity_control_adjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['quantity_control_likert'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },
    {
        'name': '60d_10p_impulse_control_adjusted',
        'event_type': 'target',
        'target_perc': 10,
        'time_window': 60,
        'predictors': ['impulse_control_likert'],
        'covariates': ['age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m'
    },


    # --- Add more scenarios as needed ---
    # Example: Dropout model
    # {
    #     'name': '60d_dropout_emotional_eating_unadjusted',
    #     'event_type': 'dropout',
    #     'target_perc': None, # Not relevant for dropout event type
    #     'time_window': 60,
    #     'predictors': ['emotional_eating_yn'],
    #     'covariates': [] #'age', 'sex_f', 'baseline_weight_kg', 'baseline_bmi', 'height_m' # No covariates for unadjusted model
    # },
]

# --- HELPERS ---

def load_data(db_path, table_name):
    """Loads data from the specified SQLite database table."""
    print(f"Loading data from {db_path}, table {table_name}...")
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file not found at {db_path}")
    conn = None
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
        print(f"Loaded {len(df)} rows and {len(df.columns)} columns.")
        # --- NEW: Add check for empty DataFrame ---
        if df.empty:
             print(f"Warning: Loaded DataFrame from {table_name} is empty.")
        return df
    except sqlite3.Error as e:
        print(f"SQLite error loading data: {e}")
        raise
    except Exception as e:
        print(f"An error occurred loading data: {e}")
        raise
    finally:
        if conn:
            conn.close()

def preprocess_data(df, scenarios):
    """Handles final data preparation specifically for Cox models."""
    print("Preprocessing data for Cox models...")
    if df.empty:
         print("  Input DataFrame is empty. Skipping preprocessing.")
         return df # Return empty df

    df_processed = df.copy()

    # --- Data Type Conversion ---
    # Identify all unique predictor/covariate columns needed across scenarios
    all_model_cols = set()
    for scenario in scenarios:
        all_model_cols.update(scenario['predictors'])
        all_model_cols.update(scenario['covariates'])

    # --- REMOVED: Categorical map and conversion loop ---
    # categorical_map = { ... }
    # for col, mapping in categorical_map.items(): ...

    # Ensure boolean columns used as events are 0/1 integers
    event_cols_to_check = set()
    for scenario in scenarios:
         if scenario['event_type'] == 'target':
              event_col = f"{scenario['target_perc']}%_wl_achieved"
              event_cols_to_check.add(event_col)
         elif scenario['event_type'] == 'dropout':
              event_col = f"{scenario['time_window']}d_dropout"
              event_cols_to_check.add(event_col)

    for col in event_cols_to_check:
         if col in df_processed.columns:
              # --- MODIFIED: Check if column is NOT 0/1 already ---
              if not df_processed[col].isin([0, 1, np.nan]).all():
                   print(f"  Warning: Event column '{col}' contains values other than 0, 1, or NaN. Attempting conversion.")
                   # Try converting boolean True/False to 1/0
                   if df_processed[col].dtype == 'bool':
                        print(f"    Converting boolean event column '{col}' to 0/1 integer.")
                        df_processed[col] = df_processed[col].astype(int)
                   else:
                        # Attempt numeric conversion, coercing errors
                        original_nan_count = df_processed[col].isnull().sum()
                        df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
                        # Check if conversion failed or resulted in non 0/1 values
                        if not df_processed[col].isin([0, 1, np.nan]).all():
                             print(f"    ERROR: Event column '{col}' could not be reliably converted to 0/1. Cox model may fail.")
                        elif df_processed[col].isnull().sum() > original_nan_count:
                             print(f"    Warning: Conversion of event column '{col}' introduced NaNs.")

              # --- Optional: Convert float 0.0/1.0 to integer ---
              elif pd.api.types.is_float_dtype(df_processed[col]) and df_processed[col].dropna().isin([0.0, 1.0]).all():
                   print(f"  Converting float event column '{col}' (containing only 0.0/1.0) to integer.")
                   # Use nullable integer type Int64 to handle potential NaNs gracefully
                   df_processed[col] = df_processed[col].astype('Int64')

         else:
              print(f"  Warning: Event column '{col}' specified in scenarios but not found in DataFrame.")


    # Ensure duration columns are numeric
    duration_cols_to_check = set()
    for scenario in scenarios:
         if scenario['event_type'] == 'target':
              duration_col = f"days_to_{scenario['target_perc']}%_wl"
              duration_cols_to_check.add(duration_col)
         elif scenario['event_type'] == 'dropout':
              # Assuming 'total_followup_days' is the primary duration for dropout
              duration_cols_to_check.add('total_followup_days')
              # Add specific window days if used, e.g., f'days_to_{scenario["time_window"]}d_measurement'
              # duration_cols_to_check.add(f'days_to_{scenario["time_window"]}d_measurement')


    for col in duration_cols_to_check:
         if col in df_processed.columns:
              if not pd.api.types.is_numeric_dtype(df_processed[col]):
                   print(f"  Attempting to convert duration column '{col}' to numeric.")
                   original_nan_count = df_processed[col].isnull().sum()
                   df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
                   if df_processed[col].isnull().sum() > original_nan_count:
                        print(f"    Warning: Conversion of duration column '{col}' introduced NaNs.")
              # --- NEW: Check for negative durations ---
              if pd.api.types.is_numeric_dtype(df_processed[col]) and (df_processed[col] < 0).any():
                   print(f"  Warning: Negative values detected in duration column '{col}'. These might cause issues.")
         else:
              print(f"  Warning: Duration column '{col}' specified in scenarios but not found in DataFrame.")


    # --- Missing Value Handling (Example: Simple Listwise Deletion per model later) ---
    # No global handling here; will be done per model based on required columns.
    # You could add imputation here if preferred, but listwise deletion per model is often safer initially.
    print("Preprocessing complete.")
    return df_processed

# --- COX REGRESSION MODULE ---

def run_cox_model(df_subset, duration_col, event_col, predictor_cols, scenario_name, plot_dir):
    """
    Fits a Cox model, checks assumptions, and extracts results.

    Args:
        df_subset (pd.DataFrame): DataFrame containing only necessary columns and no NaNs for the model.
        duration_col (str): Name of the duration column.
        event_col (str): Name of the event column (0/1).
        predictor_cols (list): List of predictor and covariate column names.
        scenario_name (str): Name of the scenario for labeling outputs.
        plot_dir (str): Directory to save assumption plots.

    Returns:
        dict: Dictionary containing model summary, concordance, and assumption results.
              Returns None if the model fails to converge or df_subset is too small.
    """
    print(f"  Running Cox model for scenario: {scenario_name}")
    print(f"    Duration: {duration_col}, Event: {event_col}")
    print(f"    Predictors: {predictor_cols}")
    print(f"    Data shape: {df_subset.shape}")

    # --- MODIFIED: Stricter check for sufficient data relative to predictors ---
    min_subjects_per_predictor = 10
    required_subjects = min_subjects_per_predictor * len(predictor_cols) if predictor_cols else 10
    if df_subset.shape[0] < max(10, required_subjects) or df_subset.shape[1] < 2:
         print(f"    Warning: Insufficient data ({df_subset.shape[0]} subjects) for {len(predictor_cols)} predictors. Need at least {max(10, required_subjects)}. Skipping model.")
         return None
    if not predictor_cols:
         print("    Warning: No predictors specified. Skipping model.")
         return None
    # --- NEW: Check if event column has variance ---
    if event_col in df_subset and df_subset[event_col].nunique() < 2:
         print(f"    Warning: Event column '{event_col}' has no variation (all 0s or all 1s). Skipping model.")
         return None
    # --- NEW: Check if duration column has variance ---
    if duration_col in df_subset and df_subset[duration_col].nunique() < 2:
         print(f"    Warning: Duration column '{duration_col}' has no variation. Skipping model.")
         return None


    cph = CoxPHFitter()
    results = {'scenario_name': scenario_name}

    try:
        # Use formula for cleaner specification if predictor_cols is not empty
        # --- MODIFIED: Ensure backticks handle various column names ---
        formula = " + ".join([f"`{str(col).replace('`', '')}`" for col in predictor_cols])
        cph.fit(df_subset, duration_col=duration_col, event_col=event_col, formula=formula)

        # Extract Results
        results['summary'] = cph.summary.reset_index().rename(columns={'covariate': 'Variable'}) # Get summary table
        results['concordance'] = cph.concordance_index_
        results['log_likelihood'] = cph.log_likelihood_
        results['n_subjects'] = cph.durations.shape[0]
        results['n_events'] = int(cph.event_observed.sum()) # Ensure integer


        # Check Proportional Hazards Assumption
        print("    Checking proportional hazards assumption...")
        try:
            # Setting show_plots=False prevents plots from displaying in notebook during check
            # --- MODIFIED: Pass dataframe subset used for fitting ---
            assumption_results = cph.check_assumptions(df_subset[[duration_col, event_col] + predictor_cols], show_plots=False, p_value_threshold=0.05)
            assumption_summary = {}
            violated_vars = []
            # --- MODIFIED: Safer iteration over assumption results ---
            for result_tuple in assumption_results:
                 if len(result_tuple) >= 4: # Ensure tuple has expected elements
                     var = result_tuple[0]
                     test_stat = result_tuple[1]
                     p_val = result_tuple[2]
                     is_proportional = result_tuple[3]
                     assumption_summary[f"{var}_ph_test_stat"] = test_stat
                     assumption_summary[f"{var}_ph_p_value"] = p_val
                     assumption_summary[f"{var}_is_proportional"] = is_proportional
                     if not is_proportional:
                          violated_vars.append(var)
                 else:
                      print(f"    Warning: Unexpected format in assumption results tuple: {result_tuple}")


            results['assumption_summary'] = assumption_summary
            results['ph_assumption_violated'] = len(violated_vars) > 0
            results['ph_violated_variables'] = ", ".join(violated_vars) if violated_vars else None

            # Optionally, save Schoenfeld residual plots for violated variables
            if violated_vars:
                print(f"    PH Assumption Violated for: {violated_vars}. Saving plots...")
                for var in violated_vars:
                     try:
                          fig, ax = plt.subplots(figsize=(8, 5))
                          # --- MODIFIED: Use plot_partial_effects_on_outcome for better visualization ---
                          # cph.plot_covariate_groups(var, values=np.percentile(df_subset[var].dropna(), [25, 75]), ax=ax) # Old method
                          cph.plot_partial_effects_on_outcome(covariates=var, values=np.percentile(df_subset[var].dropna(), [10, 50, 90]), cmap='coolwarm', ax=ax)
                          plot_filename = os.path.join(plot_dir, f"{scenario_name}_{var}_partial_effects.png") # Updated filename
                          plt.tight_layout() # Adjust layout
                          plt.savefig(plot_filename)
                          plt.close(fig) # Close the figure to prevent display in notebook
                          print(f"      Saved plot: {plot_filename}")
                     except Exception as plot_err:
                          print(f"      Error generating/saving plot for {var}: {plot_err}")


        except ValueError as e_assump_val:
             # Catch specific ValueError often related to singular matrix in PH test
             print(f"    Error during assumption check (likely data issue, e.g., low variance): {e_assump_val}")
             results['assumption_summary'] = {'error': f"ValueError: {e_assump_val}"}
             results['ph_assumption_violated'] = None
             results['ph_violated_variables'] = None
        except Exception as e_assump:
            print(f"    Error during assumption check: {e_assump}")
            results['assumption_summary'] = {'error': str(e_assump)}
            results['ph_assumption_violated'] = None
            results['ph_violated_variables'] = None

        print("    Model fitting and assumption check complete.")

    except ConvergenceError as e_conv:
        print(f"    ERROR: Cox model failed to converge for scenario {scenario_name}. Skipping. Error: {e_conv}")
        return None # Indicate failure
    except Exception as e_fit:
        print(f"    ERROR: Failed to fit Cox model for scenario {scenario_name}. Error: {e_fit}")
        import traceback
        print(traceback.format_exc()) # Uncomment for detailed traceback
        return None # Indicate failure

    return results


# --- ORCHESTRATION & EXECUTION ---

def main():
    """Main function to orchestrate the Cox regression pipeline."""
    print("========== Starting Cox Regression Pipeline ==========")
    all_results_summary = [] # To store key summary stats from each model

    try:
        # 1. Load Data
        df_raw = load_data(DB_PATH, INPUT_TABLE_NAME)
        if df_raw.empty:
             print("ERROR: Input data table is empty. Stopping pipeline.")
             return # Stop if no data loaded

        # 2. Preprocess Data
        df_analysis = preprocess_data(df_raw, SCENARIOS)
        if df_analysis.empty:
             print("ERROR: Data preprocessing resulted in an empty DataFrame. Stopping pipeline.")
             return # Stop if preprocessing failed

        # 3. Loop through scenarios and run models
        for scenario in SCENARIOS:
            scenario_name = scenario['name']
            print(f"\n--- Processing Scenario: {scenario_name} ---")

            # Determine duration and event columns based on event_type
            duration_col_orig = None # Original duration (e.g., days_to_X%_wl)
            event_col = None
            if scenario['event_type'] == 'target':
                if scenario['target_perc'] is None:
                     print("  Error: 'target_perc' must be specified for event_type 'target'. Skipping.")
                     continue
                duration_col_orig = f"days_to_{scenario['target_perc']}%_wl"
                event_col = f"{scenario['target_perc']}%_wl_achieved"
            elif scenario['event_type'] == 'dropout':
                if scenario['time_window'] is None:
                     print("  Error: 'time_window' must be specified for event_type 'dropout'. Skipping.")
                     continue
                # Decide which duration column to use for dropout analysis
                # Option 1: Use total follow-up time
                duration_col_orig = 'total_followup_days'
                # Option 2: Use time to a specific measurement (if available)
                # duration_col_orig = f'days_to_{scenario["time_window"]}d_measurement'
                event_col = f"{scenario['time_window']}d_dropout"
            else:
                print(f"  Error: Invalid event_type '{scenario['event_type']}'. Skipping.")
                continue

            # Define all columns needed for this specific model
            predictor_cols = scenario['predictors'] + scenario['covariates']
            # --- MODIFICATION START ---
            # Define base columns needed BEFORE creating the specific duration
            base_cols_needed = [event_col] + predictor_cols
            if scenario['event_type'] == 'target':
                base_cols_needed.append(duration_col_orig) # Need the time-to-event column
                if 'total_followup_days' not in df_analysis.columns:
                     print("  Error: 'total_followup_days' column is required for censoring in target events but not found. Skipping.")
                     continue
                base_cols_needed.append('total_followup_days') # Need total followup for censoring
            elif scenario['event_type'] == 'dropout':
                 base_cols_needed.append(duration_col_orig) # Need the duration column for dropout (e.g., total_followup_days)

            # Check if all necessary base columns exist
            missing_cols = [col for col in base_cols_needed if col not in df_analysis.columns]
            if missing_cols:
                print(f"  Error: Missing required columns for this scenario: {missing_cols}. Skipping.")
                continue

            # Select initial subset with base columns
            df_model_subset_initial = df_analysis[base_cols_needed].copy()

            # Create the duration column specifically for the model
            duration_for_model_col = f"{scenario_name}_duration" # Unique name for temp duration

            if scenario['event_type'] == 'target':
                 # Use time-to-event if event occurred (1), otherwise use total follow-up time
                 df_model_subset_initial[duration_for_model_col] = np.where(
                      df_model_subset_initial[event_col] == 1,
                      df_model_subset_initial[duration_col_orig],       # Use days_to_X%_wl
                      df_model_subset_initial['total_followup_days'] # Use total_followup_days
                 )
                 # Ensure the original duration_col (days_to_X%_wl) isn't NaN if event is 1
                 check_event1_nan_duration = df_model_subset_initial[
                     (df_model_subset_initial[event_col] == 1) &
                     (df_model_subset_initial[duration_col_orig].isnull())
                 ].shape[0]
                 if check_event1_nan_duration > 0:
                      print(f"  Error: {check_event1_nan_duration} rows have event=1 but NaN in duration column '{duration_col_orig}'. Check data generation. Skipping.")
                      continue

            elif scenario['event_type'] == 'dropout':
                 # Duration is already set (e.g., total_followup_days)
                 df_model_subset_initial[duration_for_model_col] = df_model_subset_initial[duration_col_orig]

            # Now, define columns to keep for the final model subset
            final_model_cols = [duration_for_model_col, event_col] + predictor_cols

            # Drop rows ONLY if event or predictors are NaN (duration_for_model_col should be populated now)
            cols_to_check_for_na = [event_col] + predictor_cols
            # Also check the final duration column for NaNs, just in case
            if duration_for_model_col not in cols_to_check_for_na:
                 cols_to_check_for_na.append(duration_for_model_col)
            df_model_subset = df_model_subset_initial[final_model_cols].dropna(subset=cols_to_check_for_na)

            # --- MODIFICATION END ---


            # Check if duration is always positive and event is 0/1 AFTER dropping NaNs
            # --- MODIFIED: Check the new duration column and require > 0 ---
            if duration_for_model_col in df_model_subset and (df_model_subset[duration_for_model_col] <= 0).any():
                 # Cox model requires positive duration
                 print(f"  Warning: Non-positive values found in effective duration column '{duration_for_model_col}' after dropping NaNs. Removing these rows.")
                 original_rows = df_model_subset.shape[0]
                 df_model_subset = df_model_subset[df_model_subset[duration_for_model_col] > 0]
                 print(f"    Removed {original_rows - df_model_subset.shape[0]} rows with non-positive duration.")


            if event_col in df_model_subset and not df_model_subset[event_col].isin([0, 1]).all():
                 print(f"  Error: Event column '{event_col}' contains values other than 0 or 1 after dropping NaNs. Skipping.")
                 continue


            # Run the Cox model
            # --- MODIFIED: Pass the new duration column name ---
            model_output = run_cox_model(
                df_model_subset,
                duration_for_model_col, # Use the corrected duration column
                event_col,
                predictor_cols, # Pass the original list of predictors/covariates
                scenario_name,
                ASSUMPTION_PLOT_DIR
            )

            # Collect results if model ran successfully
            if model_output and 'summary' in model_output:
                # Extract key info from the model summary DataFrame for aggregation
                summary_df = model_output['summary']
                # --- MODIFIED: Safer extraction using .get() with defaults ---
                for _, row in summary_df.iterrows():
                    variable_name = row.get('Variable', 'UNKNOWN')
                    all_results_summary.append({
                        'scenario_name': scenario_name,
                        'variable': variable_name,
                        'hazard_ratio': row.get('exp(coef)'),
                        'ci_lower': row.get('exp(coef) lower 95%'),
                        'ci_upper': row.get('exp(coef) upper 95%'),
                        'p_value': row.get('p'),
                        'z_score': row.get('z'), # Added z-score
                        # 'log_likelihood_ratio_p': row.get('log(p)'), # This is usually the p-value for the variable, not LRT
                        'concordance': model_output.get('concordance'),
                        'n_subjects': model_output.get('n_subjects'),
                        'n_events': model_output.get('n_events'),
                        'log_likelihood': model_output.get('log_likelihood'), # Overall model log-likelihood
                        'ph_assumption_violated': model_output.get('ph_assumption_violated'),
                        'ph_violated_variables': model_output.get('ph_violated_variables'),
                        # Add specific assumption p-values if needed
                        f"{variable_name}_ph_p_value": model_output.get('assumption_summary', {}).get(f"{variable_name}_ph_p_value")
                    })
            else:
                 print(f"  Scenario {scenario_name} did not produce valid results.")


        # 4. Save Aggregated Results
        if all_results_summary:
            print("\n--- Aggregating and Saving Results ---")
            results_df = pd.DataFrame(all_results_summary)

            conn_out = None
            try:
                conn_out = sqlite3.connect(DB_PATH) # Connect to the main DB to save results
                print(f"Saving aggregated results to table: {OUTPUT_TABLE_NAME}")
                results_df.to_sql(OUTPUT_TABLE_NAME, conn_out, if_exists='replace', index=False)
                conn_out.commit()
                print("Results saved successfully.")
            except sqlite3.Error as e:
                print(f"SQLite error saving results: {e}")
                if conn_out: conn_out.rollback()
            except Exception as e:
                print(f"An error occurred saving results: {e}")
                if conn_out: conn_out.rollback()
            finally:
                if conn_out:
                    conn_out.close()
        else:
            print("\nNo valid model results were generated to save.")

    except FileNotFoundError as e:
        print(f"ERROR: {e}")
    except sqlite3.Error as e:
        print(f"ERROR: Database error during setup - {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred in the main pipeline - {e}")
        import traceback
        print(traceback.format_exc())
    finally:
        print("========== Cox Regression Pipeline Finished ==========")

# --- Execute the main function ---
# Ensure this cell is run within the notebook context where 'paper1_directory' is defined
if __name__ == "__main__":
     # Check again if running as script vs notebook cell
     if 'paper1_directory' not in locals() and 'paper1_directory' not in globals():
          print("ERROR: 'paper1_directory' variable not defined. Cannot run main().")
          # Or define a default path if running as a standalone script
          # paper1_directory = '.' # Example: Use current directory
     else:
          main()

========== Starting Cox Regression Pipeline ==========
Loading data from C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.sqlite, table sa_input_table...
Loaded 1664 rows and 56 columns.
Preprocessing data for Cox models...
Preprocessing complete.

--- Processing Scenario: 60d_10p_hunger_unadjusted ---
  Running Cox model for scenario: 60d_10p_hunger_unadjusted
    Duration: 60d_10p_hunger_unadjusted_duration, Event: 10%_wl_achieved
    Predictors: ['hunger_yn']
    Data shape: (1664, 3)
    Checking proportional hazards assumption...
Proportional hazard assumption looks okay.
    Model fitting and assumption check complete.

--- Processing Scenario: 60d_10p_satiety_unadjusted ---
  Running Cox model for scenario: 60d_10p_satiety_unadjusted
    Duration: 60d_10p_satiety_unadjusted_duration, Event: 10%_wl_achieved
    Predictors: ['satiety_yn']
    Data shape: (1664, 3)
    Checking proportional hazards assumption...
Proportional hazard assumption looks o

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1664 total observations, 1118 right-censored observations>
         test_name = proportional_hazard_test

---
                         test_statistic      p  -log2(p)
age                km             12.63 <0.005     11.36
                   rank           13.79 <0.005     12.26
baseline_bmi       km              3.39   0.07      3.93
                   rank            2.93   0.09      3.52
baseline_weight_kg km              2.60   0.11      3.22
                   rank            2.20   0.14      2.85
height_m           km              3.97   0.05      4.43
                   rank            3.53   0.06      4.05
hunger_yn          km              0.32   0.57      0.80
                   rank            0.75   0.39      1.38
sex_f              km             40.27 <0.005     32.08
                   rank           43.70 <0.005     34.61



1. Variable 'age' failed the non-proportional test: p-value is 0.0002.

   Advice 1: the functional form of the variable 'age' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'age' using pd.cut, and then specify it in `strata=['age',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'sex_f' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['sex_f', ...]` in the call
in `.fit`. See documentation in link [E] below.

3. Variable 'height_m' failed the non-proportional test: p-value is 0.0462.

   Advice 1: the functional form of the variable 'height_m' might be incorr

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1664 total observations, 1118 right-censored observations>
         test_name = proportional_hazard_test

---
                         test_statistic      p  -log2(p)
age                km             12.86 <0.005     11.54
                   rank           13.99 <0.005     12.41
baseline_bmi       km              3.70   0.05      4.20
                   rank            3.25   0.07      3.80
baseline_weight_kg km              2.89   0.09      3.49
                   rank            2.50   0.11      3.13
height_m           km              4.32   0.04      4.73
                   rank            3.90   0.05      4.37
satiety_yn         km              0.00   0.97      0.05
                   rank            0.00   0.98      0.03
sex_f              km             40.26 <0.005     32.06
                   rank           43.54 <0.005     34.49



1. Variable 'age' failed the non-proportional test: p-value is 0.0002.

   Advice 1: the functional form of the variable 'age' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'age' using pd.cut, and then specify it in `strata=['age',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'sex_f' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['sex_f', ...]` in the call
in `.fit`. See documentation in link [E] below.

3. Variable 'baseline_bmi' failed the non-proportional test: p-value is 0.0543.

   Advice 1: the functional form of the variable 'baseline_bmi' might b

...



1. Variable 'age' failed the non-proportional test: p-value is 0.0002.

   Advice 1: the functional form of the variable 'age' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'age' using pd.cut, and then specify it in `strata=['age',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'sex_f' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['sex_f', ...]` in the call
in `.fit`. See documentation in link [E] below.

3. Variable 'height_m' failed the non-proportional test: p-value is 0.0453.

   Advice 1: the functional form of the variable 'height_m' might be incorr

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1664 total observations, 1118 right-censored observations>
         test_name = proportional_hazard_test

---
                              test_statistic      p  -log2(p)
age                     km             11.69 <0.005     10.64
                        rank           12.84 <0.005     11.52
baseline_bmi            km              3.66   0.06      4.16
                        rank            3.22   0.07      3.78
baseline_weight_kg      km              2.81   0.09      3.41
                        rank            2.43   0.12      3.07
height_m                km              4.26   0.04      4.68
                        rank            3.86   0.05      4.34
quantity_control_likert km              0.32   0.57      0.81
                        rank            0.41   0.52      0.94
sex_f                   km             40.31 <0.005     32.10
                        rank           43.57 <0.005     34.51



1. Variable 'age' failed the non-proportional test: p-value is 0.0003.

   Advice 1: the functional form of the variable 'age' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'age' using pd.cut, and then specify it in `strata=['age',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'sex_f' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['sex_f', ...]` in the call
in `.fit`. See documentation in link [E] below.

3. Variable 'height_m' failed the non-proportional test: p-value is 0.0390.

   Advice 1: the functional form of the variable 'height_m' might be incorr

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1664 total observations, 1118 right-censored observations>
         test_name = proportional_hazard_test

---
                             test_statistic      p  -log2(p)
age                    km             12.44 <0.005     11.22
                       rank           13.52 <0.005     12.05
baseline_bmi           km              3.38   0.07      3.92
                       rank            2.91   0.09      3.51
baseline_weight_kg     km              2.59   0.11      3.22
                       rank            2.19   0.14      2.84
height_m               km              3.96   0.05      4.43
                       rank            3.52   0.06      4.04
impulse_control_likert km              1.44   0.23      2.11
                       rank            1.64   0.20      2.32
sex_f                  km             40.88 <0.005     32.53
                       rank           44.23 <0.005     34.99



1. Variable 'age' failed the non-proportional test: p-value is 0.0002.

   Advice 1: the functional form of the variable 'age' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'age' using pd.cut, and then specify it in `strata=['age',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'sex_f' failed the non-proportional test: p-value is <5e-05.

   Advice: with so few unique values (only 2), you can include `strata=['sex_f', ...]` in the call
in `.fit`. See documentation in link [E] below.

3. Variable 'height_m' failed the non-proportional test: p-value is 0.0465.

   Advice 1: the functional form of the variable 'height_m' might be incorr

#### Misc

In [4]:
import sqlite3

# Connect to the SQLite database
db_path = 'C:\\Users\\Felhasználó\\Desktop\\Projects\\PNK_DB2\\paper1_emotional\\survival_analysis.sqlite'
table_name = 'sa_input_table'

# Query to fetch unique values from the weight_gain_cause column
query = f"SELECT DISTINCT weight_gain_cause FROM {table_name}"

# Execute the query and fetch results
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute(query)
    results = cursor.fetchall()

# Extract and print the unique observations
unique_weight_gain_causes = [row[0] for row in results if row[0] is not None]
print("Unique observations in the weight_gain_cause column:")
for cause in unique_weight_gain_causes:
    print(cause)

Unique observations in the weight_gain_cause column:
comer mal, estress
EMBARAZO 
Ansiedad y estrés 
PICOTEA EN EL RESTAURANTE SOLO HACE 1 COMIDA AL DIA. 
Embarazo, falta motivación...
NO PODER MOVERSE POR DOLOR LUMBARES
ANSIEDAD POR PROBLEMAS LABORALES
DEPRESION
ROTURA DE MENISCO, DETUBO EJERCICIO Y NO SE HA SABIDO CONTROLAR + MUERTE DE SU PADRE
Estrés en el trabajo y horarios cambiantes 
EDAD, MENOS ACTIVIDAD FISICA
No se cuida con cantidades. 
Estrés y poco ejercicio
QUARENTENA
malos hábitos alimentarios
COMER MAL
PROBLEMAS PERSONALES, DESORDEN
ANSIEDAD, MURIO SU MADRE HACE UN AÑO
menopausia 
MUCHAS ANSIEDAD POR TEMAS PERSONALES
EMBARAZOS
TRABAJA EN ESTACION DE SKI
PICOTEO
DEPOIS DA PANDEMIA COMEÇOU A AUMENTAR PESO GRADUALMENTE. SEDENTARISMO.
JÁ FEZ LEV, DIETA 3 PASSOS, HERBALIFE, SEM RESULTADOS A MEDIO-LONGO PRAZO.
ESTAVA ACOMPANHADA (TRABALHO), ACHEI QUE NÃO ESTAVA CONFORTÁVEL NA PORMENORIZAÇÃO DESTE TEMA E NÃO O EXPLOREI MAIS.
desorden horario, come mas por las noches durante el 

In [5]:
import sqlite3
import pandas as pd
import os

# Define the output Excel file path
output_excel_path = os.path.join(paper1_directory, 'survival_analysis.xlsx')

# Connect to the SQLite database
with sqlite3.connect(DB_PATH) as conn:
    # Get the list of all tables in the database
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(query, conn)['name'].tolist()
    
    # Create a Pandas Excel writer
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        # Loop through each table and save it as a sheet in the Excel file
        for table in tables:
            df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
            df.to_excel(writer, sheet_name=table, index=False)

print(f"Database exported to Excel at: {output_excel_path}")

Database exported to Excel at: C:\Users\Felhasználó\Desktop\Projects\PNK_DB2\paper1_emotional\survival_analysis.xlsx
